
setwd('D:/ADRENAL_GLAND_PROJECT')
only_STERO=readRDS(file='D:/ADRENAL_GLAND_PROJECT/Only_Stero')
multiomics=readRDS(file='D:/ADRENAL_GLAND_PROJECT/multiomics')


# download package 

In [ ]:
library(S4Vectors)
library(Signac)
library(Seurat)
library(BSgenome.Mmusculus.UCSC.mm10)
library(AnnotationHub)
library(ggplot2)

# Read file

In [ ]:
inputdata.Female_12 <- Read10X_h5("D:/Stage/RAW DATA MATRIX/raw_feature_bc_matrix_female12.h5")     # REPLICAT 1
fragpath.female_12 <- "D:/Stage/RAW DATA MATRIX/atac_fragments_female12.tsv.gz"

inputdata.Female_13 <- Read10X_h5("D:/Stage/RAW DATA MATRIX/raw_feature_bc_matrix_female13.h5")     # REPLICAT 2 
fragpath.female_13 <- "D:/Stage/RAW DATA MATRIX/atac_fragments_female13.tsv.gz"

inputdata.male_16 <- Read10X_h5("D:/Stage/RAW DATA MATRIX/raw_feature_bc_matrix_male16.h5")         # REPLICAT 1 
fragpath.male_16 <- "D:/Stage/RAW DATA MATRIX/atac_fragments_male16.tsv.gz"
                
inputdata.male_17 <- Read10X_h5("D:/Stage/RAW DATA MATRIX/raw_feature_bc_matrix_male17.h5")         # REPLICAT 2
fragpath.male_17 <- "D:/Stage/RAW DATA MATRIX/atac_fragments_male17.tsv.gz"                 
                                                                             

# Get annotation file for peak mapping on reference genome 

In [ ]:
annotation <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
seqlevelsStyle(annotation) <- "UCSC"
genome(annotation) <- "mm10"

# Create RNA and ATAC seurat object ; add quality control metrics for RNA and ATAC assay 

In [ ]:

# after running,  default assay will be set to ATAC 

male1 <- CreateSeuratObject(counts = inputdata.male_16$`Gene Expression`,project = 'male1',min.features = 50) # remove cell with less than      50genes
male1$identity <- "male"
male1$Replicat <- "Replicat_2"
male1[["percent.mt"]] <- PercentageFeatureSet(male1, pattern = "^mt-")
male1[["percent.ribosomal"]] <- PercentageFeatureSet(male1, pattern = "^Rp[sl]")
male1[["ATAC"]] <- CreateChromatinAssay( counts = inputdata.male_16$Peaks,sep = c(":", "-"),fragments = fragpath.male_16,annotation = annotation,min.cells = 3,genome ="mm10")  
DefaultAssay(male1)='ATAC'
male1 <- NucleosomeSignal(male1)
male1 <- TSSEnrichment(male1)
male1$blacklist_fraction <- FractionCountsInRegion(object = male1, assay = 'ATAC',regions = blacklist_mm10) 
saveRDS(male1,file = 'male1_preQC')
  
      
male2 <- CreateSeuratObject(counts = inputdata.male_17$`Gene Expression`,project='male2',min.features = 50)
male2$identity <- "male"
male2$Replicat <- "Replicat_1"
male2[["percent.mt"]] <- PercentageFeatureSet(male2, pattern = "^mt-")
male2[["percent.ribosomal"]] <- PercentageFeatureSet(male2, pattern = "^Rp[sl]")
male2[["ATAC"]]<- CreateChromatinAssay( counts = inputdata.male_17$Peaks,sep = c(":", "-"),fragments = fragpath.male_17,annotation = annotation,min.cells = 3,genome ="mm10") 
DefaultAssay(male2)='ATAC'
male2 <- NucleosomeSignal(male2)
male2 <- TSSEnrichment(male2)
male2$blacklist_fraction <- FractionCountsInRegion(object = male2, assay = 'ATAC',regions = blacklist_mm10)
saveRDS(male2,file = 'male2_preQC')


female1=CreateSeuratObject(counts = inputdata.Female_12$`Gene Expression`,project='female1',min.features = 50)
female1$identity <- "female"
female1$Replicat <- "Replicat_1"
female1[["percent.mt"]] <- PercentageFeatureSet(female1, pattern = "^mt-")
female1[["percent.ribosomal"]] <- PercentageFeatureSet(female1, pattern = "^Rp[sl]")
female1[["ATAC"]] <- CreateChromatinAssay( counts = inputdata.Female_12$Peaks,sep = c(":", "-"),fragments = fragpath.female_12,annotation = annotation,min.cells = 3,genome ="mm10") 
DefaultAssay(female1)='ATAC'
female1 <- NucleosomeSignal(female1)
female1 <- TSSEnrichment(female1)
female1$blacklist_fraction <- FractionCountsInRegion(object = female1, assay = 'ATAC',regions = blacklist_mm10)
saveRDS(female1,file = 'female1_preQC')


female2=CreateSeuratObject(counts = inputdata.Female_13$`Gene Expression`,project = 'female2',min.features = 50)
female2$identity <- "female"
female2$Replicat <- "Replicat_2"
female2[["percent.mt"]] <- PercentageFeatureSet(female2, pattern = "^mt-")
female2[["percent.ribosomal"]] <- PercentageFeatureSet(female2, pattern = "^Rp[sl]")
female2[["ATAC"]] <- CreateChromatinAssay( counts = inputdata.Female_13$Peaks,sep = c(":", "-"),fragments = fragpath.female_13,annotation = annotation,min.cells = 3,genome ="mm10" ) 
DefaultAssay(female2)='ATAC'
female2 <- NucleosomeSignal(female2)
female2 <- TSSEnrichment(female2)
female2$blacklist_fraction <- FractionCountsInRegion(object = female2, assay = 'ATAC',regions = blacklist_mm10)
saveRDS(female2,file = 'female2_preQC')



# Metrics visualisation and subset
## Male 1 

In [ ]:

metadata_male1=male1@meta.data

metadata_male1=subset(male1@meta.data, subset = nCount_RNA >50& nCount_RNA < 25000 & nCount_ATAC >5000 & nCount_ATAC <100000 &  TSS.enrichment > 4 & percent.mt< 0.50& nucleosome_signal<0.7)


plotRank.sample <- subset(metadata_male1, cells = WhichCells(metadata_male1))
plot(sort(plotRank.sample$nCount_RNA ), xlab='cell', log='y', main= paste('count per cell -mito(ordered) - '))

metadata_male1 %>% 
  	ggplot(aes(color=identity, x=nFeature_RNA ,fill=identity)) + 
  	geom_density(alpha = 0.2) + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density")  # +
   geom_vline(xintercept = 10)

metadata_male1 %>% 
  	ggplot(aes(x=nCount_RNA, y=nFeature_RNA, color=nCount_RNA)) + 
  	geom_point() + 
	scale_colour_gradient(low = "gray90", high = "black") +
  	scale_x_log10() + 
  	scale_y_log10() + 
  	theme_classic() +
  	facet_wrap(~identity)


test1 <- subset(male1,subset = nFeature_RNA >500 & nCount_RNA < 25000 & nCount_ATAC >5000 & nCount_ATAC <100000 &  TSS.enrichment > 4 & percent.mt< 0.50 & nucleosome_signal<0.7)

VlnPlot(object = test1,features =c( 'nCount_ATAC','nCount_RNA','nFeature_ATAC','nFeature_RNA','percent.mt','TSS.enrichment','nucleosome_signal'))
plot1 <- FeatureScatter(test1, feature1 = "percent.ribosomal", feature2 = "nCount_RNA",group.by = "identity")
plot2 <- FeatureScatter(test1, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by = "Replicat",raster = F)
plot3 <- FeatureScatter(test1, feature1= 'nCount_ATAC' , feature2 = 'nFeature_ATAC' , group.by = 'Replicat')
plot4 <- FeatureScatter(test1, feature1= 'TSS.enrichment' , feature2 = 'TSS.percentile' , group.by = 'Replicat')
plot1 + plot2 + plot3  +plot4

male1 <- subset(male1,subset = nFeature_RNA >500 & nCount_RNA < 25000 & nCount_ATAC >5000 & nCount_ATAC <100000 &  TSS.enrichment > 4 & percent.mt< 0.50 & nucleosome_signal<0.7)


## Male 2

In [ ]:

metadata_male2=subset(male2@meta.data , subset = nFeature_RNA > 500 & nCount_RNA <35000& percent.mt< 0.75 &nCount_ATAC>5000 &nCount_ATAC <100000 & TSS.enrichment>3)
                      
plotRank.sample <- subset(metadata_male2, cells = WhichCells(metadata_male2))
plot(sort(plotRank.sample$nCount_RNA), xlab='cell', log='y', main= paste('count per cell -mito(ordered) - '))

metadata_male2 %>% 
  	ggplot(aes(color=identity, x=nucleosome_signal, fill=identity)) + 
  	geom_density(alpha = 0.2) + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density") #+
 # geom_vline(xintercept = 5000)

metadata_male2 %>% 
  	ggplot(aes(x=nCount_ATAC, y=nFeature_ATAC, color=percent.mt)) + 
  	geom_point() + 
	scale_colour_gradient(low = "gray90", high = "black") +
  	stat_smooth(method=lm) +
  	scale_x_log10() + 
  	scale_y_log10() + 
  	theme_classic() +
  	geom_vline(xintercept = 500) +
  	geom_hline(yintercept = 250) +
  	facet_wrap(~identity)


test2 <- subset(male2, subset = nFeature_RNA > 500 & nCount_RNA <35000 & percent.mt< 0.75 &nCount_ATAC>5000 &nCount_ATAC <100000 & TSS.enrichment>3)

VlnPlot(object = test2,features =c( 'nCount_ATAC','nCount_RNA','nFeature_ATAC','nFeature_RNA','percent.mt','TSS.enrichment','nucleosome_signal'))
plot1 <- FeatureScatter(test2, feature1 = "percent.ribosomal", feature2 = "nCount_RNA",group.by = "identity")
plot2 <- FeatureScatter(test2, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by = "Replicat",raster = F)
plot3 <- FeatureScatter(test2, feature1= 'nCount_ATAC' , feature2 = 'nFeature_ATAC' , group.by = 'Replicat')
plot4 <- FeatureScatter(test2, feature1= 'TSS.enrichment' , feature2 = 'TSS.percentile' , group.by = 'Replicat')
plot1 + plot2 + plot3  +plot4


male2 <- subset(male2, subset = nFeature_RNA > 500 & nCount_RNA <35000 & percent.mt< 0.75 &nCount_ATAC>5000 &nCount_ATAC <100000 & TSS.enrichment>3)

## Female 1 

In [ ]:
                                                     

metadata_female1=subset(female1@meta.data, subset = nCount_RNA > 700 &nCount_RNA < 50000 & nCount_ATAC> 7000 & nCount_ATAC< 300000 &TSS.enrichment>3 & percent.mt< 0.75&nucleosome_signal<0.7)

plotRank.sample <- subset(metadata_female1, cells = WhichCells(metadata_female1))
plot(sort(plotRank.sample$nCount_RNA), xlab='cell', log='y', main= paste('count per cell -mito(ordered) - '))

                        

metadata_female1 %>% 
  	ggplot(aes(color=identity, x=nCount_RNA, fill=identity)) + 
  	geom_density(alpha = 0.2) + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density") #+
 # geom_vline(xintercept = 5000)

metadata_female1 %>% 
  	ggplot(aes(x=nCount_ATAC, y=nFeature_ATAC, color=percent.mt)) + 
  	geom_point() + 
	scale_colour_gradient(low = "gray90", high = "black") +
  	stat_smooth(method=lm) +
  	scale_x_log10() + 
  	scale_y_log10() + 
  	theme_classic() +
  	geom_vline(xintercept = 500) +
  	geom_hline(yintercept = 250) +
  	facet_wrap(~identity)


test3 <- subset(female1, subset = nCount_RNA > 700 &nCount_RNA < 50000 & nCount_ATAC> 7000 & nCount_ATAC< 300000 &TSS.enrichment>3 & percent.mt< 0.75&nucleosome_signal<0.7)

VlnPlot(object = test3,features =c( 'nCount_ATAC','nCount_RNA','nFeature_ATAC','nFeature_RNA','percent.mt','TSS.enrichment','nucleosome_signal'))
plot1 <- FeatureScatter(test3, feature1 = "percent.ribosomal", feature2 = "nCount_RNA",group.by = "identity")
plot2 <- FeatureScatter(test3, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by = "Replicat",raster = F)
plot3 <- FeatureScatter(test3, feature1= 'nCount_ATAC' , feature2 = 'nFeature_ATAC' , group.by = 'Replicat')
plot4 <- FeatureScatter(test3, feature1= 'TSS.enrichment' , feature2 = 'TSS.percentile' , group.by = 'Replicat')
plot1 + plot2 + plot3  +plot4

female1 <- subset(female1, subset = nCount_RNA > 700 &nCount_RNA < 50000 & nCount_ATAC> 7000 & nCount_ATAC< 300000 &TSS.enrichment>3 & percent.mt< 0.75&nucleosome_signal<0.7)




## Female 2 

In [ ]:

metadata_female2=subset(female2@meta.data,subset = nFeature_RNA > 500 & nCount_RNA<100000 & nCount_ATAC>5000 & nCount_ATAC <500000& TSS.enrichment>5 &percent.mt<0.5&nucleosome_signal<0.7)


plotRank.sample <- subset(metadata_female2, cells = WhichCells(metadata_female2))
plot(sort(plotRank.sample$nCount_RNA), xlab='cell', log='y', main= paste('count per cell -mito(ordered) - '))

metadata_female2 %>% 
  	ggplot(aes(color=identity, x=nCount_RNA, fill=identity)) + 
  	geom_density(alpha = 0.2) + 
  	scale_x_log10() + 
  	theme_classic() +
  	ylab("Cell density") #+
 # geom_vline(xintercept = 5000)

metadata_female2 %>% 
  	ggplot(aes(x=nCount_ATAC, y=nFeature_ATAC, color=percent.mt)) + 
  	geom_point() + 
	scale_colour_gradient(low = "gray90", high = "black") +
  	stat_smooth(method=lm) +
  	scale_x_log10() + 
  	scale_y_log10() + 
  	theme_classic() +
  	geom_vline(xintercept = 500) +
  	geom_hline(yintercept = 250) +
  	facet_wrap(~identity)


# total fragment
totals <- CountFragments(fragments = path.to.fragfile)
hist(log10(totals$frequency_count), breaks = 100)
cells.use <- totals[totals$frequency_count > 1000, "CB"]
counts <- FeatureMatrix(fragments = fragment.object, features = peaks, cells = cells.use)



test4 <- subset(female2, subset = nFeature_RNA > 500 & nCount_RNA < 15000&nCount_ATAC >5000& nCount_ATAC <100000 & TSS.enrichment > 3&nucleosome_signal<0.7)

VlnPlot(object = test4,features =c( 'nCount_ATAC','nCount_RNA','nFeature_ATAC','nFeature_RNA','percent.mt','TSS.enrichment','nucleosome_signal'))
plot1 <- FeatureScatter(test4, feature1 = "percent.ribosomal", feature2 = "nCount_RNA",group.by = "identity")
plot2 <- FeatureScatter(test4, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by = "Replicat",raster = F)
plot3 <- FeatureScatter(test4, feature1= 'nCount_ATAC' , feature2 = 'nFeature_ATAC' , group.by = 'Replicat')
plot4 <- FeatureScatter(test4, feature1= 'TSS.enrichment' , feature2 = 'TSS.percentile' , group.by = 'Replicat')
plot1 + plot2 + plot3  +plot4

female2 <- subset(female2, subset = nFeature_RNA > 500 & nCount_RNA < 15000&nCount_ATAC >5000& nCount_ATAC <100000 & TSS.enrichment > 3&nucleosome_signal<0.7)



# merge all 4 files 

In [ ]:

multiomics=merge(x = male1,y = c(male2,female1,female2),add.cell.ids =c( 'male1','male2','female1','female2'),project = "multiomics",merge.data = T)
saveRDS(data_filt_merged)


# Quick overview

In [ ]:
multiomics=readRDS(file = '/home/absta/Documents/DATA_FINAL_STAGE/multiomics_int')
DefaultAssay(multiomics)='RNA'
multiomics[['SCT']]=NULL
multiomics[['peaks']]=NULL
#multiomics <- multiomics[,!colnames(multiomics) %in% subset_endo] 
#multiomics <- multiomics[,!colnames(multiomics) %in% subset_medulla]
#multiomics <- multiomics[,!colnames(multiomics) %in% ENDO2]
multiomics <- multiomics[,!colnames(multiomics) %in% subset] # process by first running to the clustering and highlinghting this cells


multiomics@meta.data[15:27]=NULL
multiomics=subset(multiomics,subset  = nCount_RNA<20000)
multiomics@reductions[["umap.rna.pca"]]=NULL
multiomics@reductions[["umap.rna_ica"]]=NULL
multiomics@reductions[["umap.atac"]]=NULL
multiomics@reductions[["wnn.umap"]]=NULL

VlnPlot(multiomics,features = 'nCount_RNA')

# FULL dataset analysis ( Multiomics )

## RNA PROCESSING 

In [ ]:
#  Normalisation , scaling, PCA, ICA , UMAP and clustering


# 0) Replicat integration  
      
      DefaultAssay(multiomics)='RNA'  # take RNA assay 
      multiomics <- SplitObject(multiomics, split.by = "Replicat") # list 
      for (i in 1:length(multiomics)) {
          multiomics[[i]] <- SCTransform(multiomics[[i]],variable.features.n =NULL,variable.features.rv.th =1.1
                                         ,return.only.var.genes = FALSE,assay = 'RNA',
                                         new.assay.name = 'SCT_INT',vars.to.regress = 'percent.mt')
          }
      
      
      # Select the most variable features to use for integration
      integ_features <- SelectIntegrationFeatures(object.list = multiomics, nfeatures = 3000) 
      
      # Prepare the SCT list object for integration
      multiomics  <- PrepSCTIntegration(object.list = multiomics, 
                                         anchor.features =integ_features)
      
      
      integ_anchors <- FindIntegrationAnchors(object.list = multiomics, 
                                              normalization.method = "SCT", 
                                              anchor.features = integ_features,reduction="cca")
      
      # Integrate across conditions
      multiomics <- IntegrateData(anchorset = integ_anchors,normalization.method = "SCT")    
      
     

      
### 1)  Normalisation / PC / IC / UMAP /CLUSTERING on integrated and non integrated assay
      
    # 1.1) NORMALISATION AND PC ON NON ONTEGRATED assay       
          multiomics=SCTransform(multiomics,variable.features.n =NULL,variable.features.rv.th =1.1,return.only.var.genes = FALSE,assay = 'RNA',new.assay.name = 'SCT_NO_INT',vars.to.regress = 'percent.mt')
          multiomics=RunPCA(multiomics, verbose = FALSE,assay="SCT_NO_INT",reduction.name='PCA_NO_INT') # Default 50 
          multiomics=RunICA(multiomics, verbose = FALSE,assay="SCT_NO_INT",reduction.name='ICA_NO_INT')
          ElbowPlot(multiomics,ndims = 50,reduction = 'PCA_NO_INT') # Variance plot 
      
    # 1.2) ON INTEGRATED  assay 
          DefaultAssay(multiomics)='integrated'
          multiomics=RunPCA(multiomics, verbose = FALSE,assay="integrated",reduction.name='PCA_INT',npcs = 50)
          multiomics=RunICA(multiomics, verbose = FALSE,assay="integrated",reduction.name='ICA_INT')

          ElbowPlot(multiomics,reduction ='PCA_INT',ndims = 50 ) # choose 15
      
    # 1.3)  Get the total variance explained for each PC:
        
        total.variance   <- function(x,y) { 
              mat <- Seurat::GetAssayData(multiomics, assay = x, slot = "scale.data")
              pca <- multiomics[[y]]
              
              # Get the total variance:
              total_variance <- sum(matrixStats::rowVars(mat))
              eigValues = (pca@stdev)^2  ## EigenValues
              varExplained = eigValues / total_variance
              barplot(varExplained)
              print(varExplained)
        }
        
        total.variance('integrated','PCA_INT')
        
        

        
    # 1.4) Tcheck Variabilty explained by MITOCHONDRIAL genes 
        summary(multiomics@meta.data$percent.mt)
        # Turn mitoRatio into categorical factor vector based on quartile values
        percent.mt_pbmc_QC=multiomics # create new pbmc object for inserting  labels 
        percent.mt_pbmc_QC@meta.data$percent.mt<- cut(multiomics@meta.data$percent.mt, 
                           breaks=c(-Inf, 0, 0.03814   , 0.08238   , Inf), 
                           labels=c("Low","Medium","Medium high", "High"))
        # Plot the PCA colored by mitoFr
        DimPlot(percent.mt_pbmc_QC,reduction = "PCA_NO_INT",group.by= "percent.mt", split.by = "percent.mt")  
        # View all PC 
        runfunction <- function(z,x,y){DimPlot(object = z, dims = c(x, y), reduction = "PCA_NO_INT",group.by   = "percent.mt") 
        }
        for ( i in (1:15)){x=runfunction(percent.mt_pbmc_QC,i+1,i)  # plot 15 dims
                                     print(x)}
  

        
### 2 ) UMAP REPRESENTATION and clustering       
              
    # 2.1) INTEGRATED ASSAY
      multiomics=RunUMAP(multiomics,reduction='PCA_INT',dims = 1:20,reduction.name = 'UMAP_PCA_INT')  # UMAP on PC 
      multiomics=RunUMAP(multiomics,dims = 1:20, local.connectivity  = 6 ,reduction = 'ICA_INT',umap.method = "umap-learn",metric = "correlation",n.neighbors =200,reduction.name = 'umap.rna_ica',reduction.key='icaUMAP_') # UMAP on IC
      
    # 2.2) NON INTEGRATED ASSAY
      multiomics=RunUMAP(multiomics,reduction='PCA_NO_INT' ,dims = 1:20,reduction.name = 'UMAP_PCA_NO_INT')  # UMAP on PC 
      multiomics=RunUMAP(multiomics,dims = 1:20, local.connectivity  = 6 ,reduction = 'ICA_NO_INT',umap.method = "umap-learn",metric = "correlation",n.neighbors =200,reduction.name = 'UMAP_ICA_NO_INT',reduction.key='icaUMAP_') # UMAP on IC

  
    # 2.3) Neighbors and Clustering on non integrated pc
      multiomics=FindNeighbors(multiomics,reduction = 'UMAP_PCA_NO_INT',dims = 1:30) # take the right reduc 
      multiomics=FindClusters(multiomics, resolution = seq(from = 0.1, to = 1, by = 0.1),algorithm = 'leiden')# store cluster resolution in metadata slot

          # Test a range of cluster resolution 
                ResolutionList <- grep("_snn_res", colnames(multiomics@meta.data), value = TRUE)
                for (Resolution in ResolutionList){
                    #pdf(paste0(Resolution, "_UMAP.pdf"), width=7, height=7)
                    g <- DimPlot(object = multiomics, reduction = "umap",label=T,group.by = Resolution  )
                    print(g)
                    # dev.off()
                }
          
       


## Found variable genes and name clusters 

In [ ]:

# Find markers genes of clusters
all_markers=FindAllMarkers(object = multiomics,assay ='SCT',logfc.threshold = 0.1)


# Featureplot known markers genes 
FeaturePlot(multiomics, features=c('Wnt4','Dab2'),reduction = 'UMAP_PCA_INT')                                   # glomerulosa markers gene 
FeaturePlot(multiomics, features=c('Akr1b7','Cyp11b1'),reduction = 'UMAP_PCA_INT')                              # Fasciculata markers gene 
FeaturePlot(multiomics, features=c('Th','Slc18a1','Slc18a2','Ddc','Dbh','Lgr5'),reduction = 'UMAP_PCA_INT')     # medulla markers gene 
FeaturePlot(multiomics, features=c('Star','Cyp11a1', 'Hsd3b1', 'Scarb1'),reduction = 'UMAP_PCA_INT')                # cortex markers gene 
FeaturePlot(multiomics, features=c("Rspo3",'Gli1'),reduction = 'UMAP_PCA_INT')                                  # capsular markers gene 
FeaturePlot(multiomics, features=c('Akr1c18'),reduction = 'UMAP_PCA_INT')                                       # x zone
FeaturePlot(multiomics, features=c('Cdh5','Pecam1','Flt1'),reduction = 'UMAP_PCA_INT')                          # endothelial  markers gene 
FeaturePlot(multiomics, features=c('Star','Cyp11a1', 'Hsd3b1', 'Scarb1'),reduction = 'UMAP_PCA_INT')            # cortex markers gene 
FeaturePlot(multiomics, features=c('Ptprc'),reduction = 'UMAP_PCA_INT')                                         # C immutnitaire  markers gene
FeaturePlot(multiomics, features=c("Ano1","Col3a1","Cdh11","Dcn","Rspo3"),reduction = 'UMAP_PCA_INT')               # stromal cells 



Idents(multiomics)=multiomics@meta.data$Initial_clustering      # Save Old idents 
multiomics <- RenameIdents(multiomics,`Endothelial Cells`="Endothelial Cells",
                     `Male specific Medulla`='Male Medulla',
                     `Male Fasciculata Replicate 1`='Male Fasciculata',
                     `Male Fasciculata Replicate 2`='Male Fasciculata',
                     `Female Fasiculata Replicate 2`='Female Fasciculata',
                     `Female Glomerulosa`='Female Glomerulosa',
                     `Female Fasciculate Replicate 1`='Female Fasciculata',
                     `Male Glomerulosa`='Male Glomerulosa',
                     `Female specific Medulla`="Male Fasciculata",
                     `Medulla progenitor Cells`="Female Medulla",
                     `Capsular Cells`='Medulla progenitor Cells',
                     `Immune Cells`='Capsular Cells',
                     `Fetal adrenocortical Cells`='Immune Cells',
                     `X zone`='Fetal adrenocortical Cells')

Idents(multiomics)=multiomics$Cell_types # Store clustering in metadata slot


# check cell number
table(Idents(multiomics))


## Variable genes remove non interesing genes

In [ ]:
  DefaultAssay(multiomics)='SCT' 
  remove_multiomics=multiomics
  remove_multiomics[['peaks']] <- NULL
  counts <- GetAssayData(remove_multiomics, assay = "SCT")
  counts <- counts[-(which(rownames(counts) %in% c('Xist','Tsix','Uty','Ddx3y'))),]
  remove_multiomics <- subset(remove_multiomics, features = rownames(counts))
  
# male vs Female | fasciculata replicate
multiomics_replicat=FindMarkers(remove_multiomics,assay = 'SCT',logfc.threshold = 0.1,ident.1 =c("Male Fasciculata Replicate 2","Male Fasciculata Replicate 1"),ident.2 = c( "Female Fasciculata Replicate 1", "Female Fasiculata Replicate 2"))
multiomics_replicat=multiomics_replicat[multiomics_replicat$p_val_adj<0.05,]
write.csv(multiomics_replicat,file = 'D:/ADRENAL_GLAND_PROJECT/FASCICULATA/replicate/diff_male_pos_female_neg_fasc.csv')


# male vs male | fasciculata replicate
diff_MALE_fasc=FindMarkers(remove_multiomics,assay = 'SCT',logfc.threshold = 0.1,ident.1 =c("Male Fasciculata Replicate 2"),ident.2 = c("Male Fasciculata Replicate 1"))
diff_MALE_fasc=diff_MALE_fasc[diff_MALE_fasc$p_val_adj<0.05,]
write.csv(diff_MALE_fasc,file = 'D:/ADRENAL_GLAND_PROJECT/FASCICULATA/replicate/diff_male_2_male_1_fasc.csv')


# Female vs male | fasciculata replicate
diff_FEMALE_fasc=FindMarkers(remove_multiomics,assay = 'SCT',logfc.threshold = 0.1,ident.1 =c("Female Fasiculata Replicate 2"),ident.2 = c("Female Fasciculata Replicate 1"))
diff_FEMALE_fasc=diff_FEMALE_fasc[diff_FEMALE_fasc$p_val_adj<0.05,]
write.csv(diff_FEMALE_fasc,file = 'D:/ADRENAL_GLAND_PROJECT/FASCICULATA/Replicate/diff_female_2_female_1_fasc.csv')


## ATAC peaks calling; normalisation; UMAP representation

In [ ]:
# From https://satijalab.org/signac/articles/pbmc_multiomic.html
library(EnsDb.Mmusculus.v79)
library(BSgenome.Mmusculus.UCSC.mm10)
  
#1) Peak calling 
      DefaultAssay(multiomics)='ATAC'
      run_macs<- CallPeaks(object = multiomics,macs2.path = '/home/absta/.conda/envs/envpython/bin/macs2',effective.genome.size = 1.87e9 ,group.by = "Cell_types",assay = 'ATAC') 

      #get gene annotations for hg38
      annotation <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
      seqlevelsStyle(annotation) <- "UCSC"


      #remove peaks on nonstandard chromosomes and in genomic blacklist regions
      peaks <- keepStandardChromosomes(run_macs, pruning.mode = "coarse",species ='Mus_musculus' )
      peaks <- subsetByOverlaps(x = peaks, ranges = blacklist_mm10, invert = TRUE)

      #quantify counts in each peak
      macs2_counts <- FeatureMatrix(
          fragments = Fragments(multiomics),
          features = peaks,
          cells = colnames(multiomics)
        )


#2) create a new assay (peaks) using the MACS2 peak set and add it to the Seurat object
      multiomics[["peaks"]] <- CreateChromatinAssay(
        counts = macs2_counts,
        fragments = Fragments(multiomics),
        annotation = annotation
      )
      

#3) ATAC Normalisation UMAP representation 
    # Normalisation
    DefaultAssay(multiomics) <- "peaks"
    multiomics <- FindTopFeatures(multiomics, min.cutoff = 5) # Find top features based on total number of counts for the feature
    multiomics <- RunTFIDF(multiomics) # Run term frequency inverse document frequency (TF-IDF) normalization on a matrix.
    multiomics <- RunSVD(multiomics) # Run partial singular value decomposition using irlba
    DepthCor(multiomics)   # correlation between sequencing depth and first component 

multiomics <- RunUMAP(object = multiomics,reduction = 'lsi',dims = 2:50,reduction.name = 'lsi_umap') # UMAP visualization with RNA based idents as clusters
#multiomics <- FindNeighbors(object = multiomics,reduction = 'lsi',dims = 2:50) # use SNN for findclusters 
#multiomics <- FindClusters(object = multiomics,algorithm = 1,resolution = 0.7,verbose = FALSE)

DimPlot(object = multiomics, label = TRUE)+DimPlot(multiomics,reduction = 'UMAP_PCA_NO_INT')


#4) INTEGRATION ON REPLICAT  for atac 

      Replicat=multiomics # create new variable
      Replicat <- SplitObject(Replicat, split.by = "Replicat")  # Split Object by replicat (create list with 2 variables )

      # Find top 3000 conserved peaks features between both variable 
      x=SelectIntegrationFeatures(               
        Replicat,
        nfeatures = 3000,
        assay = c('peaks','peaks'),
        verbose = TRUE
      )

      # find integration anchors ( common features)
      integration.anchors <- FindIntegrationAnchors(
        object.list = Replicat,
        anchor.features =x,
        reduction = "rlsi",
        dims = 2:30
      )
      y=multiomics 

      # integrate LSI embeddings
      integrated <- IntegrateEmbeddings(
        anchorset = integration.anchors,
        reductions = y[["lsi"]],
        new.reduction.name = "integrated_lsi",
        dims.to.integrate = 2:30
      )

    integrated <- RunUMAP(integrated, reduction = "integrated_lsi", dims = 2:30)
    DimPlot(integrated ,group.by = 'Replicat')
       
 
    # ADD INT_LSI into mainn object OBJECT 

    multiomics@reductions$UMAP_LSI_ATAC_INT=integrated@reductions[["umap"]] # add UMAP LSI int into main object 
    multiomics@reductions$integrated_lsi=integrated@reductions[["integrated_lsi"]] # Add LSI int into main object 

    # Visualisation 
    DimPlot(object = multiomics, label = TRUE,group.by = 'Replicat',reduction = 'lsi_umap')+DimPlot(object = multiomics, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC_INT')




## WNN UMAP representation

In [ ]:

# 1) Joint UMAP visualization
ElbowPlot(object, ndims = 20, reduction = "integrated_lsi")
ElbowPlot(object, ndims = 20, reduction = "PCA_INT")


multiomics <- FindMultiModalNeighbors(object = multiomics,reduction.list = list("PCA_INT", "integrated_lsi"), dims.list = list(1:20,2:40),modality.weight.name = "weighted.nn",verbose = TRUE) # Create weighted nearest neighbors 
multiomics <- RunUMAP(object = multiomics,nn.name = "weighted.nn",assay = "SCT",verbose = TRUE,reduction.name = 'wnn.umap')

DimPlot(object = multiomics, label = TRUE,reduction='wnn.umap')
DimPlot(object = multiomics, label = TRUE,reduction='UMAP_PCA_NO_INT')
DimPlot(object = multiomics, label = TRUE,reduction='UMAP_LSI_ATAC_INT')


# 2) Findclusters on new representation 
multiomics=FindClusters(only_STERO, resolution = seq(from = 0.1, to = 2, by = 0.1),algorithm = 'leiden',graph.name = 'wsnn') # store cluster resolution in metadata slot
  
  # Test a range of cluster resolution 
ResolutionList <- grep("wsnn_res", colnames(multiomics@meta.data), value = TRUE)
for (Resolution in ResolutionList){
    #pdf(paste0(Resolution, "_UMAP.pdf"), width=7, height=7)
    g <- DimPlot(object = multiomics, reduction = "wnn.umap",label=T,group.by = Resolution  )
    DefaultAssay(multiomics)='SCT_RUN_STERO'
    assign(paste0('ALLMARKERS_',ResolutionList[Resolution]),FindAllMarkers(multiomics,assay ='SCT_RUN_STERO',logfc.threshold = 0.5))
    print(g)
    # dev.off()
}


DefaultAssay(multiomics)='SCT'
FeaturePlot(multiomics, features = c("Ptprc","Rspo3","Th","Pecam1","Wnt4","Akr1b7","Akr1c18",'Pnmt'),reduction = 'wnn.umap')  # gene markers by yasmine 

# Save 
 # saveRDS(multiomics,file = 'MULTIOMICS_FINAL')
 # multiomics=readRDS(file='D:/Stage/aposimz/multiomics_final_object')

## CELL number

In [ ]:
library(dplyr)


# plot cell number 
df_idents <- as.data.frame(table(Idents(multiomics)))

ggplot(df_idents, aes(reorder(Var1, -Freq), Freq))+
geom_bar(stat = 'identity', aes(fill = Var1))+
coord_flip()+
labs(x="Sample",y="Cells number")


# plot cell proportion 
pbmc_mix<-data.frame(sample=multiomics$orig.ident,population=Idents(multiomics))
pbmc_df <- as.data.frame(table(pbmc_mix))
pbmc_df <- pbmc_df %>% group_by(population) %>% dplyr::mutate(proportion = Freq/(sum(Freq)))

ggplot(pbmc_df, aes(population, proportion))+
geom_bar(stat = 'identity', aes(fill = sample), position = 'dodge')+theme(axis.text.x = element_text(angle = 45, hjust = 1))


# other way 
pbmc_mix<-data.frame(sample=multiomics$orig.ident,population=Idents(multiomics))
ggplot(pbmc_mix, aes(population))+coord_flip()+
geom_bar(aes(fill = sample))+theme(axis.text.x = element_text(angle = 45, hjust = 1))



# cell types sexual identity proportion
DF_Count <- multiomics@meta.data  %>%group_by(cell_types,identity) %>%
  count() %>%
  ungroup() %>%
  group_by(cell_types) %>%
  mutate(Freq = n/sum(n)*100)

ggplot(DF_Count, aes(x = cell_types, y = Freq, fill = identity))+
  geom_col()+ geom_text(aes(label = paste(round(Freq, 2),"%")),position = position_stack(vjust = 0.5))+coord_flip()




## Cell Cycle

In [ ]:
DefaultAssay(multiomics)='SCT'
m.g2m.genes=readRDS(file ='D:/Stage/aposimz/m.g2m.genes' )
m.s.genes=readRDS(file ='D:/Stage/aposimz/m.s.genes' )

#  add to the object meta data: S.Score, G2M.Score, and Phase
multiomics <- CellCycleScoring(multiomics, s.features = m.s.genes, g2m.features = m.g2m.genes, set.ident = TRUE)
DimPlot(multiomics,reduction = 'wnn.umap')


DF_Count <- multiomics@meta.data  %>%group_by(cell_types,Phase) %>%
  count() %>%
  ungroup() %>%
  group_by(cell_types) %>%
  mutate(Freq = n/sum(n)*100)

ggplot(DF_Count, aes(x = cell_types, y = Freq, fill = Phase))+
  geom_col()+ geom_text(aes(label = paste(round(Freq, 2),"%")),position = position_stack(vjust = 0.5))+coord_flip()



## Plot umap 

In [ ]:
library(ggplot2)

Idents(multiomics)=multiomics$cell_types
# PLOT UMAP
DimPlot(multiomics,reduction = 'wnn.umap',pt.size = 1.66,group.by='Replicat') + guides(color = guide_legend(override.aes = list(size=5), ncol=1) ) + theme_classic(base_size = 15)| DimPlot(multiomics,reduction = 'wnn.umap',pt.size = 1.66,label = T) + guides(color = guide_legend(override.aes = list(size=5), ncol=1) ) + theme_classic(base_size = 15)

## plot genes markers| FeaturePlot 

In [ ]:
FeaturePlot(multiomics, features=c('Wnt4','Dab2'),reduction = 'pca_umap')                                   # glomerulosa markers gene 
FeaturePlot(multiomics, features=c('Akr1b7','Cyp11b1'),reduction = 'pca_umap')                              # Fasciculata markers gene 
FeaturePlot(multiomics, features=c('Th','Slc18a1','Slc18a2','Ddc','Dbh','Lgr5'),reduction = 'pca_umap')     # medulla markers gene 
FeaturePlot(multiomics, features=c("Rspo3",'Gli1'),reduction = 'pca_umap')                                  # capsular ( progennitor compartment) markers gene 
FeaturePlot(multiomics, features=c('Akr1c18'),reduction = 'pca_umap')                                       # x zone
FeaturePlot(multiomics, features=c('Cdh5','Pecam1','Flt1'),reduction = 'pca_umap')                          # endothelial  markers gene 
FeaturePlot(multiomics, features=c('Star','Cyp11a1', 'Hsd3b1', 'Scarb1'),reduction = 'wnn.umap')            # cortex markers gene 
FeaturePlot(multiomics, features=c('Csmd1'),reduction = 'pca_umap')                                         # capsular marker genes 
FeaturePlot(multiomics, features=c("Csmd1","Ano1","Col3a1","Cdh11","Dcn","Rspo3"),reduction = 'pca_umap')   # stromal cells

##  plot genes markers| DOTPLOT

In [ ]:
# markers genes 
genes=c('Wnt4','Dab2','Akr1b7','Cyp11b1',"Csmd1","Rspo3","Tcf21", 'Flt1','Pecam1',"Lsamp","Lrrtm4",'Th','Slc18a1',"Cd74","Cd84","Akr1c18","Pik3c2g")

dotx= DotPlot(multiomics,assay= 'SCT',features = genes,dot.scale = 6)+ RotatedAxis() +  scale_colour_gradient2(low = "darkred", mid = "white",high = "darkgreen",midpoint = 0,)

 # Markers genes plot 
 # Main plot
  minmaxthis = function(x) { 
    (x - min(x))/(max(x) - min(x))
  }
  
  data.dot = dotx$data
  
  # scaled value
  scaled_data <- 
    data.dot %>%
    group_by(features.plot) %>%
    mutate(minmaxscale = minmaxthis(avg.exp))
  
  scaled_data$avg.exp2 = ( scaled_data$avg.exp - min(scaled_data$avg.exp) )    / 
    (max(scaled_data$avg.exp)- min(scaled_data$avg.exp))
  
  
  pmain <- ggplot(scaled_data)  + 
    geom_point(aes(x = features.plot, y = id, color = minmaxscale, size = pct.exp), stat = "identity") + 
    scale_size(range = c(0, 10), breaks  = seq(0,100,20), labels  = paste0(seq(0,100,20),"%"))  + 
    scale_color_gradient(low = "white",high = "#0f4d91",breaks = seq(0,1,0.2))  +
    guides(color = guide_colourbar(title = "Expression", barwidth = 1, barheight = 10),
           size = guide_legend(title = "Cells (%)",reverse=TRUE)) + 
    xlab("") + ylab("") + 
    
    theme(panel.border=element_blank(), 
          axis.line = element_line(colour = 'black', size = 1), 
          axis.ticks = element_blank(),
          axis.text = element_text(size=12, colour = "black"), 
          axis.text.x = element_text(angle = 90, hjust = 1),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          legend.key=element_blank(),
          # legend.title=element_blank(),
          legend.text=element_text(size=12))
  
pmain
  


# only steroidogenic cell types dataset analysis ( only_stero )

## ONLY STERO CELLS RNA/ATAC processing

In [ ]:
table(Idents(only_STERO))
# Subset
DefaultAssay(multiomics)='RNA'
#only_STERO <- subset(x=multiomics,idents=c("Male Fasciculata","Female Fasciculata","Female Glomerulosa","Capsular Cells","Male Glomerulosa"))
only_STERO <- subset(x=multiomics,idents=c("Male Fasciculata Replicate 1","Male Fasciculata Replicate 2","Female Fasiculata Replicate 2","Female Glomerulosa","Female Fasciculate Replicate 1" ,"Male Glomerulosa","Capsular Cells"))
only_STERO=subset(only_STERO,subset  = nCount_ATAC<100000)
only_STERO=subset(only_STERO,subset  = nCount_RNA<15000)
#only_STERO <- only_STERO[,!colnames(only_STERO) %in% mixt_clusters] # process by first running to the clustering and highlinghting this cells
#only_STERO <- only_STERO[,!colnames(only_STERO) %in% mixt_clusters_2] # process by first running to the clustering and highlinghting this cells

       ####RNA PROCESSING### 

DefaultAssay(only_STERO)='RNA'  # take RNA assay 

only_STERO <- SplitObject(only_STERO, split.by = "Replicat") # list 

for (i in 1:length(only_STERO)) {
    only_STERO[[i]] <- SCTransform(only_STERO[[i]],variable.features.n =NULL,variable.features.rv.th =1.1,return.only.var.genes = FALSE,assay = 'RNA',new.assay.name = 'SCT_INT',vars.to.regress = 'percent.mt')
    }


# Select the most variable features to use for integration
integ_features <- SelectIntegrationFeatures(object.list = only_STERO, nfeatures = 3000) 

# Prepare the SCT list object for integration
only_STERO  <- PrepSCTIntegration(object.list = only_STERO, 
                                   anchor.features =integ_features)


integ_anchors <- FindIntegrationAnchors(object.list = only_STERO, 
                                        normalization.method = "SCT", 
                                        anchor.features = integ_features,reduction="cca")

# Integrate across conditions
only_STERO <- IntegrateData(anchorset = integ_anchors,normalization.method = "SCT")                 # replace seurat list by object



# only_STERO[['SCT']]=NULL
# only_STERO[['peaks']]=NULL
# only_STERO@meta.data[15:21]=NULL


# CLustering 
DefaultAssay(only_STERO)='integrated'
only_STERO=RunPCA(only_STERO, verbose = FALSE,assay="integrated",reduction.name='PCA_INT',npcs = 50)
ElbowPlot(only_STERO,reduction ='PCA_INT',ndims = 50 ) # choose 15
only_STERO <- RunICA(only_STERO,verbose = TRUE,nics = 30,reduction.name='ICA_STERO',assay = "integrated")
only_STERO=FindNeighbors(only_STERO,assay ='integrated',reduction ='PCA_INT', dims = 1:25)
only_STERO=FindClusters(only_STERO, resolution = 0.5,algorithm = 4,graph.name = 'integrated_nn')
only_STERO=RunUMAP(only_STERO,dims = 1:20, reduction.name = 'UMAP_PCA_RNA_INT',reduction = 'PCA_INT')

# compute new assay based on RNA , it will be the new default assay for rna expression 

only_STERO=SCTransform(only_STERO,variable.features.n =NULL,variable.features.rv.th =1.1,return.only.var.genes = FALSE,assay = 'RNA',new.assay.name = 'SCT_RUN_STERO',vars.to.regress = 'percent.mt')
DefaultAssay(only_STERO)='SCT_RUN_STERO'

#subset_endo=WhichCells(object = only_STERO,idents = 5)
#subset_medulla=WhichCells(object = only_STERO,idents = 9)


FeaturePlot(only_STERO, features=c('Wnt4','Dab2'),reduction = 'UMAP_PCA_RNA_INT')                                   # glomerulosa markers gene 
FeaturePlot(only_STERO, features=c('Akr1b7','Cyp11b1'),reduction = 'UMAP_PCA_RNA_INT')                              # Fasciculata markers gene 
FeaturePlot(only_STERO, features=c('Th','Slc18a1','Slc18a2','Ddc','Dbh','Lgr5'),reduction = 'UMAP_PCA_RNA_INT')     # medulla markers gene 
FeaturePlot(only_STERO, features=c("Rspo3",'Gli1'),reduction = 'UMAP_PCA_RNA_INT')                                  # capsular ( progennitor compartment) markers gene 
FeaturePlot(only_STERO, features=c('Akr1c18'),reduction = 'UMAP_PCA_RNA_INT')                                       # x zone
FeaturePlot(only_STERO, features=c('Cdh5','Pecam1','Flt1'),reduction = 'UMAP_PCA_RNA_INT')                          # endothelial  markers gene 
FeaturePlot(only_STERO, features=c('Star','Cyp11a1', 'Hsd3b1', 'Scarb1'),reduction = 'UMAP_PCA_RNA_INT')            # cortex markers gene 
FeaturePlot(only_STERO, features=c('Csmd1'),reduction = 'UMAP_PCA_RNA_INT')                                         # capsular marker genes 
FeaturePlot(only_STERO, features=c('Ptprc'),reduction = 'UMAP_PCA_RNA_INT')                                         # C immutnitaire  markers gene 


DimPlot(only_STERO,reduction = 'UMAP_PCA_RNA_INT',label = T) | DimPlot(object = only_STERO, label = TRUE,group.by = 'identity',reduction = 'UMAP_PCA_RNA_INT') |DimPlot(object = only_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_PCA_RNA_INT')

################## NOT RUN
#there is two clusters with endothelial markers genes , and no cortex markers genes , likely doublet , i remove them for the rest of the analysis 
# take ident of bad cluusters and subset ident 7 and 9
 #mixt_clusters=WhichCells(object = only_STERO,idents = c('7','9'))
#DimPlot(only_STERO,cells.highlight = mixt_clusters_2)
#FeaturePlot(only_STERO, features=c('Cdh5','Pecam1','Flt1'),reduction = 'UMAP_PCA_RNA_INT') # endothelial  markers gene 
# remove clusters 16 ==> medulla 
# mixt_clusters_2=WhichCells(object = only_STERO,idents = c('16'))
###################NOT RUN


findmarkers=FindAllMarkers(only_STERO,assay = 'SCT_RUN_STERO')
 # annotation cluster 
only_STERO@meta.data$old_ident=Idents(only_STERO)   # save idents number 
only_STERO <- RenameIdents(only_STERO,`1`='Male_Fasciculata_1',
                     `2`='Male_Glomerulosa',
                     `3`='Female_Fasciculata_1',
                     `4`='Female_Glomerulosa',
                     `5`='Female_Fasciculata_2',
                     `6`='Male_Fasciculata_2',
                     `7`='Capsular_Cells'
                     )
only_STERO@meta.data$new_ident=Idents(only_STERO)
# only_STERO$cell_types =NULL        # remove from previous multiomics object
DimPlot(only_STERO,reduction  = 'UMAP_PCA_RNA_INT',label = T)


##### ############################################################################################ATAC
       #### ATAC processing  AND PEAKS  generating ## 
##Peak calling 
library(EnsDb.Mmusculus.v79)
library(BSgenome.Mmusculus.UCSC.mm10)
DefaultAssay(only_STERO)='ATAC'

run_macs_ONLY_STERO<- CallPeaks(object = only_STERO,macs2.path = '/home/absta/.conda/envs/envpython/bin/macs2',effective.genome.size = 1.87e9 ,group.by = "new_ident",assay = 'ATAC') 

# get gene annotations for hg38
annotation <- GetGRangesFromEnsDb(ensdb = EnsDb.Mmusculus.v79)
seqlevelsStyle(annotation) <- "UCSC"


# remove peaks on nonstandard chromosomes and in genomic blacklist regions
peaks_only_stero <- keepStandardChromosomes(run_macs_ONLY_STERO, pruning.mode = "coarse",species ='Mus_musculus' )
peaks_only_stero <- subsetByOverlaps(x = peaks_only_stero, ranges = blacklist_mm10, invert = TRUE)

# quantify counts in each peak
macs2_counts <- FeatureMatrix(
  fragments = Fragments(only_STERO),
  features = peaks_only_stero,
  cells = colnames(only_STERO)
)


# create a new assay using the MACS2 peak set and add it to the Seurat object
only_STERO[["peaks"]] <- CreateChromatinAssay(
  counts = macs2_counts,
  fragments = Fragments(only_STERO),
  annotation = annotation
)

DefaultAssay(only_STERO) <- "peaks"
only_STERO <- FindTopFeatures(only_STERO, min.cutoff = 5)
only_STERO <- RunTFIDF(only_STERO)
only_STERO <- RunSVD(only_STERO)
DepthCor(only_STERO,n = 50)

only_STERO <- RunUMAP(object = only_STERO,reduction = 'lsi',dims = 2:50,reduction.name = 'UMAP_LSI_ATAC')
only_STERO <- FindNeighbors(object = only_STERO,reduction = 'lsi',dims = 2:50) # use SNN for findclusters 
#only_STERO <- FindClusters(object = only_STERO,algorithm = 1,resolution = 0.7,verbose = FALSE)
DimPlot(object = only_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC')+DimPlot(only_STERO,reduction = 'UMAP_PCA_RNA_INT',group.by = 'Replicat')

##################################INTEGRATION ON REPLICAT  for atac 


crazy=only_STERO

crazy <- SplitObject(crazy, split.by = "Replicat") # list 


x=SelectIntegrationFeatures(
  crazy,
  nfeatures = 3000,
  assay = c('peaks','peaks'),
  verbose = TRUE
)


# find integration anchors
integration.anchors <- FindIntegrationAnchors(
  object.list = crazy,
  anchor.features =x,
  reduction = "rlsi",
  dims = 2:30
)

# integrate LSI embeddings
integrated <- IntegrateEmbeddings(
  anchorset = integration.anchors,
  reductions = X1[["lsi"]],
  new.reduction.name = "integrated_lsi",
  dims.to.integrate = 2:30
)

integrated <- RunUMAP(integrated, reduction = "integrated_lsi", dims = 2:30)

DimPlot(integrated ,group.by = 'Replicat')
 
 
 #######################ADD INT LSI ON OBJECT 

only_STERO@reductions$UMAP_LSI_ATAC_INT=integrated@reductions[["umap"]]
only_STERO@reductions$integrated_lsi=integrated@reductions[["integrated_lsi"]]



DimPlot(object = only_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC')+DimPlot(only_STERO,reduction = 'UMAP_PCA_RNA_INT',group.by = 'Replicat')+DimPlot(object = only_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC_INT')

 
 

## ONLY STERO WNN UMAP

In [ ]:

ElbowPlot(only_STERO,reduction = 'PCA_INT',ndims = 50)


only_STERO <- FindMultiModalNeighbors(
  only_STERO, reduction.list = list("PCA_INT", "integrated_lsi"), 
  dims.list = list(1:20, 2:30),weighted.nn.name = 'X1'
)
only_STERO <- RunUMAP(only_STERO, nn.name = "X1", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")


FeaturePlot(only_STERO, features=c('Wnt4','Dab2'),reduction = 'wnn.umap')                                   # glomerulosa markers gene 
FeaturePlot(only_STERO, features=c('Akr1b7','Cyp11b1'),reduction = 'wnn.umap')                              # Fasciculata markers gene 
FeaturePlot(only_STERO, features=c('Cdh5','Pecam1','Flt1'),reduction = 'wnn.umap')                          # endothelial  markers gene 
FeaturePlot(only_STERO, features=c('Star','Cyp11a1', 'Hsd3b1', 'Scarb1'),reduction = 'wnn.umap')            # cortex markers gene 
FeaturePlot(only_STERO, features=c('Csmd1'),reduction = 'wnn.umap')                                         # capsular marker genes 
DimPlot(only_STERO)# capsular marker genes 


only_STERO@meta.data$old_ident=Idents(only_STERO)
only_STERO <- RenameIdents(only_STERO,`1`='Male_Fasciculata_1',
                     `2`='Female_Fasciculata_2',
                     `3`='Male_Glomerulosa',
                     `4`='Female_Glomerulosa',
                     `5`='Male_Fasciculata_2',
                     `6`='Female_Fasciculata_1',
                     `7`='Capsular_Cells'
                        )
only_STERO@meta.data$new_ident=Idents(only_STERO)


DimPlot(only_STERO,reduction  = 'wnn.umap',label = T)



Idents(only_STERO)=only_STERO@meta.data$old_ident
only_STERO <- RenameIdents(only_STERO,`1`='Male_Fasciculata',
                     `2`='Female_Fasciculata',
                     `3`='Male_Glomerulosa',
                     `4`='Female_Glomerulosa',
                     `5`='Male_Fasciculata',
                     `6`='Female_Fasciculata',
                     `7`='Capsular_Cells'
                        )
only_STERO@meta.data$merging_ident=Idents(only_STERO)

DimPlot(only_STERO,reduction  = 'wnn.umap',label = T,pt.size = 2)




saveRDS(only_STERO,file = 'only_STERO_final')
only_STERO=readRDS(file = 'D:/Users/Documents/aposimz/only_STERO_final')

## Remove sex chromosome ( only_stero )

### ONLY STERO Remove SEX

In [ ]:

# 1) REMOVE SEX GENES 
      # need to remove previous Peaks assay

      DefaultAssay(only_STERO)='RNA'
      counts <- GetAssayData(only_STERO, assay = "RNA") # get genes from RNA 
      counts <- counts[-(which(rownames(counts) %in% chrx_y$chrX_and_Y)),] # remove sexual genes 
      remove_sex=only_STERO  # create other object 
      remove_sex[['peaks']] <- NULL
      remove_sex@reductions[["UMAP_LSI_ATAC"]] <- NULL
      remove_sex@reductions[["UMAP_LSI_ATAC_INT"]] <- NULL
      remove_sex@assays$SCT_RUN_STERO<-NULL
      remove_sex <- subset(remove_sex, features = rownames(counts))  # remove_sex= seurat object RNA ASSAY without sex genes 


# 2) remove peaks of X and Y 
      # need to remove previous RNA assay 
      
      DefaultAssay(only_STERO)='peaks'
      
      peaks=only_STERO
      peaks[['RNA']] <- NULL
      counts_peaks=as.data.frame(rownames(peaks)) #  take all the peaks
      counts_peak_grepXY=counts_peaks$`rownames(peaks)`[!grepl("chrX|Y.*\\d",counts_peaks$`rownames(peaks)`)] # subset  peaks non beginning by chrX | chrY
      peaks <- peaks[rownames(peaks) %in% counts_peak_grepXY, ]
      peaks <- GetAssayData(object =  peaks[['peaks']])
 
           
# 3) ADD peaks assay with no sex peaks in remove_sex
      remove_sex[["peaks_no_sex"]] <- peaks@assays[["peaks"]]  # peaks_no_sex
      remove_sex=RenameAssays(remove_sex,RNA='RNA_no_sex')     # RNA_no_sex


### RNA process 
  # 1) Replicat integration
      
      DefaultAssay(remove_sex)='RNA_no_sex'  # take RNA assay 
      remove_sex <- SplitObject(remove_sex, split.by = "Replicat") # list 
      for (i in 1:length(remove_sex)) {
          remove_sex[[i]] <- SCTransform(remove_sex[[i]],variable.features.n =NULL,variable.features.rv.th =1.1
                                         ,return.only.var.genes = FALSE,assay = 'RNA_no_sex',
                                         new.assay.name = 'SCT_INT',vars.to.regress = 'percent.mt')
          }
      
      
      # Select the most variable features to use for integration
      integ_features <- SelectIntegrationFeatures(object.list = remove_sex, nfeatures = 3000) 
      
      # Prepare the SCT list object for integration
      remove_sex  <- PrepSCTIntegration(object.list = remove_sex, 
                                         anchor.features =integ_features)
      
      
      integ_anchors <- FindIntegrationAnchors(object.list = remove_sex, 
                                              normalization.method = "SCT", 
                                              anchor.features = integ_features,reduction="cca")
      
      # Integrate across conditions
      remove_sex <- IntegrateData(anchorset = integ_anchors,normalization.method = "SCT")                 
      
  

   # 2)  CLustering on integrated assay 
      
      DefaultAssay(remove_sex)='integrated'
      remove_sex=RunPCA(remove_sex, verbose = FALSE,assay="integrated",reduction.name='PCA_INT',npcs = 50)
      ElbowPlot(remove_sex,reduction ='PCA_INT',ndims = 50 ) # choose 15
      remove_sex <- RunICA(remove_sex,verbose = TRUE,nics = 30,reduction.name='ICA_STERO',assay = "integrated")
      remove_sex=FindNeighbors(remove_sex,assay ='integrated',reduction ='PCA_INT', dims = 1:25)
      remove_sex=FindClusters(remove_sex, resolution = 0.5,algorithm = 4,graph.name = 'integrated_nn')
      remove_sex=RunUMAP(remove_sex,dims = 1:20, reduction.name = 'UMAP_PCA_RNA_INT',reduction = 'PCA_INT')

    # compute new assay based on RNA_no_sex , it will be the new default assay for rna expression 
        
        remove_sex=SCTransform(remove_sex,variable.features.n =NULL,variable.features.rv.th =1.1,return.only.var.genes = FALSE,assay = 'RNA_no_sex',new.assay.name = 'SCT_RUN_STERO',vars.to.regress = 'percent.mt')
        DefaultAssay(remove_sex)='SCT_RUN_STERO'


# ATAC process   
    # Normalisation and reduction
      DefaultAssay(remove_sex) <- "peaks_no_sex"
      remove_sex <- FindTopFeatures(remove_sex, min.cutoff = 5)
      remove_sex <- RunTFIDF(remove_sex)
      remove_sex <- RunSVD(remove_sex)
      DepthCor(remove_sex,n = 50)

      remove_sex <- RunUMAP(object = remove_sex,reduction = 'lsi',dims = 2:50,reduction.name = 'UMAP_LSI_ATAC')
      remove_sex <- FindNeighbors(object = remove_sex,reduction = 'lsi',dims = 2:50) # use SNN for findclusters 
      #only_STERO <- FindClusters(object = only_STERO,algorithm = 1,resolution = 0.7,verbose = FALSE)
      DimPlot(object = remove_sex, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC')+DimPlot(remove_sex,reduction = 'UMAP_PCA_RNA_INT',group.by = 'Replicat')

    #INTEGRATION ON REPLICAT  for atac 

      Replicat=remove_sex
       # Split Object 
      Replicat <- SplitObject(Replicat, split.by = "Replicat") 
      
      # Find top 3000 conserved features 
      x=SelectIntegrationFeatures(               
        Replicat,
        nfeatures = 3000,
        assay = c('peaks','peaks'),
        verbose = TRUE
      )


      # find integration anchors
      integration.anchors <- FindIntegrationAnchors(
        object.list = Replicat,
        anchor.features =x,
        reduction = "rlsi",
        dims = 2:30
      )
      X1=remove_sex 

      # integrate LSI embeddings
      integrated <- IntegrateEmbeddings(
        anchorset = integration.anchors,
        reductions = X1[["lsi"]],
        new.reduction.name = "integrated_lsi",
        dims.to.integrate = 2:30
      )

    integrated <- RunUMAP(integrated, reduction = "integrated_lsi", dims = 2:30)
    DimPlot(integrated ,group.by = 'Replicat')
       
 
    # ADD INT_LSI into OBJECT 

    remove_sex@reductions$UMAP_LSI_ATAC_INT=integrated@reductions[["umap"]] # add UMAP int
    remove_sex@reductions$integrated_lsi=integrated@reductions[["integrated_lsi"]] # Add LSI int  

    # Visualisation 
    DimPlot(object = remove_sex, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC')+DimPlot(remove_sex,reduction = 'UMAP_PCA_RNA_INT',group.by = 'Replicat')+DimPlot(object = remove_sex, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC_INT')


    # Joint UMAP visualization
   
    # non integrated multimodal umap    
    remove_sex <- FindMultiModalNeighbors(object = remove_sex,reduction.list = list("pca", "lsi"), dims.list = list(1:20,2:40),modality.weight.name = "weighted.nn",verbose = TRUE)
    ElbowPlot(remove_sex,ndims = 50)
  remove_sex <- RunUMAP(object = remove_sex,nn.name = "weighted.nn",assay = "SCT",verbose = TRUE,reduction.name = 'wnn.umap')
  
  # integrated multimodal umap 

    remove_sex <- FindMultiModalNeighbors(
      remove_sex, reduction.list = list("PCA_INT", "integrated_lsi"), 
      dims.list = list(1:20, 2:30),weighted.nn.name = 'X1'
    )
    remove_sex <- RunUMAP(remove_sex, nn.name = "X1", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")

    DimPlot(remove_sex,reduction = 'wnn.umap')




## not run 

In [ ]:
# markers genes 
 # 0) all_markers
      all_markers_replicat=FindAllMarkers(multiomics,assay = 'SCT_RUN_STERO',logfc.threshold = 0.1)
    # Write as csv 
      write.csv(all_markers$gene[all_markers$p_val_adj<0.05&all_markers$avg_log2FC>1&all_markers$cluster=="Immune Cells"],file = 'all_markers.txt',quote = F,row.names = F,col.names = F)
      
      
      
      
      
      
      
      multiomics <- RenameIdents(multiomics,`Endothelial Cells`='Endothelial Cells',
                     `Male specific Medulla`='Male Medulla',
                     `Male Fasciculata Replicate 1`='Male Fasciculata Replicate 1',
                     `Male Fasciculata Replicate 2`='Male Fasciculata Replicate 2',
                     `Female Fasiculata Replicate 2`='Female Fasciculata Replicate 2',
                     `Female Glomerulosa`='Female Glomerulosa',
                     `Female Fasciculate Replicate 1`='Female Fasciculata Replicate 1',
                     `Male Glomerulosa`='Male Glomerulosa',
                     `Female specific Medulla`='Female Medulla',
                     `Medulla progenitor Cells`='Medulla progenitor Cells',
                     `Capsular Cells`='Capsular Cells',`Immune Cells`='Immune Cells',
                     `Fetal adrenocortical Cells `= 'Stromal cells',
                     `X zone `='X zone')
      
      multiomics@meta.data$cell_types=Idents(multiomics)
      
      
      
    # 0.bis) conserved markers

      FindConservedMarkers(pbmc_small, ident.1 = 0, ident.2 = 1, grouping.var = "groups")

      all_conserved=FindMarkers(object = multiomics,ident.1 = c("Male Glomerulosa",
                                      "Male Fasciculata Replicate 1",
                                      "Male Fasciculata Replicate 2",
                                      "Female Fasciculata Replicate 1",
                                      "Female Fasiculata Replicate 2"))
      
      all_conserved$gene=rownames(all_conserved)
 # 1) reorder clusters :
        # reorder clusters
      # full dataset
        multiomics@active.ident <- factor(multiomics@active.ident,   
                                    levels=c("Female Glomerulosa",
                                              "Male Glomerulosa",
                                              "Male Fasciculata Replicate 1",
                                              "Male Fasciculata Replicate 2",
                                              "Female Fasciculata Replicate 1",
                                              "Female Fasiculata Replicate 2",
                                              "Capsular Cells",
                                              "Stromal cells",
                                              "Endothelial Cells",
                                              "Medulla progenitor Cells",
                                              "Male Medulla",
                                              "Female Medulla",
                                              "Immune Cells",
                                               "X zone"))


    # only_STERO
          only_STERO@active.ident <- factor(only_STERO@active.ident, 
                            levels=c("Male_Glomerulosa",
                                      "Female_Glomerulosa",
                                      "Female_tZone",
                                      "Male_tZone",
                                      "Male_Fasciculata",
                                      "Female_Fasciculata",
                                      "Capsular_cells"))


# FIND MARKERS REPLICATE DIFFERENCES : 
  
Replicat=FindMarkers(multiomics,ident.1 ="Female Fasciculata Replicate 1",ident.2 = "Female Fasiculata Replicate 2",assay = 'SCT' )
Replicat$genes=rownames(Replicat)
write.table(Replicat$genes,file = 'ReplicatFEMALE.txt',quote = F,col.names = F,row.names = F)


## Cell cycle

In [ ]:
DefaultAssay(only_STERO)='SCT_RUN_STERO'
m.g2m.genes=readRDS(file ='D:/Stage/aposimz/m.g2m.genes' )
m.s.genes=readRDS(file ='D:/Stage/aposimz/m.s.genes' )

#  add to the object meta data: S.Score, G2M.Score, and Phase
multiomics <- CellCycleScoring(multiomics, s.features = m.s.genes, g2m.features = m.g2m.genes, set.ident = TRUE)
DimPlot(multiomics,reduction = 'wnn.umap')


DF_Count <- multiomics@meta.data  %>%group_by(cell_types,Phase) %>%
  count() %>%
  ungroup() %>%
  group_by(cell_types) %>%
  mutate(Freq = n/sum(n)*100)

ggplot(DF_Count, aes(x = cell_types, y = Freq, fill = Phase))+
  geom_col()+ geom_text(aes(label = paste(round(Freq, 2),"%")),position = position_stack(vjust = 0.5))+coord_flip()



## Cell number 

In [ ]:

Idents(only_STERO)=only_STERO@meta.data$leiden_annotated

# plot cell number 
df_idents <- as.data.frame(table(Idents(only_STERO)))

ggplot(df_idents, aes(reorder(Var1, -Freq), Freq))+
geom_bar(stat = 'identity', aes(fill = Var1))+
coord_flip()+
labs(x="Sample",y="Cells number")


# plot cell proportion 
pbmc_mix<-data.frame(sample=only_STERO$orig.ident,population=Idents(only_STERO))
pbmc_df <- as.data.frame(table(pbmc_mix))
pbmc_df <- pbmc_df %>% group_by(population) %>% dplyr::mutate(proportion = Freq/(sum(Freq)))

ggplot(pbmc_df, aes(population, proportion))+
geom_bar(stat = 'identity', aes(fill = sample), position = 'dodge')+theme(axis.text.x = element_text(angle = 45, hjust = 1))


# other way 
pbmc_mix<-data.frame(sample=only_STERO$orig.ident,population=Idents(only_STERO))
ggplot(pbmc_mix, aes(population))+coord_flip()+
geom_bar(aes(fill = sample))+theme(axis.text.x = element_text(angle = 45, hjust = 1))



# cell types sexual identity proportion
DF_Count <- only_STERO@meta.data  %>%group_by(leiden_annotated,identity) %>%
  count() %>%
  ungroup() %>%
  group_by(leiden_annotated) %>%
  mutate(Freq = n/sum(n)*100)

ggplot(DF_Count, aes(x = leiden_annotated, y = Freq, fill = identity))+
  geom_col()+ geom_text(aes(label = paste(round(Freq, 2),"%")),position = position_stack(vjust = 0.5))+coord_flip()




## Scanpy clustering | Paga | FDG ( PYTHON )

In [ ]:
%%python



import scanpy  # pip install in  R terminal
import scanpy as sc
import numpy as np

print(np.__version__)
import pandas as pd
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nwx
import os

anndata = sc.read_h5ad('C:/Users/mehdi/Downloads/multiomics_convert(1).h5ad')

male = sc.read_h5ad('D:/TRUTH/male_STERO.h5ad')
del male.uns['neighbors']
del male.obsp['distances']
del male.varm['ICA_STERO']
del male.obsm['X_ICA_STERO']
del male.obsm['X_UMAP_LSI_ATAC']
del male.obsm['X_UMAP_PCA_RNA_INT']
del male.obsm['X_UMAP_LSI_ATAC_INT']
# del anndata.obsm['X_PCA_INT']
del male.varm['PCA_INT']
del male.obsm['X_wnn.umap']

sc.pp.neighbors(male, n_neighbors=50, use_rep='X_PCA_INT')
# sc.tl.leiden(anndata, resolution = 1)
sc.tl.paga(male, groups='leiden_merging')
sc.pl.paga(male)
male.uns['iroot'] = np.flatnonzero(male.obs['leiden_merging'] == 'Male_Glomerulosa')[0]
sc.tl.draw_graph(male, init_pos='paga')
sc.tl.dpt(male)
sc.pl.draw_graph(male, color=['leiden_merging', 'dpt_pseudotime', 'leiden', 'identity'], legend_loc='on data')

dpt_pseudotime = pd.DataFrame(index=male.obs.index.tolist(),
                              columns=["Pseudotime"],
                              data=male.obs["dpt_pseudotime"].tolist())

dpt_pseudotime.to_csv("D:/TRUTH/Pseudotime_male.csv")

female = sc.read_h5ad('D:/TRUTH/female_STERO.h5ad')
del female.uns['neighbors']
del female.obsp['distances']
del female.varm['ICA_STERO']
del female.obsm['X_ICA_STERO']
del female.obsm['X_UMAP_LSI_ATAC']
del female.obsm['X_UMAP_PCA_RNA_INT']
del female.obsm['X_UMAP_LSI_ATAC_INT']
# del anndata.obsm['X_PCA_INT']
del female.varm['PCA_INT']
del female.obsm['X_wnn.umap']

sc.pp.neighbors(female, n_neighbors=50, use_rep='X_PCA_INT')
# sc.tl.leiden(anndata, resolution = 1)
sc.tl.paga(female, groups='leiden_merging')
sc.pl.paga(female)
female.uns['iroot'] = np.flatnonzero(female.obs['leiden_merging'] == 'Female_Glomerulosa')[0]
sc.tl.draw_graph(female, init_pos='paga')
sc.tl.dpt(female)
sc.pl.draw_graph(female, color=['leiden_merging', 'dpt_pseudotime', 'leiden', 'identity'], legend_loc='on data')

dpt_pseudotime = pd.DataFrame(index=female.obs.index.tolist(),
                              columns=["Pseudotime"],
                              data=female.obs["dpt_pseudotime"].tolist())

dpt_pseudotime.to_csv("D:/TRUTH/Pseudotime_female.csv")

del anndata.uns['neighbors']
del anndata.obsp['distances']
del anndata.varm['ICA_STERO']
del anndata.obsm['X_ICA_STERO']
del anndata.obsm['X_UMAP_LSI_ATAC']
del anndata.obsm['X_UMAP_PCA_RNA_INT']
del anndata.obsm['X_UMAP_LSI_ATAC_INT']
# del anndata.obsm['X_PCA_INT']
del anndata.varm['PCA_INT']

sc.pp.neighbors(anndata, n_neighbors=50, use_rep='X_PCA_INT')
# sc.tl.leiden(anndata, resolution = 1)


sc.tl.paga(anndata, groups='leiden_annotated')
sc.pl.paga(anndata, threshold=0.09)
anndata.uns['iroot'] = np.flatnonzero(anndata.obs['leiden_merging'] == 'Capsular_cells')[0]
sc.tl.draw_graph(anndata,
                 init_pos='paga')

# root =np.flatnonzero(anndata.obs['leiden_merging']=='Capsular_cells')[0])

sc.tl.dpt(anndata)
sc.pl.draw_graph(anndata, color=['merging_ident', 'dpt_pseudotime', 'leiden', 'identity'], legend_loc='on data')

# plot

myColors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231',
            '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe',
            '#008080', '#e6beff', '#9a6324', '#000000', '#800000',
            '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080',
            '#ffffff', '#fffac8']

f, axs = plt.subplots(1, 2, figsize=(36, 12))
sns.set(font_scale=1.5)
sns.set_style("white")

sc.pl.draw_graph(anndata,
                 color='merging_ident',
                 palette=myColors,
                 legend_loc='on data',
                 show=True,
                 size=100,
                 legend_fontsize='large')

sc.pl.draw_graph(anndata,
                 color='dpt_pseudotime',
                 legend_loc='on data',
                 show=True,
                 size=400,
                 color_map="viridis",
                 legend_fontsize='large')

sns.despine(offset=10, trim=False)
plt.tight_layout()
plt.show(block=False)
plt.close("all")

# save value

dpt_pseudotime = pd.DataFrame(index=anndata.obs.index.tolist(),
                              columns=["Pseudotime"],
                              data=anndata.obs["dpt_pseudotime"].tolist())

dpt_pseudotime.to_csv("D:/Stage/Pseudotime_RNA.csv")

FA2_comp = pd.DataFrame(index=anndata.obs.index.tolist(),
                        columns=["FA1", "FA2"],
                        data=anndata.obs["X_draw_graph_fa"])

FA2_comp.to_csv("D:/Stage/Pseudotime_RNA.csvFA2_components_ATAC.csv")

# save paga embending
clust_embedding = pd.DataFrame(anndata.uns['paga']['pos'], columns=['X', 'Y'])
clust_embedding['Cluster'] = range(clust_embedding.shape[0])
clust_embedding = clust_embedding[['Cluster', 'X', 'Y']]

clust_embedding.to_csv('D:/Stage/TRUTH/PAGA/cluster_embedding.csv'),
index = False)

# save object


out_dir = 'C:/Users/mehdi/Downloads'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)


def con2edges(con, names=None, sparse=True):
    print('Converting connectivity matrix to edges...')
    n = con.shape[0]
    edges = pd.DataFrame(columns=['From', 'To', 'Connectivity'])

    for i in range(n):
        for j in range(i + 1, n):
            if names is not None:
                fr = names[i]
                to = names[j]
            else:
                fr = str(i)
                to = str(j)

            connectivity = con[i, j]
            if sparse and connectivity == 0:
                continue

            entry = {'From': fr, 'To': to,
                     'Connectivity': con[i, j]}
            edges = edges.append(entry, ignore_index=True)

    return edges


clust_con = anndata.uns['paga']['connectivities'].toarray()
clust_edges = con2edges(clust_con)
clust_edges.to_csv(os.path.join(out_dir, 'cluster_edges.csv'), index=False)

clust_tree_con = anndata.uns['paga']['connectivities_tree'].toarray()
clust_tree_edges = con2edges(clust_tree_con)
clust_tree_edges.to_csv(os.path.join(out_dir, 'cluster_tree_edges.csv'), index=False)

clust_embedding = pd.DataFrame(anndata.uns['paga']['pos'], columns=['X', 'Y'])
clust_embedding['Cluster'] = range(clust_embedding.shape[0])
clust_embedding = clust_embedding[['Cluster', 'X', 'Y']]

clust_embedding.to_csv(os.path.join(out_dir, 'cluster_embedding.csv'), index=False)

# cells edges
cells = anndata.obs['index']
cell_con = anndata.uns['neighbors']['connectivities']
cell_edges = pd.DataFrame(columns=['From', 'To', 'Connectivity'])
n_rows = len(cell_con.indptr)

for i in range(len(cell_con.indptr) - 1):
    row_ind = cell_con.indices[cell_con.indptr[i]:cell_con.indptr[i + 1]]
    print(f'\r\tRow {i} of {n_rows}', end='')
    for k, j in enumerate(row_ind):
        if j > i:
            con = cell_con.data[cell_con.indptr[i] + k]
            fr = cells[i]
            to = cells[j]
            entry = {'From': fr, 'To': to, 'Connectivity': con}
            cell_edges = cell_edges.append(entry, ignore_index=True)
print('\n')

cell_edges.to_csv(os.path.join(out_dir, 'cell_edges.csv'), index=False)

print('outputting cell embedding...')
x = anndata.obsm['X_draw_graph_fr'][:, 0]
y = anndata.obsm['X_draw_graph_fr'][:, 1]
cell_embedding = pd.DataFrame({'Cell': cells, 'X': x, 'Y': y})
cell_embedding.to_csv(os.path.join(out_dir, 'cell_embedding.csv'), index=False)

print('Done!')



## FIND ALL MARKERS on LEIDEN clustering

In [ ]:
# use scanpy clustering on FA graph 
Idents(only_STERO)=only_STERO$leiden
DimPlot(only_STERO,reduction = 'FA',pt.size = 3,label = T)+DimPlot(only_STERO,reduction = 'FA',pt.size = 3,group.by = 'identity')
all_markers=FindAllMarkers(only_STERO,assay = 'SCT_RUN_STERO',logfc.threshold = 0.1,min.pct = 0.1)
markers_foldchange=all_markers[all_markers$avg_log2FC>0.5,]
 # annotation cluster highlinghting sub zone 
only_STERO <- RenameIdents(only_STERO,
                     `0`='Male_Fasciculata_1',  
                     `1`='Female_Glomerulosa',
                     `2`='Male_Glomerulosa',
                     `3`='Female_Fasciculata_1',
                     `4`='Male_Fasciculata_2',
                     `5`='Female_Fasciculata_2',
                     `6`='Male_tZone',
                     `7`='Female_tZone',
                     `8`='Capsular_cells'
                     )
only_STERO@meta.data$leiden_annotated=Idents(only_STERO)

# Create new ident merging 
Idents(only_STERO)=only_STERO$leiden
only_STERO <- RenameIdents(only_STERO,
                     `0`='Male_Fasciculata',  
                     `1`='Female_Glomerulosa',
                     `2`='Male_Glomerulosa',
                     `3`='Female_Fasciculata',
                     `4`='Male_Fasciculata',
                     `5`='Female_Fasciculata',
                     `6`='Male_tZone',
                     `7`='Female_tZone',
                     `8`='Capsular_cells'
                     )

only_STERO$leiden_merging=Idents(only_STERO)





## Get sexual related genes and TF

In [ ]:

library(biomaRt)
library(BSgenome.Mmusculus.UCSC.mm10)

DefaultAssay(only_STERO)='SCT_RUN_STERO'
mart <- useMart("ENSEMBL_MART_ENSEMBL", dataset = "mmusculus_gene_ensembl")

genes.table <- try(biomaRt::getBM(attributes = c("ensembl_gene_id", "external_gene_name",
"description", "gene_biotype", "chromosome_name", "start_position"), mart = mart, useCache = F))   # fetch chromosome info plus some other annotations

chrY.gene = genes.table$external_gene_name[genes.table$chromosome_name == "Y"] 
chrX.gene = genes.table$external_gene_name[genes.table$chromosome_name == "X"]
chrX_and_Y=append(chrY.gene,chrX.gene)
mouse_TF=read.table('D:/ADRENAL_GLAND_PROJECT/mouseTF.txt')
chrx_y=as.data.frame(chrX_and_Y)


## DE ANALYSIS , for merging  and sub clusters and sub clusters intra comparison 

In [ ]:

# 0) optional ) remove genes ==> create an assay object without genes xyz
  remove_only_stero=only_STERO
  remove_only_stero[['peaks']] <- NULL
  counts <- GetAssayData(only_STERO, assay = "SCT_RUN_STERO")
  counts <- counts[-(which(rownames(counts) %in% c('Xist','Tsix','Uty','Ddx3y'))),]
  remove_only_stero <- subset(remove_only_stero, features = rownames(counts))
  
  
# 1) take the right ident - old / new /subset 
  # COMPARING EACH MERGING CLUSTERS 
Idents(remove_only_stero)=remove_only_stero@meta.data$leiden_merging
DefaultAssay(remove_only_stero)='SCT_RUN_STERO' #take good assay 
levels(remove_only_stero)
vector_clusters_F=c( "Female_Fasciculata","Female_Glomerulosa","Female_tZone")
vector_clusters_M=c( "Male_Fasciculata", "Male_Glomerulosa","Male_tZone")

for( i in 1:length(vector_clusters_F)){
  assign(paste0('DE_STERO_',vector_clusters_F[i]),FindMarkers(remove_only_stero,ident.1 =vector_clusters_F[i],ident.2 = vector_clusters_M[i],assay = 'SCT_RUN_STERO',min.pct = 0.1))

}




# FIND all 
ALL_diffmarkers=FindMarkers(remove_only_stero,ident.1 =c("Female_Fasciculata","Female_tZone","Female_Glomerulosa"),ident.2=c("Male_Fasciculata","Male_tZone","Male_Glomerulosa"),assay = 'SCT_RUN_STERO' ,logfc.threshold = 0.1)
ALL_diffmarkers<- ALL_diffmarkers[ALL_diffmarkers$p_val_adj < 0.05,]
ALL_diffmarkers$gene_names <- rownames(ALL_diffmarkers)
ALL_diffmarkers$TF <- mouse_TF$V1[match(ALL_diffmarkers$gene_names, mouse_TF$V1)]
ALL_diffmarkers$sex <- chrx_y$chrX_and_Y[match(ALL_diffmarkers$gene_names, chrx_y$chrX_and_Y)]
write.csv(ALL_diffmarkers,"D:/ADRENAL_GLAND_PROJECT/ALL_diffmarkers.csv", row.names = FALSE)




# FEMALE ZF 
DE_STERO_Female_ZF<- DE_STERO_Female_Fasciculata[DE_STERO_Female_Fasciculata$p_val_adj < 0.05&DE_STERO_Female_Fasciculata$avg_log2FC>0.1,]
DE_STERO_Female_ZF$gene_names <- rownames(DE_STERO_Female_ZF)
DE_STERO_Female_ZF$TF <- mouse_TF$V1[match(DE_STERO_Female_ZF$gene_names, mouse_TF$V1)]
DE_STERO_Female_ZF$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Female_ZF$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Female_ZF,"D:/ADRENAL_GLAND_PROJECT/FASCICULATA/DE_STERO_Female_ZF.csv", row.names = FALSE)



# Male ZF 
DE_STERO_Male_ZF <- DE_STERO_Female_Fasciculata[DE_STERO_Female_Fasciculata$p_val_adj < 0.05&DE_STERO_Female_Fasciculata$avg_log2FC< - 0.1,]
DE_STERO_Male_ZF$gene_names <- rownames(DE_STERO_Male_ZF)
DE_STERO_Male_ZF$TF <- mouse_TF$V1[match(DE_STERO_Male_ZF$gene_names, mouse_TF$V1)]
DE_STERO_Male_ZF$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Male_ZF$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Male_ZF,"D:/ADRENAL_GLAND_PROJECT/FASCICULATA/DE_STERO_Male_ZF.csv", row.names = FALSE)


# Female ZG 
DE_STERO_Female_ZG<- DE_STERO_Female_Glomerulosa[DE_STERO_Female_Glomerulosa$p_val_adj < 0.05 & DE_STERO_Female_Glomerulosa$avg_log2FC>0.1,]
DE_STERO_Female_ZG$gene_names <- rownames(DE_STERO_Female_ZG)
DE_STERO_Female_ZG$TF <- mouse_TF$V1[match(DE_STERO_Female_ZG$gene_names, mouse_TF$V1)]
DE_STERO_Female_ZG$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Female_ZG$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Female_ZG,"D:/ADRENAL_GLAND_PROJECT/GLOMERULOSA/DE_STERO_Female_ZG.csv", row.names = FALSE)


# Male ZG 
DE_STERO_Male_ZG<- DE_STERO_Female_Glomerulosa[DE_STERO_Female_Glomerulosa$p_val_adj < 0.05&DE_STERO_Female_Glomerulosa$avg_log2FC< - 0.1,]
DE_STERO_Male_ZG$gene_names <- rownames(DE_STERO_Male_ZG)
DE_STERO_Male_ZG$TF <- mouse_TF$V1[match(DE_STERO_Male_ZG$gene_names, mouse_TF$V1)]
DE_STERO_Male_ZG$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Male_ZG$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Male_ZG,"D:/ADRENAL_GLAND_PROJECT/GLOMERULOSA/DE_STERO_Male_ZG.csv", row.names = FALSE)


# Female tZ 
DE_STERO_Female_tZ<- DE_STERO_Female_tZone[DE_STERO_Female_tZone$p_val_adj < 0.05 & DE_STERO_Female_tZone$avg_log2FC>0.1,]
DE_STERO_Female_tZ$gene_names <- rownames(DE_STERO_Female_tZ)
DE_STERO_Female_tZ$TF <- mouse_TF$V1[match(DE_STERO_Female_tZ$gene_names, mouse_TF$V1)]
DE_STERO_Female_tZ$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Female_tZ$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Female_tZ,"D:/ADRENAL_GLAND_PROJECT/TZONE/DE_STERO_Female_tZ.csv", row.names = FALSE)


# Male tZ
DE_STERO_Male_tZ<- DE_STERO_Female_tZone[DE_STERO_Female_tZone$p_val_adj < 0.05&DE_STERO_Female_tZone$avg_log2FC< - 0.1,]
DE_STERO_Male_tZ$gene_names <- rownames(DE_STERO_Male_tZ)
DE_STERO_Male_tZ$TF <- mouse_TF$V1[match(DE_STERO_Male_tZ$gene_names, mouse_TF$V1)]
DE_STERO_Male_tZ$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Male_tZ$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Male_tZ,"D:/ADRENAL_GLAND_PROJECT/TZONE/DE_STERO_Male_tZ.csv", row.names = FALSE)





rm(DE_STERO_SUBFemale_Fasciculata_1,DE_STERO_SUBFemale_Fasciculata_2)

## differential peaks accesibility AND motif accessibility

In [ ]:

Idents(only_STERO)=only_STERO@meta.data$leiden_merging
DefaultAssay(only_STERO)='peaks'
vector_clusters_F=c( "Female_Fasciculata","Female_Glomerulosa","Female_tZone")
vector_clusters_M=c( "Male_Fasciculata", "Male_Glomerulosa","Male_tZone")


for( i in 1:length(vector_clusters_F)){ 
  assign(paste0('DA_STERO_',vector_clusters_F[i]),FindMarkers(only_STERO,
                                                        ident.1 =vector_clusters_F[i],
                                                        ident.2 = vector_clusters_M[i],
                                                        assay = 'peaks',
                                                        only.pos = F, test.use = 'LR',
                                                        min.pct = 0.05,
                                                        latent.vars = 'nCount_peaks'))
}



# FASCICULATA 
DA_STERO_Female_Fasciculata$peaks=rownames(DA_STERO_Female_Fasciculata)
DA_STERO_Female_Fasciculata <- DA_STERO_Female_Fasciculata[DA_STERO_Female_Fasciculata$p_val_adj < 0.05,]

peaks_Fem_FASC=DA_STERO_Female_Fasciculata$peaks[DA_STERO_Female_Fasciculata$p_val_adj < 0.05&DA_STERO_Female_Fasciculata$avg_log2FC>0.1]
peaks_Male_FASC=DA_STERO_Female_Fasciculata$peaks[DA_STERO_Female_Fasciculata$p_val_adj < 0.05&DA_STERO_Female_Fasciculata$avg_log2FC< -0.1]


# GLOMERULOSA 
DA_STERO_Female_Glomerulosa$peaks=rownames(DA_STERO_Female_Glomerulosa)
DA_STERO_Female_Glomerulosa <- DA_STERO_Female_Glomerulosa[DA_STERO_Female_Glomerulosa$p_val_adj < 0.05,]

peaks_Fem_GLO=DA_STERO_Female_Glomerulosa$peaks[DA_STERO_Female_Glomerulosa$p_val_adj < 0.05&DA_STERO_Female_Glomerulosa$avg_log2FC>0.1]
peaks_Male_GLO=DA_STERO_Female_Glomerulosa$peaks[DA_STERO_Female_Glomerulosa$p_val_adj < 0.05&DA_STERO_Female_Glomerulosa$avg_log2FC< -0.1]


#  Tzone
DA_STERO_Female_tZone$peaks=rownames(DA_STERO_Female_tZone)
DA_STERO_Female_tZone <- DA_STERO_Female_tZone[DA_STERO_Female_tZone$p_val_adj < 0.05,]

peaks_Fem_tzone=DA_STERO_Female_tZone$peaks[DA_STERO_Female_tZone$p_val_adj < 0.05&DA_STERO_Female_tZone$avg_log2FC>0.1]
peaks_Male_tzone=DA_STERO_Female_tZone$peaks[DA_STERO_Female_tZone$p_val_adj < 0.05&DA_STERO_Female_tZone$avg_log2FC< -0.1]


# Background and motif enrichement :
              
              # Fasciculata : 
                      open.peaks <- AccessiblePeaks(only_STERO, idents = c("Female_Fasciculata","Male_Fasciculata"),assay = 'peaks') 
                      
                      meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")
                      # Female 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Fem_FASC, ],
                            n = 50000
                            )
                              
                            enrich_motif_Fem_FASC <- FindMotifs(object = only_STERO,features = peaks_Fem_FASC,background = peaks.matched)
                            enrich_motif_Fem_FASC <- enrich_motif_Fem_FASC[enrich_motif_Fem_FASC$pvalue<0.05,]
                                          
                      # Male 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Male_FASC, ],
                            n = 50000
                            )
                            enrich_motif_Male_FASC <- FindMotifs(object = only_STERO,features = peaks_Male_FASC,background = peaks.matched)
                            enrich_motif_Male_FASC <- enrich_motif_Male_FASC[enrich_motif_Male_FASC$pvalue<0.05,]

              # Glomerulosa : 
                      open.peaks <- AccessiblePeaks(only_STERO, idents = c("Male_Glomerulosa","Female_Glomerulosa"),assay = 'peaks') 
                      
                      meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")
                      # Female 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Fem_GLO, ],
                            n = 50000
                            )
                              
                            enrich_motif_Fem_GLO<- FindMotifs(object = only_STERO,features = peaks_Fem_GLO,background = peaks.matched)
                            enrich_motif_Fem_GLO <- enrich_motif_Fem_GLO[enrich_motif_Fem_GLO$pvalue<0.05,]

                                          
                      # Male 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Male_GLO, ],
                            n = 50000
                            )
                            enrich_motif_Male_GLO <- FindMotifs(object = only_STERO,features = peaks_Male_GLO,background = peaks.matched)
                            enrich_motif_Male_GLO <- enrich_motif_Male_GLO[enrich_motif_Male_GLO$pvalue<0.05,]

                            
                # TZONE  : 
                      open.peaks <- AccessiblePeaks(only_STERO, idents = c("Female_tZone","Male_tZone"),assay = 'peaks') 
                      
                      meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")
                      # Female 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Fem_tzone, ],
                            n = 50000
                            )
                              
                            enrich_motif_Fem_TZONE<- FindMotifs(object = only_STERO,features = peaks_Fem_tzone,background = peaks.matched)
                            enrich_motif_Fem_TZONE <- enrich_motif_Fem_TZONE[enrich_motif_Fem_TZONE$pvalue<0.05,]
                                          
                      # Male 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Male_tzone, ],
                            n = 50000
                            )
                            enrich_motif_Male_TZONE <- FindMotifs(object = only_STERO,features = peaks_Male_tzone,background = peaks.matched)
                            enrich_motif_Male_TZONE <- enrich_motif_Male_TZONE[enrich_motif_Male_TZONE$pvalue<0.05,]
                            
                            

                          
                            



In [ ]:
library(ggplot2)
# PLOT UMAP
DimPlot(only_STERO,reduction = 'FA',pt.size = 1.66) + guides(color = guide_legend(override.aes = list(size=5), ncol=1) ) + theme_classic(base_size = 15)|
  DimPlot(multiomics,reduction = 'wnn.umap',pt.size = 1.66,group.by='Replicat') + guides(color = guide_legend(override.aes = list(size=5), ncol=1) ) + theme_classic(base_size = 15)| DimPlot(multiomics,reduction = 'wnn.umap',pt.size = 1.66) + guides(color = guide_legend(override.aes = list(size=5), ncol=1) ) + theme_classic(base_size = 15)

In [ ]:
FeaturePlot(multiomics, features=c('Wnt4','Dab2'),reduction = 'pca_umap')                                   # glomerulosa markers gene 
FeaturePlot(multiomics, features=c('Akr1b7','Cyp11b1'),reduction = 'pca_umap')                              # Fasciculata markers gene 
FeaturePlot(multiomics, features=c('Th','Slc18a1','Slc18a2','Ddc','Dbh','Lgr5'),reduction = 'pca_umap')     # medulla markers gene 
FeaturePlot(multiomics, features=c("Rspo3",'Gli1'),reduction = 'pca_umap')                                  # capsular ( progennitor compartment) markers gene 
FeaturePlot(multiomics, features=c('Akr1c18'),reduction = 'pca_umap')                                       # x zone
FeaturePlot(multiomics, features=c('Cdh5','Pecam1','Flt1'),reduction = 'pca_umap')                          # endothelial  markers gene 
FeaturePlot(multiomics, features=c('Star','Cyp11a1', 'Hsd3b1', 'Scarb1'),reduction = 'wnn.umap')            # cortex markers gene 
FeaturePlot(multiomics, features=c('Csmd1'),reduction = 'pca_umap')                                         # capsular marker genes 
FeaturePlot(multiomics, features=c("Csmd1","Ano1","Col3a1","Cdh11","Dcn","Rspo3"),reduction = 'pca_umap')   # stromal cells

In [ ]:
# markers genes 
genes=c('Wnt4','Dab2','Akr1b7','Cyp11b1',"Csmd1","Rspo3","Tcf21", 'Flt1','Pecam1',"Lsamp","Lrrtm4",'Th','Slc18a1',"Cd74","Cd84","Akr1c18","Pik3c2g")

dotx= DotPlot(multiomics,assay= 'SCT',features = genes,dot.scale = 6)+ RotatedAxis() +  scale_colour_gradient2(low = "darkred", mid = "white",high = "darkgreen",midpoint = 0,)

 # Markers genes plot 
 # Main plot
  minmaxthis = function(x) { 
    (x - min(x))/(max(x) - min(x))
  }
  
  data.dot = dotx$data
  
  # scaled value
  scaled_data <- 
    data.dot %>%
    group_by(features.plot) %>%
    mutate(minmaxscale = minmaxthis(avg.exp))
  
  scaled_data$avg.exp2 = ( scaled_data$avg.exp - min(scaled_data$avg.exp) )    / 
    (max(scaled_data$avg.exp)- min(scaled_data$avg.exp))
  
  
  pmain <- ggplot(scaled_data)  + 
    geom_point(aes(x = features.plot, y = id, color = minmaxscale, size = pct.exp), stat = "identity") + 
    scale_size(range = c(0, 10), breaks  = seq(0,100,20), labels  = paste0(seq(0,100,20),"%"))  + 
    scale_color_gradient(low = "white",high = "#0f4d91",breaks = seq(0,1,0.2))  +
    guides(color = guide_colourbar(title = "Expression", barwidth = 1, barheight = 10),
           size = guide_legend(title = "Cells (%)",reverse=TRUE)) + 
    xlab("") + ylab("") + 
    
    theme(panel.border=element_blank(), 
          axis.line = element_line(colour = 'black', size = 1), 
          axis.ticks = element_blank(),
          axis.text = element_text(size=12, colour = "black"), 
          axis.text.x = element_text(angle = 90, hjust = 1),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          legend.key=element_blank(),
          # legend.title=element_blank(),
          legend.text=element_text(size=12))
  
pmain
  


In [ ]:
  DefaultAssay(multiomics)='SCT'
  remove_multiomics=multiomics
  remove_multiomics[['peaks']] <- NULL
  counts <- GetAssayData(remove_multiomics, assay = "SCT")
  counts <- counts[-(which(rownames(counts) %in% c('Xist','Tsix','Uty','Ddx3y'))),]
  remove_multiomics <- subset(remove_multiomics, features = rownames(counts))
  
# male vs Female | fasciculata replicate
multiomics_replicat=FindMarkers(remove_multiomics,assay = 'SCT',logfc.threshold = 0.1,ident.1 =c("Male Fasciculata Replicate 2","Male Fasciculata Replicate 1"),ident.2 = c( "Female Fasciculata Replicate 1", "Female Fasiculata Replicate 2"))
multiomics_replicat=multiomics_replicat[multiomics_replicat$p_val_adj<0.05,]

# male vs male | fasciculata replicate
diff_MALE_fasc=FindMarkers(remove_multiomics,assay = 'SCT',logfc.threshold = 0.1,ident.1 =c("Male Fasciculata Replicate 2"),ident.2 = c("Male Fasciculata Replicate 1"))
diff_MALE_fasc=diff_MALE_fasc[diff_MALE_fasc$p_val_adj<0.05,]

# Female vs male | fasciculata replicate
diff_FEMALE_fasc=FindMarkers(remove_multiomics,assay = 'SCT',logfc.threshold = 0.1,ident.1 =c("Female Fasiculata Replicate 2"),ident.2 = c("Female Fasciculata Replicate 1"))
diff_FEMALE_fasc=diff_FEMALE_fasc[diff_FEMALE_fasc$p_val_adj<0.05,]

write.csv(diff_FEMALE_fasc,file = 'D:/ADRENAL_PROJECT/FASCICULATA/Fasciculata_replicate/diff_FEMALE_fasc.csv')


In [ ]:
# markers genes 
 # 0) all_markers
      all_markers_replicat=FindAllMarkers(multiomics,assay = 'SCT_RUN_STERO',logfc.threshold = 0.1)
    # Write as csv 
      write.csv(all_markers$gene[all_markers$p_val_adj<0.05&all_markers$avg_log2FC>1&all_markers$cluster=="Immune Cells"],file = 'all_markers.txt',quote = F,row.names = F,col.names = F)
      
      
      
      
      
      
      
      multiomics <- RenameIdents(multiomics,`Endothelial Cells`='Endothelial Cells',
                     `Male specific Medulla`='Male Medulla',
                     `Male Fasciculata Replicate 1`='Male Fasciculata Replicate 1',
                     `Male Fasciculata Replicate 2`='Male Fasciculata Replicate 2',
                     `Female Fasiculata Replicate 2`='Female Fasciculata Replicate 2',
                     `Female Glomerulosa`='Female Glomerulosa',
                     `Female Fasciculate Replicate 1`='Female Fasciculata Replicate 1',
                     `Male Glomerulosa`='Male Glomerulosa',
                     `Female specific Medulla`='Female Medulla',
                     `Medulla progenitor Cells`='Medulla progenitor Cells',
                     `Capsular Cells`='Capsular Cells',`Immune Cells`='Immune Cells',
                     `Fetal adrenocortical Cells `= 'Stromal cells',
                     `X zone `='X zone')
      
      multiomics@meta.data$cell_types=Idents(multiomics)
      
      
      
    # 0.bis) conserved markers

      FindConservedMarkers(pbmc_small, ident.1 = 0, ident.2 = 1, grouping.var = "groups")

      all_conserved=FindMarkers(object = multiomics,ident.1 = c("Male Glomerulosa",
                                      "Male Fasciculata Replicate 1",
                                      "Male Fasciculata Replicate 2",
                                      "Female Fasciculata Replicate 1",
                                      "Female Fasiculata Replicate 2"))
      
      all_conserved$gene=rownames(all_conserved)
 # 1) reorder clusters :
        # reorder clusters
      # full dataset
        multiomics@active.ident <- factor(multiomics@active.ident,   
                                    levels=c("Female Glomerulosa",
                                              "Male Glomerulosa",
                                              "Male Fasciculata Replicate 1",
                                              "Male Fasciculata Replicate 2",
                                              "Female Fasciculata Replicate 1",
                                              "Female Fasiculata Replicate 2",
                                              "Capsular Cells",
                                              "Stromal cells",
                                              "Endothelial Cells",
                                              "Medulla progenitor Cells",
                                              "Male Medulla",
                                              "Female Medulla",
                                              "Immune Cells",
                                               "X zone"))


    # only_STERO
          only_STERO@active.ident <- factor(only_STERO@active.ident, 
                            levels=c("Male_Glomerulosa",
                                      "Female_Glomerulosa",
                                      "Female_tZone",
                                      "Male_tZone",
                                      "Male_Fasciculata",
                                      "Female_Fasciculata",
                                      "Capsular_cells"))


# FIND MARKERS REPLICATE DIFFERENCES : 
  
Replicat=FindMarkers(multiomics,ident.1 ="Female Fasciculata Replicate 1",ident.2 = "Female Fasiculata Replicate 2",assay = 'SCT' )
Replicat$genes=rownames(Replicat)
write.table(Replicat$genes,file = 'ReplicatFEMALE.txt',quote = F,col.names = F,row.names = F)


In [ ]:
# use scanpy clustering on FA graph 
Idents(only_STERO)=only_STERO$leiden
DimPlot(only_STERO,reduction = 'FA',pt.size = 3,label = T)+DimPlot(only_STERO,reduction = 'FA',pt.size = 3,group.by = 'identity')
all_markers=FindAllMarkers(only_STERO,assay = 'SCT_RUN_STERO',logfc.threshold = 0.1,min.pct = 0.1)
markers_foldchange=all_markers[all_markers$avg_log2FC>0.5,]
 # annotation cluster highlinghting sub zone 
only_STERO <- RenameIdents(only_STERO,
                     `0`='Male_Fasciculata_1',  
                     `1`='Female_Glomerulosa',
                     `2`='Male_Glomerulosa',
                     `3`='Female_Fasciculata_1',
                     `4`='Male_Fasciculata_2',
                     `5`='Female_Fasciculata_2',
                     `6`='Male_tZone',
                     `7`='Female_tZone',
                     `8`='Capsular_cells'
                     )
only_STERO@meta.data$leiden_annotated=Idents(only_STERO)

# Create new ident merging 
Idents(only_STERO)=only_STERO$leiden
only_STERO <- RenameIdents(only_STERO,
                     `0`='Male_Fasciculata',  
                     `1`='Female_Glomerulosa',
                     `2`='Male_Glomerulosa',
                     `3`='Female_Fasciculata',
                     `4`='Male_Fasciculata',
                     `5`='Female_Fasciculata',
                     `6`='Male_tZone',
                     `7`='Female_tZone',
                     `8`='Capsular_cells'
                     )

only_STERO$leiden_merging=Idents(only_STERO)





#DE ANALYSIS , for merging  and sub clusters and sub clusters intra comparison 

In [ ]:

# install packages

library(biomaRt)
library(BSgenome.Mmusculus.UCSC.mm10)

DefaultAssay(only_STERO)='SCT_RUN_STERO'
mart <- useMart("ENSEMBL_MART_ENSEMBL", dataset = "mmusculus_gene_ensembl")

genes.table <- try(biomaRt::getBM(attributes = c("ensembl_gene_id", "external_gene_name",
"description", "gene_biotype", "chromosome_name", "start_position"), mart = mart, useCache = F))   # fetch chromosome info plus some other annotations

chrY.gene = genes.table$external_gene_name[genes.table$chromosome_name == "Y"] 
chrX.gene = genes.table$external_gene_name[genes.table$chromosome_name == "X"]
chrX_and_Y=append(chrY.gene,chrX.gene)
mouse_TF=read.table('D:/ADRENAL_PROJECT/mouseTF.txt')
chrx_y=as.data.frame(chrX_and_Y)



# 0) optional ) remove genes ==> create an assay object without genes xyz
  remove_only_stero=only_STERO
  remove_only_stero[['peaks']] <- NULL
  counts <- GetAssayData(only_STERO, assay = "SCT_RUN_STERO")
  counts <- counts[-(which(rownames(counts) %in% c('Xist','Tsix','Uty','Ddx3y'))),]
  remove_only_stero <- subset(remove_only_stero, features = rownames(counts))
  
  
# 1) take the right ident - old / new /subset 
  # COMPARING EACH MERGING CLUSTERS 
Idents(remove_only_stero)=remove_only_stero@meta.data$leiden_merging
DefaultAssay(remove_only_stero)='SCT_RUN_STERO' #take good assay 
levels(remove_only_stero)
vector_clusters_F=c( "Female_Fasciculata","Female_Glomerulosa","Female_tZone")
vector_clusters_M=c( "Male_Fasciculata", "Male_Glomerulosa","Male_tZone")

for( i in 1:length(vector_clusters_F)){
  assign(paste0('DE_STERO_',vector_clusters_F[i]),FindMarkers(remove_only_stero,ident.1 =vector_clusters_F[i],ident.2 = vector_clusters_M[i],assay = 'SCT_RUN_STERO',min.pct = 0.1))

}




# FIND all 
ALL_diffmarkers=FindMarkers(remove_only_stero,ident.1 =c("Female_Fasciculata","Female_tZone","Female_Glomerulosa"),ident.2=c("Male_Fasciculata","Male_tZone","Male_Glomerulosa"),assay = 'SCT_RUN_STERO' ,logfc.threshold = 0.1)
ALL_diffmarkers<- ALL_diffmarkers[ALL_diffmarkers$p_val_adj < 0.05,]
ALL_diffmarkers$gene_names <- rownames(ALL_diffmarkers)
ALL_diffmarkers$TF <- mouse_TF$V1[match(ALL_diffmarkers$gene_names, mouse_TF$V1)]
ALL_diffmarkers$sex <- chrx_y$chrX_and_Y[match(ALL_diffmarkers$gene_names, chrx_y$chrX_and_Y)]
write.csv(ALL_diffmarkers,"D:/ADRENAL_PROJECT/ALL_diffmarkers.csv", row.names = FALSE)




# FEMALE ZF 
DE_STERO_Female_ZF<- DE_STERO_Female_Fasciculata[DE_STERO_Female_Fasciculata$p_val_adj < 0.05&DE_STERO_Female_Fasciculata$avg_log2FC>0.1,]
DE_STERO_Female_ZF$gene_names <- rownames(DE_STERO_Female_ZF)
DE_STERO_Female_ZF$TF <- mouse_TF$V1[match(DE_STERO_Female_ZF$gene_names, mouse_TF$V1)]
DE_STERO_Female_ZF$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Female_ZF$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Female_ZF,"D:/ADRENAL_PROJECT/FASCICULATA/DE_STERO_Female_ZF.csv", row.names = FALSE)



# Male ZF 
DE_STERO_Male_ZF <- DE_STERO_Female_Fasciculata[DE_STERO_Female_Fasciculata$p_val_adj < 0.05&DE_STERO_Female_Fasciculata$avg_log2FC< - 0.1,]
DE_STERO_Male_ZF$gene_names <- rownames(DE_STERO_Male_ZF)
DE_STERO_Male_ZF$TF <- mouse_TF$V1[match(DE_STERO_Male_ZF$gene_names, mouse_TF$V1)]
DE_STERO_Male_ZF$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Male_ZF$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Male_ZF,"D:/ADRENAL_PROJECT/FASCICULATA/DE_STERO_Male_ZF.csv", row.names = FALSE)


# Female ZG 
DE_STERO_Female_ZG<- DE_STERO_Female_Glomerulosa[DE_STERO_Female_Glomerulosa$p_val_adj < 0.05 & DE_STERO_Female_Glomerulosa$avg_log2FC>0.1,]
DE_STERO_Female_ZG$gene_names <- rownames(DE_STERO_Female_ZG)
DE_STERO_Female_ZG$TF <- mouse_TF$V1[match(DE_STERO_Female_ZG$gene_names, mouse_TF$V1)]
DE_STERO_Female_ZG$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Female_ZG$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Female_ZG,"D:/ADRENAL_PROJECT/GLOMERULOSA/DE_STERO_Female_ZG.csv", row.names = FALSE)



# Male ZG 
DE_STERO_Male_ZG<- DE_STERO_Female_Glomerulosa[DE_STERO_Female_Glomerulosa$p_val_adj < 0.05&DE_STERO_Female_Glomerulosa$avg_log2FC< - 0.1,]
DE_STERO_Male_ZG$gene_names <- rownames(DE_STERO_Male_ZG)
DE_STERO_Male_ZG$TF <- mouse_TF$V1[match(DE_STERO_Male_ZG$gene_names, mouse_TF$V1)]
DE_STERO_Male_ZG$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Male_ZG$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Male_ZG,"D:/ADRENAL_PROJECT/GLOMERULOSA/DE_STERO_Male_ZG.csv", row.names = FALSE)




# Female tZ 
DE_STERO_Female_tZ<- DE_STERO_Female_tZone[DE_STERO_Female_tZone$p_val_adj < 0.05 & DE_STERO_Female_tZone$avg_log2FC>0.1,]
DE_STERO_Female_tZ$gene_names <- rownames(DE_STERO_Female_tZ)
DE_STERO_Female_tZ$TF <- mouse_TF$V1[match(DE_STERO_Female_tZ$gene_names, mouse_TF$V1)]
DE_STERO_Female_tZ$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Female_tZ$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Female_tZ,"D:/ADRENAL_PROJECT/TZONE/DE_STERO_Female_tZ.csv", row.names = FALSE)


# Male tZ
DE_STERO_Male_tZ<- DE_STERO_Female_tZone[DE_STERO_Female_tZone$p_val_adj < 0.05&DE_STERO_Female_tZone$avg_log2FC< - 0.1,]
DE_STERO_Male_tZ$gene_names <- rownames(DE_STERO_Male_tZ)
DE_STERO_Male_tZ$TF <- mouse_TF$V1[match(DE_STERO_Male_tZ$gene_names, mouse_TF$V1)]
DE_STERO_Male_tZ$sex <- chrx_y$chrX_and_Y[match(DE_STERO_Male_tZ$gene_names, chrx_y$chrX_and_Y)]
write.csv(DE_STERO_Male_tZ,"D:/ADRENAL_PROJECT/TZONE/DE_STERO_Male_tZ.csv", row.names = FALSE)




In [ ]:

library(reshape)

test_data <-data.frame (Female_DEG = c('42','32','370'),
                        Male_DEG = c('28','92','268'),
                        Male_DAR = c('25','88','1326'),
                        Female_DAR = c('2','16','3402'),
                        
                        pseudotime =  c('Glomerulosa','Tzone','Fasciculata'))
            
test_data$pseudotime =factor(test_data$pseudotime,levels=c('Glomerulosa','Tzone','Fasciculata'))
test_data_long <- melt(test_data, id="pseudotime") 


test_data_long$value <- factor(test_data_long$value, levels=c('2','16','25','28','32','42','88','92','268','370','1326','3402'))

test_data_long %>% 
  ggplot(aes(x =pseudotime,y = value, color = variable, group = variable)) + 
  geom_line() + 
  geom_point() +
  ylab("EI1 (Expected Influence with Neighbor)") +
  xlab("Variables")


df1 <- data.frame(x,y,day)
df2 <- reshape::melt(test_data, id="pseudotime")
  ggplot(data = df2, aes(x = pseudotime, y = value, fill = variable)) + geom_bar(stat = "identity")+ facet_wrap(~ variable)





test_DEG <-data.frame (Female_DEG = c('0','42','32','370'),
                        Male_DEG = c('0','28','92','268'),
                        pseudotime =  c('capsule','Glomerulosa','Tzone','Fasciculata'))

test_DEG$pseudotime =factor(test_DEG$pseudotime,levels=c('capsule','Glomerulosa','Tzone','Fasciculata'))
test_DEG <- melt(test_DEG, id="pseudotime") 


test_DEG$value <- factor(test_DEG$value, levels=c('0','28','32','42','92','268','370'))

PLOT_deg=test_DEG %>% 
  ggplot(aes(x =pseudotime,y = value, color = variable, group = variable)) + 
  geom_line() + 
  geom_point() +
  ylab("Variable genes") +
  xlab("Cell types")+
  geom_line(size=2)+
  scale_color_manual(values=c('#D16103','#4E84C4'))
    






test_DAR <-data.frame (
                        Male_DAR = c('0','25','88','1326'),
                        Female_DAR = c('0','2','16','3402'),
                        
                        pseudotime =  c('capsule','Glomerulosa','Tzone','Fasciculata'))
            
test_DAR$pseudotime =factor(test_DAR$pseudotime,levels=c('capsule','Glomerulosa','Tzone','Fasciculata'))
test_DAR <- melt(test_DAR, id="pseudotime") 


test_DAR$value <- factor(test_DAR$value, levels=c('0','2','16','25','88','1326','3402'))

Plot_DAR=test_DAR %>% 
  ggplot(aes(x =pseudotime,y = value, color = variable, group = variable)) + 
  geom_line() + 
  geom_point() +
  ylab("Variable region") +
  xlab("Cell types")+
  geom_line(size=2)+
  scale_color_manual(values=c('#4E84C4','#D16103'))


Plot_DAR 
PLOT_deg


In [ ]:
library(BSgenome.Mmusculus.UCSC.mm10)
library(Signac)
# # link peaks to genes Glomerulosa
   #  compute gene to link with peaks :

            DE_STERO_Female_tZ=DE_STERO_Female_tZ$gene_names
            DE_STERO_Female_ZF=DE_STERO_Female_ZF$gene_names
            DE_STERO_Female_ZG=DE_STERO_Female_ZG$gene_names
            DE_STERO_Male_tZ=DE_STERO_Male_tZ$gene_names
            DE_STERO_Male_ZF=DE_STERO_Male_ZF$gene_names
            DE_STERO_Male_ZG=DE_STERO_Male_ZG$gene_names


# LINK PEAK with DEG
DefaultAssay(only_STERO)='peaks'
Idents(only_STERO)=only_STERO@meta.data$leiden_merging

            # 1) female TZ 
LINK_DE_STERO_Female_tZ<- LinkPeaks(
      object = only_STERO,
      peak.assay = "peaks",
      expression.assay = "SCT_RUN_STERO",
      genes.use = DE_STERO_Female_tZ,
      )
      # take peaks with significant correlation
      LINK_DE_STERO_Female_tZ=data.frame(Links(LINK_DE_STERO_Female_tZ))
      saveRDS(LINK_DE_STERO_Female_tZ,file = 'LINK_DE_STERO_Female_tZ')
      LINK_Female_tZ_gene_peaks=LINK_DE_STERO_Female_tZ$peak[LINK_DE_STERO_Female_tZ$score>0.1 &LINK_DE_STERO_Female_tZ$pvalue<0.05 ]
# Background
open.peaks <- AccessiblePeaks(only_STERO, idents = c("Female_tZone"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
peaks.matched <- MatchRegionStats(meta.feature = meta.feature[open.peaks, ],query.feature = meta.feature[LINK_Female_tZ_gene_peaks, ],n = 50000)
# Motif recovery 
enriched.motifs_LINK_Female_tZ<- FindMotifs(
object = only_STERO,
features = LINK_Female_tZ_gene_peaks,           # take vector of peak
background = peaks.matched
)
enriched.motifs_LINK_Female_tZ=enriched.motifs_LINK_Female_tZ[enriched.motifs_LINK_Female_tZ$pvalue<0.05,]


        # 2) FEMALE ZF    
LINK_DE_STERO_Female_ZF<- LinkPeaks(
      object = only_STERO,
      peak.assay = "peaks",
      expression.assay = "SCT_RUN_STERO",
      genes.use = DE_STERO_Female_ZF,
      )
          # take peaks with significant correlation
      LINK_DE_STERO_Female_ZF=data.frame(Links(LINK_DE_STERO_Female_ZF))
      saveRDS(LINK_DE_STERO_Female_ZF,file = 'LINK_DE_STERO_Female_ZF')
      LINK_Female_ZF_gene_peaks=LINK_DE_STERO_Female_ZF$peak[LINK_DE_STERO_Female_ZF$score>0.1 &LINK_DE_STERO_Female_ZF$pvalue<0.05 ]

       # Background
open.peaks <- AccessiblePeaks(only_STERO, idents = c("Female_Fasciculata"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
peaks.matched <- MatchRegionStats(meta.feature = meta.feature[open.peaks, ],query.feature = meta.feature[LINK_Female_ZF_gene_peaks, ],n = 50000)
# Motif recovery 
enriched.motifs_LINK_Female_ZF <- FindMotifs(
object = only_STERO,
features = LINK_Female_ZF_gene_peaks  ,           # take vector of peak
background = peaks.matched
)

  # 3 FEMALE ZG 
LINK_DE_STERO_Female_ZG<- LinkPeaks(
      object = only_STERO,
      peak.assay = "peaks",
      expression.assay = "SCT_RUN_STERO",
      genes.use = DE_STERO_Female_ZG,
      )
      # take peaks with significant correlation
      LINK_DE_STERO_Female_ZG=data.frame(Links(LINK_DE_STERO_Female_ZG))
      saveRDS(LINK_DE_STERO_Female_ZG,file = 'LINK_DE_STERO_Female_ZG')
      LINK_Female_ZG_gene_peaks=LINK_DE_STERO_Female_ZG$peak[LINK_DE_STERO_Female_ZG$score>0.1 &LINK_DE_STERO_Female_ZG$pvalue<0.05 ]
      
             # Background
open.peaks <- AccessiblePeaks(only_STERO, idents = c("Female_Glomerulosa"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
peaks.matched <- MatchRegionStats(meta.feature = meta.feature[open.peaks, ],query.feature = meta.feature[LINK_Female_ZG_gene_peaks, ],n = 50000)
# Motif recovery 
enriched.motifs_LINK_Female_ZG <- FindMotifs(
object = only_STERO,
features = LINK_Female_ZG_gene_peaks  ,           # take vector of peak
background = peaks.matched
)

   
      
# ON MALE TZ        
LINK_DE_STERO_Male_tZ<- LinkPeaks(
      object = only_STERO,
      peak.assay = "peaks",
      expression.assay = "SCT_RUN_STERO",
      genes.use = DE_STERO_Male_tZ,
      )

      # take peaks with significant correlation
      LINK_DE_STERO_Male_tZ=data.frame(Links(LINK_DE_STERO_Male_tZ))
      saveRDS(LINK_DE_STERO_Male_tZ,file = 'LINK_DE_STERO_Male_tZ')
      LINK_DE_STERO_Male_tZ_gene_peaks=LINK_DE_STERO_Male_tZ$peak[LINK_DE_STERO_Male_tZ$score>0.1 &LINK_DE_STERO_Male_tZ$pvalue<0.05 ]
      
             # Background
open.peaks <- AccessiblePeaks(only_STERO, idents = c("Male_tZone"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
peaks.matched <- MatchRegionStats(meta.feature = meta.feature[open.peaks, ],query.feature = meta.feature[LINK_DE_STERO_Male_tZ_gene_peaks, ],n = 50000)
# Motif recovery 
enriched.motifs_LINK_Male_tZ <- FindMotifs(
object = only_STERO,
features = LINK_DE_STERO_Male_tZ_gene_peaks  ,           # take vector of peak
background = peaks.matched
)



LINK_DE_STERO_Male_ZF=readRDS(file = 'D:/Stage/TRUTH/LINK_DE_STERO_Male_ZF')

         
only_STERO<- LinkPeaks(
      object = only_STERO,
      peak.assay = "peaks",
      expression.assay = "SCT_RUN_STERO",
      genes.use = DE_STERO_Male_ZF$gene_names,
      )
      # take peaks with significant correlation
      LINK_DE_STERO_Male_ZF=data.frame(Links(LINK_DE_STERO_Male_ZF))
      saveRDS(LINK_DE_STERO_Male_ZF,file = 'LINK_DE_STERO_Male_ZF')
      LINK_DE_STERO_Male_ZF_gene_peaks=LINK_DE_STERO_Male_ZF$peak[LINK_DE_STERO_Male_ZF$score>0.1 &LINK_DE_STERO_Male_ZF$pvalue<0.05 ]
      
             # Background
open.peaks <- AccessiblePeaks(only_STERO, idents = c("Male_Fasciculata"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
peaks.matched <- MatchRegionStats(meta.feature = meta.feature[open.peaks, ],query.feature = meta.feature[LINK_DE_STERO_Male_ZF_gene_peaks, ],n = 10000)
# Motif recovery 
enriched.motifs_LINK_Male_ZF <- FindMotifs(
object = only_STERO,
features = LINK_DE_STERO_Male_ZF_gene_peaks  ,           # take vector of peak
background = peaks.matched,assay = 'peaks'
)

enriched.motifs_LINK_Male_ZF=enriched.motifs_LINK_Male_ZF[enriched.motifs_LINK_Male_ZF$pvalue<0.001,]



          
LINK_DE_STERO_Male_ZG<- LinkPeaks(
      object = only_STERO,
      peak.assay = "peaks",
      expression.assay = "SCT_RUN_STERO",
      genes.use = DE_STERO_Male_ZG,
      )
      # take peaks with significant correlation
      LINK_DE_STERO_Male_ZG=data.frame(Links(LINK_DE_STERO_Male_ZG))
      saveRDS(LINK_DE_STERO_Male_ZG,file = 'LINK_DE_STERO_Male_ZG')
      LINK_DE_STERO_Male_ZG_gene_peaks=LINK_DE_STERO_Male_ZG$peak[LINK_DE_STERO_Male_ZG$score>0.1 &LINK_DE_STERO_Male_ZG$pvalue<0.05 ]
      
             # Background
open.peaks <- AccessiblePeaks(only_STERO, idents = c("Male_Glomerulosa"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
peaks.matched <- MatchRegionStats(meta.feature = meta.feature[open.peaks, ],query.feature = meta.feature[LINK_DE_STERO_Male_ZG_gene_peaks, ],n = 50000)
# Motif recovery 
enriched.motifs_LINK_Male_ZG <- FindMotifs(
object = only_STERO,
features = LINK_DE_STERO_Male_ZG_gene_peaks  ,           # take vector of peak
background = peaks.matched
)




TRY ALL TEST GENE REGULATORY NETWORK

In [ ]:

library(motifmatchr)
library(Signac)
library(Seurat)
library(JASPAR2020)
library(TFBSTools)
library(BSgenome.Mmusculus.UCSC.mm10)
library(patchwork)
LINK_DE_STERO_Female_tZ=readRDS('D:/Stage/TRUTH/LINK_DE_STERO_Female_tZ')

# LINK PEAK with DEG
DefaultAssay(only_STERO)='peaks'
Idents(only_STERO)=only_STERO@meta.data$leiden_merging

            # 1) female TZ 
only_STERO<- LinkPeaks(
      object = only_STERO,
      peak.assay = "peaks",
      expression.assay = "SCT_RUN_STERO",
      genes.use = DE_STERO_Male_ZF$gene_names,
      )
      # take peaks with significant correlation
      LINK_DE_STERO_Female_tZ=data.frame(Links(LINK_DE_STERO_Female_tZ))
  LINK_DE_STERO_Female_ZG=readRDS('D:/Stage/TRUTH/LINK_DE_STERO_Male_ZF')
      LINK_DE_STERO_Female_ZG_peaks=LINK_DE_STERO_Female_ZG$peak[LINK_DE_STERO_Female_ZG$score > 0.1 &LINK_DE_STERO_Female_ZG$pvalue<0.05 ]
      
      
# Background
      
Idents(only_STERO)=only_STERO$leiden_merging
open.peaks <- AccessiblePeaks(only_STERO, idents = c("Male_Fasciculata"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
peaks.matched <- MatchRegionStats(meta.feature = meta.feature[open.peaks, ],query.feature = meta.feature[LINK_DE_STERO_Female_ZG_peaks, ],n = 1000)
# Motif recovery 
enriched.motifs_LINK_Female_tZ<- FindMotifs(
object = only_STERO,
features = LINK_DE_STERO_Female_ZG_peaks,           # take vector of peak
background = peaks.matched,assay = 'peaks'
)

enriched.motifs_LINK_Female_tZ=enriched.motifs_LINK_Female_tZ[enriched.motifs_LINK_Female_tZ$pvalue<0.05,]

# 1) prepare motif from jaspar 
pfm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
)

# take Granges object from only_STERO
motif.matrix <- CreateMotifMatrix(
  features =StringToGRanges(LINK_DE_STERO_Female_ZG_peaks ,sep = c("-", "-")),  # GET RIGHT PEAKS
  pwm = pfm,
  genome = BSgenome.Mmusculus.UCSC.mm10,score = F
)


motif.matrix =as.data.frame(motif.matrix)
dim(motif.matrix)
motif.matrix=motif.matrix[, intersect( colnames(motif.matrix),enriched.motifs_LINK_Female_tZ$motif)]


motif.matrix$peaks=rownames(motif.matrix)
motif.matrix$peaks = (gsub("\\.", "-", motif.matrix$peaks))

motif.matrix=left_join(enriched.motifs_LINK_Female_tZ,motif.matrix, by = c("peak"="peaks"))
motif.matrix=motif.matrix[!apply(is.na(motif.matrix[,11:length(motif.matrix)]), 1, all),]

######
var_names <- enriched.motifs_LINK_Female_tZ %>% 
  # new variable names, old variable names
  select(motif.name, motif) %>% 
  deframe()

df_updated <-  motif.matrix %>% 
  rename(!!!var_names)

# find all tf expressed in at least 10% of the cells 
Idents(only_STERO)=only_STERO$leiden_merging
all_male_ZF=FindMarkers(only_STERO,assay = 'SCT_RUN_STERO',logfc.threshold = 0,min.pct = 0.2,ident.1 ="Male_Fasciculata" )  
all_male_ZF$gene_names <- rownames(all_male_ZF)
all_male_ZF$val2 <- mouse_TF$V1[match(all_male_ZF$gene_names, mouse_TF$V1)]


drops=c(na.omit(c(toupper(all_male_ZF$val2),all_male_ZF$val2))) # vector of TF present in 20% (en maj et minuscule)


retain=as.vector(drops)

new_df <- df_updated[,which(names(df_updated) %in% names(df_updated)[1:10] | names(df_updated) %in% retain)]

new_df <- new_df[new_df$Ar !=0,]


idents.plot <- c(levels(only_STERO))


DefaultAssay(only_STERO)='peaks'
 CoveragePlot(
  object = only_STERO,
  region = "chr8-84634863-84635340",
  expression.assay = "SCT_RUN_STERO",features = c('Junb'),
  idents = idents.plot,
  extend.upstream = 5000,
  extend.downstream = 5000,links = F
  
)

Idents(only_STERO)=only_STERO$two_conditions
idents.plot <- c("Male_tZone_male","Male_Fasciculata_male","Male_Glomerulosa_male","Capsular_cells_male",   "Female_Glomerulosa_female","Female_tZone_female","Female_Fasciculata_female", "Capsular_cells_female")

Idents(only_STERO)=only_STERO$leiden_merging

CoveragePlot(only_STERO, features = "B4galt5",region ="B4galt5" )


# MOTIF RECOVERY ON LINK PEAKS

In [ ]:


  open.peaks <- AccessiblePeaks(multiomics, idents = c("Female Fasciculata"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
      
      meta.feature <- GetAssayData(multiomics, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
      peaks.matched <- MatchRegionStats(
        meta.feature = meta.feature[open.peaks, ],
        query.feature = meta.feature[peak_LINK_F_F, ],
        n = 50000
      )



enriched.motifs_male <- FindMotifs(
  object = multiomics,
  features = peak_LINK_F_M  ,           # take vector of peak
   background = peaks.matched
)

enriched.motifs_female <- FindMotifs(
  object = multiomics,
  features = peak_LINK_F_F  ,           # take vector of peak
   background = peaks.matched
)



View(enriched.motifs_female)
View(enriched.motifs_male)

enriched.motifs_male=enriched.motifs_male[enriched.motifs_male$fold.enrichment>2 &enriched.motifs_male$pvalue<0.05 ,]

enriched.motifs_female=enriched.motifs_female[enriched.motifs_female$fold.enrichment>2 &enriched.motifs_female$pvalue<0.05 ,]

female_up = enriched.motifs_female$motif[order(enriched.motifs_female$fold.enrichment, decreasing = TRUE)]
female_up=head(female_up)

male_up = enriched.motifs_male$motif[order(enriched.motifs_male$fold.enrichment, decreasing = TRUE)]
male_up=head(male_up)

DefaultAssay(multiomics) <- 'chromvar'

p2 <- FeaturePlot(
  object = multiomics,
  features = male_up,
  min.cutoff = 'q10',
  max.cutoff = 'q90',
  pt.size = 0.1,reduction = 'wnn.umap'
)


# differential peaks accesibility AND motif accessibility

In [ ]:

Idents(only_STERO)=only_STERO@meta.data$leiden_merging
DefaultAssay(only_STERO)='peaks'
vector_clusters_F=c( "Female_Fasciculata","Female_Glomerulosa","Female_tZone")
vector_clusters_M=c( "Male_Fasciculata", "Male_Glomerulosa","Male_tZone")


for( i in 1:length(vector_clusters_F)){ 
  assign(paste0('DA_STERO_',vector_clusters_F[i]),FindMarkers(only_STERO,
                                                        ident.1 =vector_clusters_F[i],
                                                        ident.2 = vector_clusters_M[i],
                                                        assay = 'peaks',
                                                        only.pos = F, test.use = 'LR',
                                                        min.pct = 0.05,
                                                        latent.vars = 'nCount_peaks'))
}



# FASCICULATA 
DA_STERO_Female_Fasciculata$peaks=rownames(DA_STERO_Female_Fasciculata)
DA_STERO_Female_Fasciculata <- DA_STERO_Female_Fasciculata[DA_STERO_Female_Fasciculata$p_val_adj < 0.05,]

peaks_Fem_FASC=DA_STERO_Female_Fasciculata$peaks[DA_STERO_Female_Fasciculata$p_val_adj < 0.05&DA_STERO_Female_Fasciculata$avg_log2FC>0.1]
peaks_Male_FASC=DA_STERO_Female_Fasciculata$peaks[DA_STERO_Female_Fasciculata$p_val_adj < 0.05&DA_STERO_Female_Fasciculata$avg_log2FC< -0.1]


# GLOMERULOSA 
DA_STERO_Female_Glomerulosa$peaks=rownames(DA_STERO_Female_Glomerulosa)
DA_STERO_Female_Glomerulosa <- DA_STERO_Female_Glomerulosa[DA_STERO_Female_Glomerulosa$p_val_adj < 0.05,]

peaks_Fem_GLO=DA_STERO_Female_Glomerulosa$peaks[DA_STERO_Female_Glomerulosa$p_val_adj < 0.05&DA_STERO_Female_Glomerulosa$avg_log2FC>0.1]
peaks_Male_GLO=DA_STERO_Female_Glomerulosa$peaks[DA_STERO_Female_Glomerulosa$p_val_adj < 0.05&DA_STERO_Female_Glomerulosa$avg_log2FC< -0.1]


#  Tzone
DA_STERO_Female_tZone$peaks=rownames(DA_STERO_Female_tZone)
DA_STERO_Female_tZone <- DA_STERO_Female_tZone[DA_STERO_Female_tZone$p_val_adj < 0.05,]

peaks_Fem_tzone=DA_STERO_Female_tZone$peaks[DA_STERO_Female_tZone$p_val_adj < 0.05&DA_STERO_Female_tZone$avg_log2FC>0.1]
peaks_Male_tzone=DA_STERO_Female_tZone$peaks[DA_STERO_Female_tZone$p_val_adj < 0.05&DA_STERO_Female_tZone$avg_log2FC< -0.1]


# Background and motif enrichement :
              
              # Fasciculata : 
                      open.peaks <- AccessiblePeaks(only_STERO, idents = c("Female_Fasciculata","Male_Fasciculata"),assay = 'peaks') 
                      
                      meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")
                      # Female 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Fem_FASC, ],
                            n = 50000
                            )
                              
                            enrich_motif_Fem_FASC <- FindMotifs(object = only_STERO,features = peaks_Fem_FASC,background = peaks.matched)
                            enrich_motif_Fem_FASC <- enrich_motif_Fem_FASC[enrich_motif_Fem_FASC$pvalue<0.05,]
                                          
                      # Male 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Male_FASC, ],
                            n = 50000
                            )
                            enrich_motif_Male_FASC <- FindMotifs(object = only_STERO,features = peaks_Male_FASC,background = peaks.matched)
                            enrich_motif_Male_FASC <- enrich_motif_Male_FASC[enrich_motif_Male_FASC$pvalue<0.05,]

              # Glomerulosa : 
                      open.peaks <- AccessiblePeaks(only_STERO, idents = c("Male_Glomerulosa","Female_Glomerulosa"),assay = 'peaks') 
                      
                      meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")
                      # Female 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Fem_GLO, ],
                            n = 50000
                            )
                              
                            enrich_motif_Fem_GLO<- FindMotifs(object = only_STERO,features = peaks_Fem_GLO,background = peaks.matched)
                            enrich_motif_Fem_GLO <- enrich_motif_Fem_GLO[enrich_motif_Fem_GLO$pvalue<0.05,]

                                          
                      # Male 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Male_GLO, ],
                            n = 50000
                            )
                            enrich_motif_Male_GLO <- FindMotifs(object = only_STERO,features = peaks_Male_GLO,background = peaks.matched)
                            enrich_motif_Male_GLO <- enrich_motif_Male_GLO[enrich_motif_Male_GLO$pvalue<0.05,]

                            
                # TZONE  : 
                      open.peaks <- AccessiblePeaks(only_STERO, idents = c("Female_tZone","Male_tZone"),assay = 'peaks') 
                      
                      meta.feature <- GetAssayData(only_STERO, assay = "peaks", slot = "meta.features")
                      # Female 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Fem_tzone, ],
                            n = 50000
                            )
                              
                            enrich_motif_Fem_TZONE<- FindMotifs(object = only_STERO,features = peaks_Fem_tzone,background = peaks.matched)
                            enrich_motif_Fem_TZONE <- enrich_motif_Fem_TZONE[enrich_motif_Fem_TZONE$pvalue<0.05,]
                                          
                      # Male 
                            peaks.matched <- MatchRegionStats(
                            meta.feature = meta.feature[open.peaks, ],
                            query.feature = meta.feature[peaks_Male_tzone, ],
                            n = 50000
                            )
                            enrich_motif_Male_TZONE <- FindMotifs(object = only_STERO,features = peaks_Male_tzone,background = peaks.matched)
                            enrich_motif_Male_TZONE <- enrich_motif_Male_TZONE[enrich_motif_Male_TZONE$pvalue<0.05,]
                            
                            

                          
                            


# ON SUB CLUSTERS

Idents(only_STERO)=only_STERO@meta.data$new_ident
DefaultAssay(only_STERO)='peaks'
vector_clusters_F=c( "Female_Fasciculata_1", "Male_Fasciculata_1")
vector_clusters_M=c( "Female_Fasciculata_2", "Male_Fasciculata_2")

for( i in 1:length(vector_clusters_F)){
  assign(paste0('DA_STERO_SUB',vector_clusters_F[i]),FindMarkers(only_STERO,
                                                        ident.1 =vector_clusters_F[i],
                                                        ident.2 = vector_clusters_M[i],
                                                        assay = 'peaks',
                                                        only.pos = F, test.use = 'LR',
                                                        min.pct = 0.05,
                                                        latent.vars = 'nCount_peaks'))
}



DA_STERO_SUBFemale_Fasciculata_1$peaks=rownames(DA_STERO_SUBFemale_Fasciculata_1)
DA_STERO_SUBFemale_Fasciculata_1 <- DA_STERO_SUBFemale_Fasciculata_1[DA_STERO_SUBFemale_Fasciculata_1$p_val_adj < 0.05,]


DA_STERO_SUBMale_Fasciculata_1$peaks=rownames(DA_STERO_SUBMale_Fasciculata_1)
DA_STERO_SUBMale_Fasciculata_1<- DA_STERO_SUBMale_Fasciculata_1[DA_STERO_SUBMale_Fasciculata_1$p_val_adj < 0.05,]



# do a bar plot of total number of DE AND DA between glomerulosa and fasciculata with and without sex

# run it on homer for finding differentially MOTIF and TF recruitment 



# Motif get jaspar database

In [ ]:

# 1) prepare motif from jaspar 
pfm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
)

# take Granges object from only_STERO
motif.matrix <- as.data.frame( CreateMotifMatrix(
  features =StringToGRanges( DA_STERO_Female_Fasciculata$peaks[DA_STERO_Female_Fasciculata$avg_log2FC< - 0.25 & DA_STERO_Female_Fasciculata$p_val_adj<0.01],sep = c("-", "-")),  # GET RIGHT PEAKS
  pwm = pfm,
  genome = BSgenome.Mmusculus.UCSC.mm10,score = F
))


motif.matrix =as.data.frame(motif.matrix)
dim(motif.matrix)
motif.matrix=motif.matrix[, intersect( colnames(motif.matrix),enrich_motif_Male_FASC$motif)] # take enriched motif 


motif.matrix$peaks=rownames(motif.matrix)
motif.matrix$peaks = (gsub("\\.", "-", motif.matrix$peaks))

motif.matrix=left_join(enrich_motif_Fem_FASC,motif.matrix, by = c("peak"="peaks"))
motif.matrix=motif.matrix[!apply(is.na(motif.matrix[,11:length(motif.matrix)]), 1, all),]

######
var_names <- enriched.motifs_LINK_Female_tZ %>% 
  # new variable names, old variable names
  select(motif.name, motif) %>% 
  deframe()

df_updated <-  motif.matrix %>% 
  rename(!!!var_names)



In [ ]:
library(Seurat)
library(Signac)
library(TFBSTools)
library(rlist)
library(BSgenome.Mmusculus.UCSC.mm10)
library(JASPAR2020)

DefaultAssay(only_STERO)='peaks'

# 1) prepare motif from jaspar 
pfm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
)
  
  # 2) add motif information
      # construct a Motif object and adds it to our multiomics dataset, along with other information such as the base composition                 of each peak. 
only_STERO <- AddMotifs(
  object = only_STERO,
  genome = BSgenome.Mmusculus.UCSC.mm10,
  pfm = pfm
)

only_STERO <- RegionStats(
  only_STERO, 
  genome = BSgenome.Mmusculus.UCSC.mm10,
  assay = 'peaks')  


 # Run chromvar  

only_STERO <- RunChromVAR(
  object = only_STERO,
  genome = BSgenome.Mmusculus.UCSC.mm10,
  assay = "peaks"
)


####DATA FRAME annotation add ALL ID 

In [ ]:
# GSEA Fast comparison and dataframe annotation ENSEMBLID / ENTREZID (Kegg)

# annotate according to Ensembl and kegg  DATABASE
library(AnnotationHub)
library(dplyr)
library(clusterProfiler)
library(fgsea)
library(ggplot2)
library(enrichplot)
library(org.Mm.eg.db)
library(DOSE)
library(biomaRt)
# get mouse genes mm10
ensembl <- useMart("ensembl", dataset="mmusculus_gene_ensembl")

                                              # DE_STERO_Female_tZ
# get ensembl ID
gene_IDs <- getBM(filters= 'mgi_symbol', attributes= c("ensembl_gene_id",'mgi_symbol','entrezgene_id','description'),values = DE_STERO_Female_tZ$gene_names, mart= ensembl)
# Add into DE DATAFRAME
DE_STERO_Female_tZ=left_join(DE_STERO_Female_tZ,gene_IDs, by = c("gene_names"="mgi_symbol"))
# check duplicate value 
DE_STERO_Female_tZ$gene[duplicated(DE_STERO_Female_tZ$gene_names)]
# remove duplicate value
DE_STERO_Female_tZ=DE_STERO_Female_tZ[!duplicated(DE_STERO_Female_tZ$gene_names), ]
write.table(DE_STERO_Female_tZ$gene_names,file = 'DE_STERO_Female_tZ.txt',quote = F,row.names = F,col.names = F)

                                              # DE_STERO_Male_tZ
gene_IDs <- getBM(filters= 'mgi_symbol', attributes= c("ensembl_gene_id",'mgi_symbol','entrezgene_id','description'),values = DE_STERO_Male_tZ$gene_names, mart= ensembl)
# Add into DE DATAFRAME
DE_STERO_Male_tZ=left_join(DE_STERO_Male_tZ,gene_IDs, by = c("gene_names"="mgi_symbol"))
# check duplicate value 
DE_STERO_Male_tZ$gene[duplicated(DE_STERO_Male_tZ$gene_names)]
# remove duplicate value
DE_STERO_Male_tZ=DE_STERO_Male_tZ[!duplicated(DE_STERO_Male_tZ$gene_names), ]
write.table(DE_STERO_Male_tZ$gene_names,file = 'DE_STERO_Male_tZ.txt',quote = F,row.names = F,col.names = F)




# trajectory 

In [ ]:
# re find clusters with high resolution 
#only_STERO<- FindClusters(only_STERO, graph.name = "wsnn", algorithm = 1, resolution = 1, verbose = FALSE)
DimPlot(only_STERO,reduction  = 'wnn.umap',label = T,pt.size = 2)
# create new IDENTS FOR SEPARING CAPSULAR CELL MALE AND FEMALE
Idents(only_STERO)=only_STERO$merging_ident
only_STERO$merging_ident_classed_by_sex <- paste(Idents(only_STERO), only_STERO$identity, sep = "_")
levels(only_STERO)

new.cluster.ids <- c("Male_Glomerulosa_male"="Male_Glomerulosa",
                      "Male_Fasciculata_male"= "Male_Fasciculata", 
                      "Capsular_Cells_male" = "Capsular_Cells_male",
                     "Female_Glomerulosa_male"="Female_Glomerulosa",
                     "Female_Glomerulosa_female"="Female_Glomerulosa",
                     "Female_Fasciculata_female" ="Female_Fasciculata",
                     "Capsular_Cells_female"='Capsular_Cells_female' ,
                     "Male_Glomerulosa_female" ="Male_Glomerulosa",
                     "Male_Fasciculata_female"='Male_Fasciculata')



names(new.cluster.ids) <- levels(only_STERO)
only_STERO <- RenameIdents(only_STERO, new.cluster.ids)
DimPlot(pbmc, reduction = "umap", label = TRUE, pt.size = 0.5) + NoLegend()
only_STERO$merging_ident_classed_by_sex=Idents(only_STERO)


# END 

# RUN  DE ==>

mouse_TF=read.table('C:/Users/User/Documents/aposimz/mm_mgi_tfs.txt')

mouse_TF=read.table('C:/Users/User/Documents/aposimz/mm_mgi_tfs.txt')
chrx_y=as.data.frame(chrX_and_Y)


# Findmarkers on the new idents 
DefaultAssay(remove_only_stero)='SCT_RUN_STERO'   
Idents(remove_only_stero)=remove_only_stero$leiden_annotated
# CLUSTERS  1 - MALE 
Male_Glomerulosa = FindMarkers(remove_only_stero,ident.1 ="Male_Glomerulosa",ident.2 = c("Male_tZone","Female_Glomerulosa"),assay = 'SCT_RUN_STERO')
Male_Glomerulosa$gene_names=rownames(Male_Glomerulosa)
Male_Glomerulosa$val2 <- mouse_TF$V1[match(Male_Glomerulosa$gene_names, mouse_TF$V1)]
Male_Glomerulosa$sex <- chrx_y$chrX_and_Y[match(Male_Glomerulosa$gene_names, chrx_y$chrX_and_Y)]
Male_Glomerulosa = Male_Glomerulosa[Male_Glomerulosa$p_val_adj<0.05&Male_Glomerulosa$avg_log2FC>0.5,]


Male_tZone = FindMarkers(remove_only_stero,ident.1 ='Male_tZone',ident.2 = c("Male_Glomerulosa","Male_Fasciculata_1","Female_tZone"),assay = 'SCT_RUN_STERO')
Male_tZone$gene_names=rownames(Male_tZone)
Male_tZone$val2 <- mouse_TF$V1[match(Male_tZone$gene_names, mouse_TF$V1)]
Male_tZone$sex <- chrx_y$chrX_and_Y[match(Male_tZone$gene_names, chrx_y$chrX_and_Y)]
Male_tZone = Male_tZone[Male_tZone$p_val_adj<0.05&Male_tZone$avg_log2FC>0.5,]


Male_Fasciculata_1 = FindMarkers(remove_only_stero,ident.1 ='Male_Fasciculata_1',ident.2 = c("Male_tZone","Male_Fasciculata_2","Female_Fasciculata_1"),assay = 'SCT_RUN_STERO')
Male_Fasciculata_1$gene_names=rownames(Male_Fasciculata_1)
Male_Fasciculata_1$val2 <- mouse_TF$V1[match(Male_Fasciculata_1$gene_names, mouse_TF$V1)]
Male_Fasciculata_1$sex <- chrx_y$chrX_and_Y[match(Male_Fasciculata_1$gene_names, chrx_y$chrX_and_Y)]
Male_Fasciculata_1 = Male_Fasciculata_1[Male_Fasciculata_1$p_val_adj<0.05&Male_Fasciculata_1$avg_log2FC>0.5,]



Female_Glomerulosa = FindMarkers(remove_only_stero,ident.1 ='Female_Glomerulosa',ident.2 = c("Female_tZone",'Male_Glomerulosa'),assay = 'SCT_RUN_STERO')
Female_Glomerulosa$gene_names=rownames(Female_Glomerulosa)
Female_Glomerulosa$val2 <- mouse_TF$V1[match(Female_Glomerulosa$gene_names, mouse_TF$V1)]
Female_Glomerulosa$sex <- chrx_y$chrX_and_Y[match(Female_Glomerulosa$gene_names, chrx_y$chrX_and_Y)]
Female_Glomerulosa = Female_Glomerulosa[Female_Glomerulosa$p_val_adj<0.05&Female_Glomerulosa$avg_log2FC>0.5,]


getwd()
Female_tZone = FindMarkers(remove_only_stero,ident.1 ='Female_tZone',ident.2 = c("Female_Glomerulosa","Female_Fasciculata_1","Male_tZone"),assay = 'SCT_RUN_STERO')
Female_tZone$gene_names=rownames(Female_tZone)
Female_tZone$val2 <- mouse_TF$V1[match(Female_tZone$gene_names, mouse_TF$V1)]
Female_tZone$sex <- chrx_y$chrX_and_Y[match(Female_tZone$gene_names, chrx_y$chrX_and_Y)]
Female_tZone = Female_tZone[Female_tZone$p_val_adj<0.05&Female_tZone$avg_log2FC>0.5,]


Female_Fasciculata_1 = FindMarkers(remove_only_stero,ident.1 ='Female_Fasciculata_1',ident.2 = c("Female_tZone","Female_Fasciculata_2","Male_Fasciculata_1"),assay = 'SCT_RUN_STERO')
Female_Fasciculata_1$gene_names=rownames(Female_Fasciculata_1)
Female_Fasciculata_1$val2 <- mouse_TF$V1[match(Female_Fasciculata_1$gene_names, mouse_TF$V1)]
Female_Fasciculata_1$sex <- chrx_y$chrX_and_Y[match(Female_Fasciculata_1$gene_names, chrx_y$chrX_and_Y)]
Female_Fasciculata_1 = Female_Fasciculata_1[Female_Fasciculata_1$p_val_adj<0.05&Female_Fasciculata_1$avg_log2FC>0.5,]


Female_Fasciculata_2 = FindMarkers(remove_only_stero,ident.1 ='Female_Fasciculata_2',ident.2 = c('Female_Fasciculata_1',"Male_Fasciculata_2"),assay = 'SCT_RUN_STERO')
Female_Fasciculata_2$gene_names=rownames(Female_Fasciculata_2)
Female_Fasciculata_2$val2 <- mouse_TF$V1[match(Female_Fasciculata_2$gene_names, mouse_TF$V1)]
Female_Fasciculata_2$sex <- chrx_y$chrX_and_Y[match(Female_Fasciculata_2$gene_names, chrx_y$chrX_and_Y)]
Female_Fasciculata_2 = Female_Fasciculata_2[Female_Fasciculata_2$p_val_adj<0.05&Female_Fasciculata_2$avg_log2FC>0.5,]


Male_Fasciculata_2 = FindMarkers(remove_only_stero,ident.1 ='Male_Fasciculata_2',ident.2 = c("Male_Fasciculata_1","Female_Fasciculata_2"),assay = 'SCT_RUN_STERO')
Male_Fasciculata_2$gene_names=rownames(Male_Fasciculata_2)
Male_Fasciculata_2$val2 <- mouse_TF$V1[match(Male_Fasciculata_2$gene_names, mouse_TF$V1)]
Male_Fasciculata_2$sex <- chrx_y$chrX_and_Y[match(Male_Fasciculata_2$gene_names, chrx_y$chrX_and_Y)]
Male_Fasciculata_2 = Male_Fasciculata_2[Male_Fasciculata_2$p_val_adj<0.05&Male_Fasciculata_2$avg_log2FC>0.5,]


# CAPSULAR CELL BY CONDITIONS 
Idents(only_STERO)=only_STERO$leiden_merging
only_STERO$two_conditions <- paste(Idents(only_STERO), only_STERO$identity, sep = "_")
Idents(only_STERO) <- only_STERO$two_conditions


Capsular_cells_female = FindMarkers(only_STERO,ident.1 ='Capsular_cells_female',ident.2 = c('Capsular_cells_male'),logfc.threshold = 0.1,assay = 'SCT_RUN_STERO')
Capsular_cells_female$gene_names=rownames(Capsular_cells_female)
Capsular_cells_female$val2 <- mouse_TF$V1[match(Capsular_cells_female$gene_names, mouse_TF$V1)]
Capsular_cells_female$sex <- chrx_y$chrX_and_Y[match(Capsular_cells_female$gene_names, chrx_y$chrX_and_Y)]
Capsular_cells_female = Capsular_cells_female[Capsular_cells_female$p_val_adj<0.05,]


Capsular_cells_male = FindMarkers(remove_only_stero,ident.1 ='Capsular_cells_male',ident.2 = c('Capsular_cells_female'))
Capsular_cells_male$gene_names=rownames(Capsular_cells_male)
Capsular_cells_male$val2 <- mouse_TF$V1[match(Capsular_cells_male$gene_names, mouse_TF$V1)]
Capsular_cells_male$sex <- chrX_and_Y$chrX_and_Y[match(Capsular_cells_male$gene_names, chrX_and_Y$chrX_and_Y)]
Capsular_cells_male = Capsular_cells_male[Capsular_cells_male$p_val_adj<0.05,]



Capsular_cells_male = FindMarkers(remove_only_stero,ident.1 ='Capsular_cells_male',ident.2 = c('Capsular_cells_female'))


Capsular_cells_female = FindMarkers(only_STERO,ident.1 ='Capsular_cells_female',ident.2 = c('Capsular_cells_male'),assay = 'peaks', test.use = 'LR', latent.vars = 'nCount_peaks', min.pct = 0.05)



for( i in 1:length(vector_clusters_F)){ 
  assign(paste0('DA_STERO_',vector_clusters_F[i]),FindMarkers(only_STERO,
                                                        ident.1 =vector_clusters_F[i],
                                                        ident.2 = vector_clusters_M[i],
                                                        assay = 'peaks',
                                                        only.pos = F, test.use = 'LR',
                                                        min.pct = 0.05,
                                                        latent.vars = 'nCount_peaks'))
}






Idents(only_STERO) <- only_STERO$leiden_annotated


can pass if already run 

In [ ]:
library(Seurat)
library(Signac)
library(TFBSTools)
library(rlist)
library(BSgenome.Mmusculus.UCSC.mm10)
library(JASPAR2020)

DefaultAssay(only_STERO)='peaks'

# 1) prepare motif from jaspar 
pfm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
)
  
  # 2) add motif information
      # construct a Motif object and adds it to our multiomics dataset, along with other information such as the base composition                 of each peak. 
only_STERO <- AddMotifs(
  object = only_STERO,
  genome = BSgenome.Mmusculus.UCSC.mm10,
  pfm = pfm
)

only_STERO <- RegionStats(
  only_STERO, 
  genome = BSgenome.Mmusculus.UCSC.mm10,
  assay = 'peaks')  


 # Run chromvar  

only_STERO <- RunChromVAR(
  object = only_STERO,
  genome = BSgenome.Mmusculus.UCSC.mm10,
  assay = "peaks"
)


# plot motif along pseudotime

In [ ]:

library(dittoSeq)
Idents(only_STERO) <- only_STERO$two_conditions
Idents(only_STERO) <- only_STERO$leiden_merging

Idents(only_STERO)=only_STERO$two_conditions



#Female

issou=subset(only_STERO,idents = c("Female_Glomerulosa_female","Female_tZone_female","Female_Fasciculata_female","Male_Glomerulosa_male","Male_tZone_male","Male_Fasciculata_male"))

female_traj=read.csv(file = 'E:/TRUTH/Pseudotime_female.csv')
names(female_traj)[1]='index'
male_traj=read.csv(file = 'E:/TRUTH/Pseudotime_male.csv')
names(male_traj)[1]='index'
out=rbind(x = female_traj,male_traj)
rownames(out)=out$index


xd$new_issoux <- out$Pseudotime
xd$Pseudotime <- out$Pseudotime[match(xd$index, out$index)]


issox= c('identity','val2')

names(xd@meta.data)[46]='Cell types'
names(xd@meta.data)[4]='Sex'



dittoHeatmap(xd,genes=c('Fat3','Ly6d','Lynx1','Crem','Fam129a','Slc25a3','Pde10a','Akr1b7','Srd5a2','Slc25a30','Abcb1b','Aldh1a1','Runx2','Agtr1b','Vsnl1','Cdh6','Ecrg4','Pip5k1b','Elmo1','Efna5','Ror1','Adh1','Pdzrn3','A530020G20Rik','Akr1d1','Lama4','Cyp2f2')
             ,show_colnames = F
             ,order.by = c('Sex','Pseudotime')
             ,annot.by = c('Cell types','Sex','Pseudotime')
             ,assay = 'SCT_RUN_STERO'
             ,slot = 'scale.data'
             ,cluster_rows=FALSE
             ,complex = TRUE,
heatmap.colors=colorRampPalette(c("royalblue", "white", "red"))(50),
breaks=seq(-4, 4, length.out=51)
)

    breaks = seq(0, 0.65, length.out = 26)



heatmap.colors=colorRampPalette(c("blue", "white", "red"))(50),
breaks=seq(-4, 4, length.out=50)








genix=c("Agtr1b","Vsnl1","Elmo1","Igfbp7","Agtr1b","Vsnl1","Pde7a" ,"Pdzrn3","Akr1d1" ,"Prr16","Il1rapl2") 

breaks = seq(-6, 6, 0.06))


genix=c("Agtr1b","Vsnl1","Elmo1","Igfbp7","Agtr1b","Vsnl1","Pde7a" ,"Pdzrn3","Akr1d1" ,"Prr16","Il1rapl2") 
dittoHeatmap(Male,order.by = 'new_X1z',annot.by = 'leiden_merging',assay = 'SCT_RUN_STERO',genes=all_markers_MG,show_colnames = F,  complex = TRUE,use_raster = TRUE,scale = "none",cluster_cols = FALSE,heatmap.colors = colorRampPalette(c("blue","firebrick4","firebrick4","firebrick4"))(4),scaled.to.max = T,slot = 'scale.data')



dittoHeatmap(x,order.by = 'new_X1z',annot.by = 'leiden_merging',assay = 'SCT_RUN_STERO',genes=all_markers_MG,show_colnames = F,  complex = TRUE,use_raster = TRUE,scale = "none",cluster_cols = FALSE,heatmap.colors = colorRampPalette(c("blue","firebrick4","firebrick4","firebrick4"))(4),scaled.to.max = T,slot = 'scale.data')




all_markers_FEMALE=c('Agtr1b','Vsnl1','cdh6','Ecgr4',
                    'Pip5k1b','Elmo1','Efna5','Ror1',
                    'Adh1','Pdzrn3','A530020G20Rik',
                    'Akr1d1','Cyp2f2','Lama4','Cyp2f2'
                    )

Female_tZone_genes=Female_tZone$gene_names[Female_tZone$p_val_adj<0.05]
Male_GLOMER8ULOSA_genes=Female_Glomerulosa$gene_names[Female_Glomerulosa$p_val_adj<0.05]
Male_tZone_genes=Female_Fasciculata_2$gene_names[Female_Fasciculata_2$p_val_adj<0.05&Female_Fasciculata_2$avg_log2FC>1]
Male_tZone_genes=Male_tZone$gene_names[Male_tZone$p_val_adj<0.05,]
Male_tZone_genes=Male_tZone$gene_names[Male_tZone$p_val_adj<0.05]




library(ComplexHeatmap)
library(circlize)



In [ ]:
library(reticulate)
py$anndata['paga']


# add embending 
embedding <- py$anndata$obsm["X_draw_graph_fr"]
rownames(embedding) <- py$anndata$obs_names$to_list()
colnames(embedding) <- c("FA_1", "FA_2")
only_STERO[["FA"]] <- CreateDimReducObject(embedding, key = "FA_")
# ad dmeta data
only_STERO <- AddMetaData(only_STERO, py$anndata$obs)

# FIND ALL MARKERS FOR LEIDEN RE CLUSTERING 
Idents(only_STERO)=only_STERO$leiden
allclusters=FindAllMarkers(only_STERO)
write.csv(allclusters,file = 'allclusters.csv',sep = ,)
write.csv(data, "data.csv") 
DimPlot(only_STERO,reduction = 'FA')|DimPlot(only_STERO,reduction = 'FA',group.by = 'identity')
setwd('D:/Stage/')
# DEG
all_stero_diff=FindMarkers(only_STERO,ident.1 = c('1','7','3','5'),ident.2 = c('2','6','0','4'),split.by = 'identity')

all_stero_diff=all_stero_diff[all_stero_diff$p_val_adj<0.05,]
all_stero_diff$gene_names=rownames(all_stero_diff)

# male
male_diff=all_stero_diff[all_stero_diff$avg_log2FC<0,]
male_diff$TF <- mouse_TF$V1[match(male_diff$gene_names, mouse_TF$V1)]
male_diff$sex <- chrx_y$chrX_and_Y[match(male_diff$gene_names, chrx_y$chrX_and_Y)]

# female
female_diff=all_stero_diff[all_stero_diff$avg_log2FC > 0,]
female_diff$TF <- mouse_TF$V1[match(female_diff$gene_names, mouse_TF$V1)]
female_diff$sex <- chrx_y$chrX_and_Y[match(female_diff$gene_names, chrx_y$chrX_and_Y)]


#
all_genes=subset(only_STERO,idents ="Capsular_Cells" ,invert=T,)
virus=FindAllMarkers(all_genes,logfc.threshold=0,min.pct=0.05,assay = 'SCT_RUN_STERO')

    X1iX1=FindMarkers(only_STERO,ident.1 = c('1','7','3','5'),ident.2 = c('2','6','0','4'),min.pct = 0.05,logfc.threshold = 0,assay = 'SCT_RUN_STERO')
    



In [ ]:
library(lemon)
library(stringr)
library(data.table)
library(stringr)
library(ggplot2)

# 1) loading objects
# trajectory coordinates generated by scan^py
trajFA = read.csv("D:/Stage/Pseudotime_RNA.csvFA2_components_ATAC.csv")
colnames(trajFA) = c("index","FA1","FA2")

minmaxthis = function(x) { 
  (x - min(x))/(max(x) - min(x))
}

trajFA=only_STERO@reductions[["FA"]]@cell.embeddings

# plot figure 
TFsnames = c("MA0007.3", "MA0113.3", "MA0727.1", "MA1655.1" ,"MA0102.4", "MA0836.2","MA1636.1")
plot_list = list() 

    for (x in TFsnames) {
      print(x)
      
      DefaultAssay(only_STERO) = "chromvar"
      p10 = FeaturePlot(
        object = only_STERO,
        features = str_replace_all(x, fixed("_"), "-"),
        min.cutoff = 'q0',
        max.cutoff = 'q100',
        pt.size = 0.3)
      
      chromvardf = p10$data
      chromvardf$index = rownames(chromvardf)
      
      mergedFAchromvar = merge(chromvardf, trajFA, by = "index")
      
      
      TFactplot = ggplot(mergedFAchromvar, aes( x = FA1, y = FA2, 
                                                colour = minmaxthis(mergedFAchromvar[,5]))) + 
        geom_point(size = 1, alpha = 1) + 
        scale_color_gradient(low = "gray93",high = "red1",breaks = seq(0,1,0.2))+
        xlab("FA1") + ylab("FA2") +
        theme(panel.border=element_blank(), 
              axis.line = element_line(colour = 'black', size = 1), 
              axis.ticks = element_blank(),
              axis.text = element_blank(), 
              panel.grid.major = element_blank(),
              panel.grid.minor = element_blank(),
              panel.background = element_blank(),
              legend.key=element_blank(),
              legend.title=element_blank(),
              legend.text=element_text(size=12)) + 
        # guides(color = guide_legend(override.aes max.cutoff= list(size=5))) + 
        coord_capped_cart(bottom='right', left='none')  + 
      ggtitle(x)
      TFactplot
      
      
      print(TFactplot)
  
}



















x=FindMarkers(only_STERO,ident.1 ="Female_Fasciculata_1",ident.2 = "Female_Fasciculata_2",assay = 'chromvar')



TFsnames = c("MA0007.3", "MA0113.3", "MA0727.1", 
             
             "MA1655.1" ,"MA0102.4", "MA0836.2","MA1636.1")




trajFA=as.data.frame(only_STERO@reductions[["wnn.umap"]]@cell.embeddings)
trajFA$index=rownames(trajFA)

minmaxthis = function(x) { 
  (x - min(x))/(max(x) - min(x))
}


plot_list = list() 


    for (x in TFsnames) {
      print(x)
      
      DefaultAssay(only_STERO) = "chromvar"
      p10 = FeaturePlot(
        object = only_STERO,
        features = str_replace_all(x, fixed("_"), "-"),
        min.cutoff = 'q0',
        max.cutoff = 'q100',
        pt.size = 0.2)
      
      chromvardf = p10$data
      chromvardf$index = rownames(chromvardf)
      
      mergedFAchromvar = merge(chromvardf, trajFA, by = "index")
      
      
      TFactplot = ggplot(mergedFAchromvar, aes( x = FA_1, y=FA_2,colour = minmaxthis(mergedFAchromvar[,5]))) + 
        geom_point(size = 1, alpha = 1) + 
        scale_color_gradient(low = "gray93",high = "red1",breaks = seq(0,1,0.2))+
        xlab("FA1") + ylab("FA2") +
        theme(panel.border=element_blank(), 
              axis.line = element_line(colour = 'black', size = 1), 
              axis.ticks = element_blank(),
              axis.text = element_blank(), 
              panel.grid.major = element_blank(),
              panel.grid.minor = element_blank(),
              panel.background = element_blank(),
              legend.key=element_blank(),
              legend.title=element_blank(),
              legend.text=element_text(size=12)) + 
        # guides(color = guide_legend(override.aes = list(size=5))) + 
        coord_capped_cart(bottom='right', left='none')  + 
      ggtitle(x)
      TFactplot
      
      
      print(TFactplot)
  
}






In [ ]:
library(BSgenome.Mmusculus.UCSC.mm10)
library(Signac)
# # link peaks to genes Glomerulosa
   #  compute gene to link with peaks :

            Female_Fasc_gene=DE_STERO_Female_Fasciculata$gene_names[DE_STERO_Female_Fasciculata$avg_log2FC> 0]   #Testing 300 genes and 178944 peaks Found gene coordinates for 278 genes
            Male_Fasc_gene=DE_STERO_Female_Fasciculata$gene_names[DE_STERO_Female_Fasciculata$avg_log2FC< 0]     #Testing 287 genes and 178944 peaks Found gene coordinates for 279 genes
            
            Female_Glo_gene=DE_STERO_Female_Glomerulosa$gene_names[DE_STERO_Female_Glomerulosa$avg_log2FC> 0]    #Testing 69 genes and 178944 peaks Found gene coordinates for 65 genes
            Male_Glo_gene=DE_STERO_Female_Glomerulosa$gene_names[DE_STERO_Female_Glomerulosa$avg_log2FC< 0]      #Testing 39 genes and 178944 peaks Found gene coordinates for 37 genes

# LINK PEAK with DEG
LINK_Female_Fasc_gene<- LinkPeaks(
      object = only_STERO,
      peak.assay = "peaks",
      expression.assay = "SCT_RUN_STERO",
      genes.use = Female_Fasc_gene,
      )
LINK_Male_Fasc_gene<- LinkPeaks(
    object = only_STERO,
    peak.assay = "peaks",
    expression.assay = "SCT_RUN_STERO",
    genes.use = Male_Fasc_gene,
    )
LINK_Female_Glo_gene<- LinkPeaks(
    object = only_STERO,
    peak.assay = "peaks",
    expression.assay = "SCT_RUN_STERO",
    genes.use = Female_Glo_gene,
    )
LINK_Male_Glo_gene<- LinkPeaks(
    object = only_STERO,
    peak.assay = "peaks",
    expression.assay = "SCT_RUN_STERO",
    genes.use = Male_Glo_gene,
    )


# take peaks with significant correlation
LINK_Female_Fasc_gene=data.frame(Links(LINK_Female_Fasc_gene))
saveRDS(LINK_Female_Fasc_gene,file = 'LINK_Female_Fasc_gene')
LINK_Female_Fasc_gene_peaks=LINK_Female_Fasc_gene$peak[LINK_Female_Fasc_gene$score>0.1 &LINK_Female_Fasc_gene$pvalue<0.05 ]


LINK_Male_Fasc_gene=data.frame(Links(LINK_Male_Fasc_gene))
saveRDS(LINK_Male_Fasc_gene,file = 'LINK_Male_Fasc_gene')
LINK_Male_Fasc_gene_peaks=LINK_Male_Fasc_gene$peak[LINK_Male_Fasc_gene$score>0.1 &LINK_Male_Fasc_gene$pvalue<0.05]


LINK_Female_Glo_gene=data.frame(Links(LINK_Female_Glo_gene))
saveRDS(LINK_Female_Glo_gene,file = 'LINK_Female_Glo_gene')
LINK_Female_Glo_gene_peaks=LINK_Female_Glo_gene$peak[LINK_Female_Glo_gene$score>0.1 &LINK_Female_Glo_gene$pvalue<0.05 ]


LINK_Male_Glo_gene=data.frame(Links(LINK_Male_Glo_gene))
saveRDS(LINK_Male_Glo_gene,file = 'LINK_Male_Glo_gene')
LINK_Male_Glo_gene_peaks=LINK_Male_Glo_gene$peak[LINK_Male_Glo_gene$score>0.1 &LINK_Male_Glo_gene$pvalue<0.05 ]




https://bookdown.org/ytliu13207/SingleCellMultiOmicsDataAnalysis/scenic.html

In [ ]:
library(Seurat)
library(tidyverse)
library(magrittr)
library(SCENIC)
library(SingleCellExperiment)

cellInfo <- data.frame(only_STERO=Idents(only_STERO))
getwd()
only_STERO=readRDS('D:/Stage/aposimz/only_STERO_FINAL')



LINK_Male_Fasc_gene_peaks

https://stdworkflow.com/282/single-cell-transcriptional-regulatory-network-analysis-tool-tf-motifs-scenic

In [ ]:
# 1)Prepare cell meta information
cellInfo <- data.frame(only_STERO@meta.data)
colnames(cellInfo)[which(colnames(cellInfo)=="orig.ident")] <- "sample"
colnames(cellInfo)[which(colnames(cellInfo)=="identity")] <- "sexe"
colnames(cellInfo)[which(colnames(cellInfo)=="nFeature_SCT_RUN_STERO")] <- "nGene"
colnames(cellInfo)[which(colnames(cellInfo)=="nCount_SCT_RUN_STERO")] <- "nUMI"
colnames(cellInfo)[which(colnames(cellInfo)=="new_ident")] <- "celltype"
colnames(cellInfo)[which(colnames(cellInfo)=="merging_ident")] <- "celltype_merged"

cellInfo <- cellInfo[,c("sample","nGene","nUMI","celltype","celltype_merged","sexe")]
saveRDS(cellInfo, file="cellInfo.Rds")


# Set the celltype color
colVars <- list(celltype=c("Male_Fasciculata_1"="forestgreen", 
                           "Female_Fasciculata_2"="darkorange", 
                           "Male_Glomerulosa"="magenta4", 
                           "Female_Glomerulosa"="hotpink", 
                           "Male_Fasciculata_2"="red3", 
                           "Female_Fasciculata_1"="skyblue", 
                           "Capsular_Cells"="darkblue"))
colVars$celltype <- colVars$celltype[intersect(names(colVars$celltype), cellInfo$celltype)]


# 2) Initialize SCENIC settings, set up the analysis environment
org <- "mgi"
dbFiles <- 'C:/Users/User/Documents/aposimz/Rcis_target/'
myDatasetTitle <- "SCENIC mouse adrenal " # choose a name for your analysis
data(defaultDbNames)
# change to mm10
defaultDbNames$mgi[1] <- "mm10__refseq-r80__10kb_up_and_down_tss.mc9nr.feather"
defaultDbNames$mgi[2] <- "mm10__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.feather"

# create scenic object
scenicOptions <- initializeScenic(org= "mgi", datasetTitle = myDatasetTitle, dbDir=dbFiles,dbs =defaultDbNames[['mgi']],nCores = 4 )

scenicOptions@inputDatasetInfo$cellInfo <- cellInfo
scenicOptions@inputDatasetInfo$colVars <- colVars

# create matrix Build a Co-expression network

exprMat <- as.matrix(only_STERO@assays$SCT_RUN_STERO@counts) #Prepare the expression matrix. In order to save computing resources, only some cells are randomly selected to calculate the co-expression network

genesKept <- geneFiltering(exprMat, 
                           scenicOptions=scenicOptions,minCountsPerGene=3*.01*ncol(exprMat),minSamples=ncol(exprMat)*.01) #Gene filtering/selection, remove genes that are most likely to be noise

exprMat_filtered <- exprMat[genesKept, ]
dim(exprMat_filtered)
runCorrelation(exprMat_filtered, scenicOptions) ##Calculate the correlation matrix, 1.2_corrMat.Rds: the correlation matrix between genes








saveRDS(scenicOptions, file="int/scenicOptions.Rds")
saveRDS(cellInfo, file="int/cellInfo.Rds")
saveRDS(colVars, file="int/colVars.Rds")




##GENIE3, the input of GENIE3 is usually an expression matrix and a list of candidate regulators.
exprMat_filtered <- log2(exprMat_filtered+1) #complete matrix
runGenie3(exprMat_filtered, scenicOptions, nParts = 10) #nParts parameter, is to divide the expression matrix into n parts and calculate separately
#1.4, GENIE3_linkList.Rds: The final result of GENIE3 is to merge the files starting with "1.3_"


# Build and score the GRN (runSCENIC_…)

exprMat_log <- log2(exprMat+1) #log standard words original matrix
#scenicOptions <- readRDS("int/scenicOptions.Rds")
scenicOptions@settings$verbose <- TRUE
scenicOptions@settings$nCores <- 1
scenicOptions@settings$seed <- 123
scenicOptions <- runSCENIC_1_coexNetwork2modules(scenicOptions) #1. Get coexpression module
scenicOptions <- runSCENIC_2_createRegulons(scenicOptions) #2. Get regulons
#You can use the parameter coexMethod=c("top5perTarget"), coexMethod is available. The author tried a variety of strategies to filter low-relevance TF-Target, and suggested that all six filtering criteria should be used. In fact, it does not take much time. Both are calculated by default.
#w001: Use each TF as the core to reserve genes with weight>0.001 to form a co-expression module; #w005, #top50
#top5perTarget: Each gene retains the weight value of top5 TF to get a simplified TF-Target association table, and then assign the gene to TF to construct a co-expression module; #top10perTarget; #top50perTarget
scenicOptions <- runSCENIC_3_scoreCells(scenicOptions, exprMat_log) #3. Scoring the GRN (regulator) in the cell
saveRDS(scenicOptions, file="int/scenicOptions.Rds")
runSCENIC_4_aucell_binarize(scenicOptions, exprMat=exprMat_log)


# ADDING IN SEURAT 


##Import the original regulonAUC matrix, which is the AUC matrix generated after running runSCENIC_3_scoreCells, you can view the AUC activity score of each GRN in each cell

setwd("C:/Users/User/Documents/aposimz")
AUCmatrix <- readRDS("int/3.4_regulonAUC.Rds")


AUCmatrix <- data.frame(t(AUCmatrix@assays@data@listData$AUC), check.names=F)
RegulonName_AUC <- colnames(AUCmatrix)
RegulonName_AUC <- gsub(' \\(','_',RegulonName_AUC) #replace "(" with "_"
RegulonName_AUC <- gsub('\\)','',RegulonName_AUC) # Remove ")" and finally replace KLF3_extended (79g) with KLF3_extended_79g
colnames(AUCmatrix) <- RegulonName_AUC
only_STERO <- AddMetaData(only_STERO, AUCmatrix) # Add the AUC matrix to the metadata information of pbmc
saveRDS(only_STERO,'only_STERO_final')


AUCmatrix <- readRDS("int/3.4_regulonAUC.Rds")
regulonAUC <-  AUCmatrix@assays@data$AUC
rownames(regulonAUC) <- gsub("[(+)]", "", rownames(regulonAUC))

only_STERO[['AUC']] <- CreateAssayObject(data = regulonAUC)
  
# Identify differential regulons based AUC scores

DefaultAssay(only_STERO) <- 'AUC'

vector_clusters_F=c( "Female_Fasciculata","Female_Glomerulosa")
vector_clusters_M=c( "Male_Fasciculata", "Male_Glomerulosa")

for( i in 1:length(vector_clusters_F)){
  assign(paste0('REGULON_STERO_',vector_clusters_F[i]),FindMarkers(only_STERO,ident.1 =vector_clusters_F[i],ident.2 = vector_clusters_M[i],assay = 'AUC',logfc.threshold = 0.005))

}

REGULON_STERO_Female_Fasciculata<- REGULON_STERO_Female_Fasciculata[REGULON_STERO_Female_Fasciculata$p_val_adj < 0.05,]
REGULON_F_F_up <- REGULON_STERO_Female_Fasciculata[which(REGULON_STERO_Female_Fasciculata$avg_log2FC > 0), ]
REGULON_M_F_up <- REGULON_STERO_Female_Fasciculata[which(REGULON_STERO_Female_Fasciculata$avg_log2FC < 0), ]


deg.ls <- list(REGULON_F_F_up, REGULON_M_F_up)
names(deg.ls) <- c('Female_Fasciculata', 'Male_Fasciculata')

gene.ls <- lapply(deg.ls, rownames)

DefaultAssay(only_STERO) <- 'AUC'

only_STERO <- ScaleData(only_STERO, assay = 'AUC', features = rownames(only_STERO))

mg <- unique(Reduce('c', gene.ls[c(2)]))

DoHeatmap(only_STERO, features = mg, raster = F,slot = 'scale.data',assay = 'AUC') + scale_fill_viridis()




In [ ]:
library(monocle3)
library(monocle)
library(Seurat)
library(Signac)
library(Signac)
library(Seurat)
library(SeuratWrappers)
library(monocle3)
library(Matrix)
library(ggplot2)
library(patchwork)

#####choose datasets - FEMALE ####
female_STERO<-only_STERO[,  only_STERO$identity %in% c('female')]
# Subset
female_STERO <- SplitObject(female_STERO, split.by = "Replicat") # list 

############# RNA REPROCESSING ##############
for (i in 1:length(female_STERO)) {
    female_STERO[[i]] <- SCTransform(female_STERO[[i]],variable.features.n =NULL,
    variable.features.rv.th =1.1,return.only.var.genes = FALSE,
    assay = 'RNA',new.assay.name = 'SCT_INT',vars.to.regress = 'percent.mt')
    }
# Select the most variable features to use for integration
integ_features <-SelectIntegrationFeatures(object.list = female_STERO, nfeatures = 3000) 
# Prepare the SCT list object for integration
female_STERO  <- PrepSCTIntegration(object.list = female_STERO, 
                                   anchor.features =integ_features)

integ_anchors <- FindIntegrationAnchors(object.list = female_STERO, 
                                        normalization.method = "SCT", 
                                        anchor.features = integ_features,reduction="cca")

# Integrate across conditions
female_STERO <- IntegrateData(anchorset = integ_anchors,normalization.method = "SCT")                 # replace seurat list by object

# CLustering 
DefaultAssay(female_STERO)='integrated'
female_STERO=RunPCA(female_STERO, verbose = FALSE,assay="integrated",reduction.name='PCA_INT',npcs = 50)
female_STERO=RunICA(female_STERO,nics = 20)
ElbowPlot(female_STERO,reduction ='PCA_INT',ndims = 50 )# choose 15
female_STERO=FindNeighbors(female_STERO,assay ='integrated',reduction ='PCA_INT', dims = 1:10)
female_STERO=FindClusters(female_STERO, resolution = 0.3,algorithm = 1,graph.name = 'integrated_nn')
female_STERO=RunUMAP(female_STERO,dims = 1:10, reduction.name = 'UMAP_PCA_RNA_INT',reduction = 'PCA_INT')
DimPlot(female_STERO)
#####
###########
###############
####################
                                    ## ATAC processing  AND PEAKS  generating ## 

DefaultAssay(female_STERO) <- "peaks"
female_STERO <- FindTopFeatures(female_STERO, min.cutoff = 5)
female_STERO <- RunTFIDF(female_STERO)
female_STERO <- RunSVD(female_STERO)
DepthCor(female_STERO,n = 50) | ElbowPlot(female_STERO,reduction = 'lsi')

female_STERO <- RunUMAP(object = female_STERO,reduction = 'lsi',dims = 2:5,reduction.name = 'UMAP_LSI_ATAC')
female_STERO <- FindNeighbors(object = female_STERO,reduction = 'lsi',dims = 2:5) # use SNN for findclusters 
#only_STERO <- FindClusters(object = only_STERO,algorithm = 1,resolution = 0.7,verbose = FALSE)
DimPlot(object = female_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC')+DimPlot(female_STERO,reduction = 'UMAP_PCA_RNA_INT',group.by = 'Replicat')

##################################INTEGRATION ON REPLICAT  for atac 

crazy=female_STERO

crazy <- SplitObject(crazy, split.by = "Replicat") # list 


x=SelectIntegrationFeatures(
  crazy,
  nfeatures = 3000,
  assay = c('peaks','peaks'),
  verbose = TRUE
)


# find integration anchors
integration.anchors <- FindIntegrationAnchors(
  object.list = crazy,
  anchor.features =x,
  reduction = "rlsi",
  dims = 2:30
)

rm(crazy)
X1=female_STERO

# integrate LSI embeddings
integrated <- IntegrateEmbeddings(
  anchorset = integration.anchors,
  reductions = X1[["lsi"]],
  new.reduction.name = "integrated_lsi",
  dims.to.integrate = 2:30
)

integrated <- RunUMAP(integrated, reduction = "integrated_lsi", dims = 2:20)

DimPlot(integrated ,group.by = 'Replicat')
 
 
 #######################ADD INT LSI ON OBJECT 

female_STERO@reductions$UMAP_LSI_ATAC_INT=integrated@reductions[["umap"]]
female_STERO@reductions$integrated_lsi=integrated@reductions[["integrated_lsi"]]



DimPlot(object = female_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC')+DimPlot(female_STERO,reduction = 'UMAP_PCA_RNA_INT',group.by = 'Replicat')+DimPlot(object = female_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC_INT')



female_STERO <- FindMultiModalNeighbors(
  female_STERO, reduction.list = list("PCA_INT", "integrated_lsi"), 
  dims.list = list(1:10, 2:5),weighted.nn.name = 'X1'
)
female_STERO <- RunUMAP(female_STERO, nn.name = "X1", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")
female_STERO=FindClusters(female_STERO,graph.name = 'wknn',resolution = 0.5)

DimPlot(female_STERO,reduction = 'wnn.umap')
female_STERO$new_ident=Idents(female_STERO)





                                            ### TRAJECTORY FEMALE ####

Female_glomerulosa_root <- female_STERO[, female_STERO$new_ident %in% c("4")]
vector_root_F=colnames(Female_glomerulosa_root)


female_STERO@reductions$UMAP=female_STERO@reductions[["UMAP_PCA_RNA_INT"]]
Female.cds <- as.cell_data_set(female_STERO,assay = 'SCT_RUN_STERO')
Female.cds <- cluster_cells(cds = Female.cds, reduction_method = "UMAP",resolution=30)

# find all possible partitions
all_partitions <- unique(Female.cds@clusters$UMAP$partitions)
all_partitions <- all_partitions[all_partitions != "1"]
# set all partitions to 1
Female.cds@clusters$UMAP$partitions[Female.cds@clusters$UMAP$partitions %in% all_partitions] <- "1"

Female.cds <- learn_graph(Female.cds, use_partition = F)

Female.cds <- order_cells(Female.cds, reduction_method = "UMAP" ,root_cells =vector_root_F )
plot_cells(
  cds = Female.cds ,
  color_cells_by = "pseudotime",
  show_trajectory_graph = TRUE,cell_size = 2,trajectory_graph_segment_size = 2,graph_label_size = 2,label_cell_groups = F,label_branch_points = T,)

## add to only stero

only_STERO<- AddMetaData(
  object = only_STERO,
  metadata = Female.cds@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "RNA_ONLY_Female_trajectory"
)

FeaturePlot(only_STERO, c("WNN_ONLY_Female_trajectory"), pt.size = 1,reduction = 'wnn.umap',label = T) & scale_color_viridis_c()




#####choose datasets - MALE ####
male_STERO<-only_STERO[,  only_STERO$identity %in% c('male')]
# Subset
male_STERO <- SplitObject(male_STERO, split.by = "Replicat") # list 

############# RNA REPROCESSING ##############
for (i in 1:length(male_STERO)) {
    male_STERO[[i]] <- SCTransform(male_STERO[[i]],variable.features.n =NULL,
    variable.features.rv.th =1.1,return.only.var.genes = FALSE,
    assay = 'RNA',new.assay.name = 'SCT_INT',vars.to.regress = 'percent.mt')
    }
# Select the most variable features to use for integration
integ_features <-SelectIntegrationFeatures(object.list = male_STERO, nfeatures = 3000) 
# Prepare the SCT list object for integration
male_STERO  <- PrepSCTIntegration(object.list = male_STERO, 
                                   anchor.features =integ_features)

integ_anchors <- FindIntegrationAnchors(object.list = male_STERO, 
                                        normalization.method = "SCT", 
                                        anchor.features = integ_features,reduction="cca")

# Integrate across conditions
male_STERO <- IntegrateData(anchorset = integ_anchors,normalization.method = "SCT")                 # replace seurat list by object

# CLustering 
DefaultAssay(male_STERO)='integrated'
male_STERO=RunPCA(male_STERO, verbose = FALSE,assay="integrated",reduction.name='PCA_INT',npcs = 50)
ElbowPlot(male_STERO,reduction ='PCA_INT',ndims = 50 ) # choose 15
male_STERO=FindNeighbors(male_STERO,assay ='integrated',reduction ='PCA_INT', dims = 1:20)
male_STERO=FindClusters(male_STERO, resolution = 0.3,algorithm = 1,graph.name = 'integrated_nn')
male_STERO=RunUMAP(male_STERO,dims = 1:20, reduction.name = 'UMAP_PCA_RNA_INT',reduction = 'PCA_INT')
DimPlot(male_STERO)
#####
###########
###############
####################
                                    ## ATAC processing  AND PEAKS  generating ## 

DefaultAssay(male_STERO) <- "peaks"
male_STERO <- FindTopFeatures(male_STERO, min.cutoff = 5)
male_STERO <- RunTFIDF(male_STERO)
male_STERO <- RunSVD(male_STERO)
DepthCor(male_STERO,n = 50)

male_STERO <- RunUMAP(object = male_STERO,reduction = 'lsi',dims = 2:20,reduction.name = 'UMAP_LSI_ATAC')
male_STERO <- FindNeighbors(object = male_STERO,reduction = 'lsi',dims = 2:20) # use SNN for findclusters 
#only_STERO <- FindClusters(object = only_STERO,algorithm = 1,resolution = 0.7,verbose = FALSE)
DimPlot(object = male_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC')+DimPlot(male_STERO,reduction = 'UMAP_PCA_RNA_INT',group.by = 'Replicat')

##################################INTEGRATION ON REPLICAT  for atac 

crazy=male_STERO

crazy <- SplitObject(crazy, split.by = "Replicat") # list 


x=SelectIntegrationFeatures(
  crazy,
  nfeatures = 3000,
  assay = c('peaks','peaks'),
  verbose = TRUE
)


# find integration anchors
integration.anchors <- FindIntegrationAnchors(
  object.list = crazy,
  anchor.features =x,
  reduction = "rlsi",
  dims = 2:30
)

rm(crazy)
X1=male_STERO

# integrate LSI embeddings
integrated <- IntegrateEmbeddings(
  anchorset = integration.anchors,
  reductions = X1[["lsi"]],
  new.reduction.name = "integrated_lsi",
  dims.to.integrate = 2:30
)

integrated <- RunUMAP(integrated, reduction = "integrated_lsi", dims = 2:20)

DimPlot(integrated ,group.by = 'Replicat')
 
 
 #######################ADD INT LSI ON OBJECT 

male_STERO@reductions$UMAP_LSI_ATAC_INT=integrated@reductions[["umap"]]
male_STERO@reductions$integrated_lsi=integrated@reductions[["integrated_lsi"]]



DimPlot(object = male_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC')+DimPlot(male_STERO,reduction = 'UMAP_PCA_RNA_INT',group.by = 'Replicat')+DimPlot(object = male_STERO, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_LSI_ATAC_INT')



male_STERO <- FindMultiModalNeighbors(
  male_STERO, reduction.list = list("PCA_INT", "integrated_lsi"), 
  dims.list = list(1:20, 2:30),weighted.nn.name = 'X1'
)
male_STERO <- RunUMAP(male_STERO, nn.name = "X1", reduction.name = "wnn.umap", reduction.key = "wnnUMAP_")
male_STERO=FindClusters(male_STERO,graph.name = 'wknn',resolution = 0.5)

DimPlot(male_STERO,reduction = 'wnn.umap')
male_STERO$new_ident=Idents(male_STERO)


###  MALE TRAJECTORY ####

male_glomerulosa_root <- male_STERO[, male_STERO$new_ident %in% c("3")]
vector_root_M=colnames(male_glomerulosa_root)


male_STERO@reductions$UMAP=male_STERO@reductions[["UMAP_PCA_RNA_INT"]]
male.cds <- as.cell_data_set(male_STERO,assay = 'SCT_RUN_STERO')
male.cds <- cluster_cells(cds = male.cds, reduction_method = "UMAP",resolution=30)


# find all possible partitions
all_partitions <- unique(male.cds@clusters$UMAP$partitions)
all_partitions <- all_partitions[all_partitions != "1"]
# set all partitions to 1
male.cds@clusters$UMAP$partitions[male.cds@clusters$UMAP$partitions %in% all_partitions] <- "1"

male.cds <- learn_graph(male.cds, use_partition = FALSE)

male.cds <- order_cells(male.cds, reduction_method = "UMAP" ,root_cells =vector_root_M )
plot_cells(
  cds = male.cds ,
  color_cells_by = "pseudotime",
  show_trajectory_graph = TRUE,cell_size = 2,trajectory_graph_segment_size = 2,graph_label_size = 2,label_cell_groups = F,label_branch_points = T,)

## add to only stero

only_STERO<- AddMetaData(
  object = only_STERO,
  metadata = male.cds@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "RNA_ONLY_male_trajectory"
)

FeaturePlot(only_STERO, c("RNA_ONLY_male_trajectory","RNA_ONLY_Female_trajectory"), pt.size = 1,reduction = 'wnn.umap',label = T) & scale_color_viridis_c()


# with algo 

get_earliest_principal_node <- function(cds){
  cell_ids <- which(colData(cds)[, "new_ident"] %in% c("3"))
  
  closest_vertex <-
  cds@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex
  closest_vertex <- as.matrix(closest_vertex[colnames(cds), ])
  root_pr_nodes <-
  igraph::V(principal_graph(cds)[["UMAP"]])$name[as.numeric(names
  (which.max(table(closest_vertex[cell_ids,]))))]
  
  root_pr_nodes
}

cds_male <- order_cells(male.cds, root_pr_nodes=get_earliest_principal_node(male.cds))
cds_female <- order_cells(female.cds, root_pr_nodes=get_earliest_principal_node(female.cds))



# add  umap pseudotime value in seurat object 


only_STERO<- AddMetaData(
  object = only_STERO,
  metadata = cds_female@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "female_trajectory"
)


only_STERO <- AddMetaData(
  object = only_STERO,
  metadata = cds_male@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "male_trajectory"
)

FeaturePlot(only_STERO, c("male_trajectory", "female_trajectory"), pt.size = 1,reduction = 'wnn.umap',label = T) & scale_color_viridis_c()

Idents(only_STERO)=only_STERO$new_ident
FeaturePlot(only_STERO,features = c('male_trajectory','female_trajectory'),reduction = 'wnn.umap')

In [ ]:
##########

genes=rownames(all.cds)
genes=head(genes)



pt.matrix <- exprs(all.cds)[match(genes,rownames(rowData(all.cds))),order(pseudotime(all.cds))]
#Can also use "normalized_counts" instead of "exprs" to use various normalization methods, for example:
#normalized_counts(cds, norm_method = "log")

pt.matrix <- t(apply(pt.matrix,1,function(x){smooth.spline(x,df=3)$y}))
pt.matrix <- t(apply(pt.matrix,1,function(x){(x-mean(x))/sd(x)}))
rownames(pt.matrix) <- genes;


hthc <- Heatmap(
  pt.matrix,
  name                         = "z-score",
  col                          = colorRamp2(seq(from=-2,to=2,length=11),rev(brewer.pal(11, "Spectral"))),
  show_row_names               = TRUE,
  show_column_names            = T,
  row_names_gp                 = gpar(fontsize = 6),
  clustering_method_rows = "ward.D2",
  clustering_method_columns = "ward.D2",
  row_title_rot                = 0,
  cluster_rows                 = TRUE,
  cluster_row_slices           = T,
  cluster_columns              = T)







### take reduction 
all_STERO=only_STERO

all_STERO@reductions$UMAP=all_STERO@reductions[["wnn.umap"]]

# root for trajectory 
all_root <- all_STERO[, all_STERO$new_ident %in% c( 'Capsular_Cells',"Female_Glomerulosa","Male_Glomerulosa")]
vector_root_all=colnames(all_root)



# Building trajectories with Monocle 3



all.cds <- as.cell_data_set(all_STERO)
all.cds <- cluster_cells(cds = all.cds, reduction_method = "UMAP",resolution = 1,max.overlaprs=10)



# find all possible partitions
all_partitions <- unique(all.cds@clusters$UMAP$partitions)
all_partitions <- all_partitions[all_partitions != "1"]
# set all partitions to 1
all.cds@clusters$UMAP$partitions[all.cds@clusters$UMAP$partitions %in% all_partitions] <- "1"
all.cds <- learn_graph(all.cds, use_partition = FALSE)


# order cells
all.cds <- order_cells(all.cds, reduction_method = "UMAP" ,root_cells =vector_root_all )



x=plot_cells(
  cds = all.cds,
  color_cells_by = "pseudotime",
  show_trajectory_graph = TRUE,cell_size = 2,trajectory_graph_segment_size = 2,graph_label_size = 2,label_cell_groups = F,label_branch_points = T,)+
  plot_cells(
  cds = all.cds,
  color_cells_by = "pseudotime",
  show_trajectory_graph = TRUE,cell_size = 2,trajectory_graph_segment_size = 2,graph_label_size = 2,label_cell_groups = F,label_branch_points = T)



# # a helper function to identify the root principal points:
get_earliest_principal_node <- function(cds){
  cell_ids <- which(colData(cds)[, "new_ident"] %in% c('Capsular_Cells',"Female_Glomerulosa","Male_Glomerulosa"))
  
  closest_vertex <-
  cds@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex
  closest_vertex <- as.matrix(closest_vertex[colnames(cds), ])
  root_pr_nodes <-
  igraph::V(principal_graph(cds)[["UMAP"]])$name[as.numeric(names
  (which.max(table(closest_vertex[cell_ids,]))))]
  
  root_pr_nodes
}
cds_male <- order_cells(male.cds, root_pr_nodes=get_earliest_principal_node(male.cds))
cds_female <- order_cells(female.cds, root_pr_nodes=get_earliest_principal_node(female.cds))
all.cds <- order_cells(all.cds, root_pr_nodes=get_earliest_principal_node(all.cds))


# add  umap pseudotime value in seurat object 


only_STERO <- AddMetaData(
  object = only_STERO,
  metadata = all.cds@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "all_sexe_trajectory_umap"
)


multiomics <- AddMetaData(
  object = multiomics,
  metadata = male.cds@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "male"
)

FeaturePlot(multiomics, c("female", "male"), pt.size = 1,reduction = 'wnn.umap',label = T) & scale_color_viridis_c()
Idents(only_STERO)=only_STERO$new_ident


In [ ]:

library(dyno)
library(tidyverse)

DefaultAssay(only_STERO)='SCT_RUN_STERO'
# PREPARING THE DATA

object_counts <- Matrix::t(as(as.matrix(only_STERO@assays$SCT_RUN_STERO@counts), 'sparseMatrix'))
object_expression <- Matrix::t(as(as.matrix(only_STERO@assays$SCT_RUN_STERO@data), 'sparseMatrix'))
object_cellinfo <- only_STERO@meta.data #optional, i needed the cell type IDs
object_cellinfo <- only_STERO@meta.data[["merging_ident"]]


dyno_object <- wrap_expression(
  counts = object_counts,
  expression = object_expression
  )


 dyno_object <- add_grouping(
   dyno_object,
  only_STERO@meta.data$merging_ident
 )

dyno_object <- add_dimred(dyno_object,
                          as.matrix(only_STERO@reductions$wnn.umap@cell.embeddings)
                          )

 
# guidelines : 
 guidelines_shiny(dataset = dyno_object)
 
 guidelines <- guidelines(
  dyno_object,
  answers = answer_questions(
    dyno_object,
    multiple_disconnected = FALSE,
    expect_topology = TRUE,
    expected_topology = "linear"
  )
)
 
# INFERRING TRAJECTORIES

 
model <- infer_trajectory(dyno_object, ti_paga_tree())




dyno_object <-add_dimred(dimred =as.matrix(only_STERO@reductions$wnn.umap@cell.embeddings),expression_source = dyno_object$expression)

dyno_object <- add_dimred(dyno_object,as.matrix(only_STERO@reductions$wnn.umap@cell.embeddings))



plot_dimred(
  model, 
  expression_source = dyno_object$expression, 
  grouping = object_cellinfo 
)

In [ ]:

library(slingshot)
library(BUSpaRse)
library(tidyverse)
library(tidymodels)
library(Seurat)
library(scales)
library(viridis)
library(Matrix)

sce <- SingleCellExperiment(female_STERO)


In [ ]:
library(SeuratDisk)
setwd('D:/Stage')

female_STERO=readRDS('D:/Stage/female_stero')
female_STERO@assays[["peaks"]]@motifs =NULL # error if features in motif slot 
i <- sapply(female_STERO@meta.data, is.factor)
female_STERO@meta.data[i] <- lapply(female_STERO@meta.data[i], as.character)
SaveH5Seurat(female_STERO, filename = "female_STERO",overwrite = T)
Convert("female_STERO.h5seurat", dest = "h5ad",overwrite =T  ,assay= c('integrated'))

male_STERO=readRDS('D:/Stage/male_STERO')
male_STERO@assays[["peaks"]]@motifs =NULL # error if features in motif slot 
i <- sapply(male_STERO@meta.data, is.factor)
male_STERO@meta.data[i] <- lapply(male_STERO@meta.data[i], as.character)
SaveH5Seurat(male_STERO, filename = "male_STERO",overwrite = T)
Convert("male_STERO.h5seurat", dest = "h5ad",overwrite =T  ,assay= c('integrated'))


 # 1) Convert seurat object to anndata 
      # object seurat should not have a motif slot , otherwise ==> bug 
only_STERO_nomotif=only_STERO
only_STERO_nomotif@assays[["peaks"]]@motifs =NULL # error if features in motif slot
#metadata need to be as.character 
i <- sapply(only_STERO_nomotif@meta.data, is.factor) 
only_STERO_nomotif@meta.data[i] <- lapply(only_STERO@meta.data[i], as.character)
# To import lsi dim 
only_STERO_nomotif@reductions[["integrated_lsi"]]@assay.used ='integrated'
# Save file 
SaveH5Seurat(only_STERO_nomotif, filename = "only_STERO_nomotif_convert.h5Seurat",overwrite = T)
# Convert file 
# just take the assay you want 
Convert("only_STERO_nomotif_convert.h5Seurat", dest = "h5ad",overwrite =T  ,assay= c('integrated'))



In [ ]:
%%python
import scanpy  # pip install in  R terminal
import scanpy as sc
import numpy as np
print(np.__version__)
import pandas as pd
from matplotlib import rcParams
from pyWNN import pyWNN
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nwx
import os


sc.pl.paga_compare(
  anndata,
  basis = "X_draw_graph_fr",
  threshold = 0.15,
  edge_width_scale = 0.5,
)

################
anndata=sc.read_h5ad('D:/Stage/TRUTH/multiomics_convert.h5ad')
del anndata.uns['neighbors']
del anndata.obsp['distances']
del anndata.varm['ICA_STERO']
del anndata.obsm['X_ICA_STERO']
del anndata.obsm['X_UMAP_LSI_ATAC']
del anndata.obsm['X_UMAP_PCA_RNA_INT']
del anndata.obsm['X_UMAP_LSI_ATAC_INT']
#del anndata.obsm['X_PCA_INT']
del anndata.varm['PCA_INT']



sc.pp.neighbors(anndata, n_neighbors=50, use_rep='X_PCA_INT')
 # sc.tl.leiden(anndata, resolution = 1)



sc.tl.paga(anndata,groups='leiden_merging')
sc.pl.paga(anndata)
sc.tl.draw_graph(anndata)

anndata.uns['iroot'] = np.flatnonzero(anndata.obs['leiden_merging']  =='Capsular_cells')[0]
sc.tl.draw_graph(anndata, 
                init_pos='paga')# root =np.flatnonzero(anndata.obs['leiden_merging']=='Capsular_cells')[0])


sc.tl.dpt(anndata)
sc.pl.draw_graph(anndata, color=['merging_ident', 'dpt_pseudotime','leiden','identity'], legend_loc='on data')




# plot 

myColors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231',
            '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe',
            '#008080', '#e6beff', '#9a6324', '#000000', '#800000',
            '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', 
            '#ffffff', '#fffac8']

f, axs = plt.subplots(1,2,figsize=(36,12))
sns.set(font_scale=1.5)
sns.set_style("white")

sc.pl.draw_graph(anndata, 
                 color           = 'merging_ident', 
                 palette         = myColors,
                 legend_loc      = 'on data', 
                 show            = True, 
                 size            = 100, 
                 legend_fontsize = 'large')
                 
                 
sc.pl.draw_graph(anndata, 
                 color           = 'dpt_pseudotime', 
                 legend_loc      = 'on data', 
                 show            = True, 
                 size            = 400, 
                 color_map       = "viridis",
                 legend_fontsize = 'large')

                 
sns.despine(offset=10, trim=False)
plt.tight_layout()
plt.show(block=False)
plt.close("all")



# save value

dpt_pseudotime = pd.DataFrame(index   = anndata.obs.index.tolist(),
                              columns = ["Pseudotime"],
                              data    = anndata.obs["dpt_pseudotime"].tolist())

dpt_pseudotime.to_csv("D:/Stage/Pseudotime_RNA.csv")



FA2_comp = pd.DataFrame(index   = anndata.obs.index.tolist(),
                        columns = ["FA1", "FA2"],
                        data    = anndata.obs["X_draw_graph_fa"])

FA2_comp.to_csv("D:/Stage/Pseudotime_RNA.csvFA2_components_ATAC.csv")


# save paga embending 
clust_embedding = pd.DataFrame(anndata.uns['paga']['pos'], columns=['X', 'Y'])
clust_embedding['Cluster'] = range(clust_embedding.shape[0])
clust_embedding = clust_embedding[['Cluster', 'X', 'Y']]

clust_embedding.to_csv('D:/Stage/TRUTH/PAGA/cluster_embedding.csv'),
                       index=False)

# save object 
sc.read_h5ad(anndata)



names=None,
sparse=True
con=clust_con


def con2edges(con, names=None, sparse=True):
    print('Converting connectivity matrix to edges...')
    n = con.shape[0]
    edges = pd.DataFrame(columns=['From', 'To', 'Connectivity'])

    for i in range(n):
        for j in range(i + 1, n):
            if names is not None:
                fr = names[i]
                to = names[j]
            else:
                fr = str(i)
                to = str(j)

            connectivity = con[i, j]
            if sparse and connectivity == 0:
                continue

            entry = {'From' : fr, 'To' : to,
                     'Connectivity' : con[i, j]}
            edges = edges.append(entry, ignore_index=True)

    return edges


clust_con = anndata.uns['paga']['connectivities'].toarray()
clust_edges = con2edges(clust_con)
clust_edges.to_csv(os.path.join(out_dir, 'cluster_edges.csv'),
                   index=False)




print('outputting cluster edges...')
clust_con = anndata.uns['paga']['connectivities'].toarray()
clust_edges = con2edges(clust_con)
clust_edges.to_csv(os.path.join(out_dir, 'cluster_edges.csv'),
                   index=False)






In [ ]:




clust_embedding <- read_csv(
    here::here('D:/Stage/TRUTH/PAGA/cluster_embedding.csv'),
    col_types = cols(
        .default = col_double()))




clust_plot <- ggplot(clust_embedding, aes(x = X, y = Y)) +
    geom_segment(data = clust_edges,
                 aes(x = FromX, y = FromY, xend = ToX, yend = ToY,
                     colour = Connectivity, alpha = Connectivity), size = 4) +
    scale_colour_viridis_c(limits = c(0, 1)) +
    scale_alpha_continuous(limits = c(0, 1), range = c(0, 1), guide = FALSE) +
    geom_point(aes(fill = Cluster, size = Size), shape = 21) +
    geom_text(aes(label = Cluster)) +
    scale_size(range = c(6, 20), guide = FALSE) +
    scale_fill_discrete(guide = FALSE) +
    guides(colour = guide_colourbar(barwidth = 20)) +
    ggtitle("PAGA cluster graph") +
    theme_void() +
    theme(plot.title = element_text(size = rel(1.2), hjust = 0.1, 
                                    vjust = 1, margin = margin(5)),
          legend.position = "bottom")










Create new object without sexual chromosome genes and normalize/ clustering / plot 

In [ ]:
library(biomaRt)
DefaultAssay(only_STERO)='SCT'
mart <- useMart("ENSEMBL_MART_ENSEMBL", dataset = "mmusculus_gene_ensembl")

genes.table <- try(biomaRt::getBM(attributes = c("ensembl_gene_id", "external_gene_name",
"description", "gene_biotype", "chromosome_name", "start_position"), mart = mart, useCache = F))   # fetch chromosome info plus some other annotations

chrY.gene = genes.table$external_gene_name[genes.table$chromosome_name == "Y"] 
chrX.gene = genes.table$external_gene_name[genes.table$chromosome_name == "X"]
chrX_and_Y=append(chrY.gene,chrX.gene)

counts <- GetAssayData(multiomics, assay = "RNA")
counts <- counts[-(which(rownames(counts) %in% chrX_and_Y)),]
multiomics_sex_removed <- subset(multiomics, features = rownames(counts)) 


# re compute a UMAP

# re normalize , find variable feature and scaling
multiomics_sex_removed=SCTransform(multiomics_sex_removed,assay = 'RNA',variable.features.rv.th = 1.1,variable.features.n = NULL,new.assay.name = 'SCT_nosex',return.only.var.genes = FALSE)
DefaultAssay(multiomics_sex_removed)='SCT_nosex'

# reduce dimension , umap and clustering
multiomics_sex_removed=RunPCA(multiomics_sex_removed, verbose = FALSE,assay="SCT_nosex",reduction.name='PCA_nosex')
multiomics_sex_removed=FindNeighbors(multiomics_sex_removed,assay ='SCT_nosex',reduction ='PCA_nosex', dims = 1:50)
   # multiomics_sex_removed=FindClusters(multiomics_sex_removed, resolution = 0.5,algorithm = 1)
multiomics_sex_removed=RunUMAP(multiomics_sex_removed,dims = 1:50, reduction.name = 'UMAP_PCA_RNA_NOSEX')


DimPlot(multiomics_sex_removed,reduction = 'UMAP_PCA_RNA_NOSEX',label = T) | DimPlot(object = multiomics_sex_removed, label = TRUE,group.by = 'identity',reduction = 'UMAP_PCA_RNA_NOSEX') |DimPlot(object = multiomics_sex_removed, label = TRUE,group.by = 'Replicat',reduction = 'UMAP_PCA_RNA_NOSEX')

# glomerulosa don't split between sexe . fasiculata split with sexe , and by replicat


In [ ]:
# http://htmlpreview.github.io/?https://github.com/aertslab/SCENIC/blob/master/inst/doc/SCENIC_Setup.html
# http://htmlpreview.github.io/?https://github.com/aertslab/SCENIC/blob/master/inst/doc/SCENIC_Running.html

BiocManager::install(c("AUCell", "RcisTarget"))
BiocManager::install(c("GENIE3")) # Optional. Can be replaced by GRNBoost
BiocManager::install(c("GRNBoost"))
BiocManager::install(c("zoo", "mixtools", "rbokeh"))
BiocManager::install(c("DT", "NMF", "ComplexHeatmap", "R2HTML", "Rtsne"))
BiocManager::install(c("doMC", "doRNG"))
devtools::install_github("aertslab/SCopeLoomR", build_vignettes = TRUE)     #To export/visualize in http://scope.aertslab.org
devtools::install_github("aertslab/SCENIC") 
packageVersion("SCENIC")

# loading with wget "", linux :
dbFiles <- c("https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg19/refseq_r45/mc9nr/gene_based/hg19-500bp-upstream-7species.mc9nr.feather",
"https://resources.aertslab.org/cistarget/databases/homo_sapiens/hg19/refseq_r45/mc9nr/gene_based/hg19-tss-centered-10kb-7species.mc9nr.feather")
# mc9nr: Motif collection version 9: 24k motifs
dbFiles <- c("https://resources.aertslab.org/cistarget/databases/mus_musculus/mm9/refseq_r45/mc9nr/gene_based/mm9-500bp-upstream-7species.mc9nr.feather",
"https://resources.aertslab.org/cistarget/databases/mus_musculus/mm9/refseq_r45/mc9nr/gene_based/mm9-tss-centered-10kb-7species.mc9nr.feather")
# mc9nr: Motif collection version 9: 24k motifs
dbFiles <- c("https://resources.aertslab.org/cistarget/databases/drosophila_melanogaster/dm6/flybase_r6.02/mc8nr/gene_based/dm6-5kb-upstream-full-tx-11species.mc8nr.feather")
# mc8nr: Motif collection version 8: 20k motifs


Enrichment analysis (DE) with Gprofiler , here we just need the gene name as input

In [ ]:
library(gprofiler2)
#Analysing multiple gene lists

# get the significant up/down-regulated genes
X1=FindMarkers(only_STERO,ident.1 = c("Male_Fasciculata","Male_Glomerulosa","Male_tZone"),ident.2 = c("Female_tZone","Female_Fasciculata","Female_Glomerulosa"))
X1$gene_names=rownames(X1)
female_up=X1[X1$p_val_adj<0.05&X1$avg_log2FC>0,]
male_up=X1[X1$p_val_adj<0.05&X1$avg_log2FC<0,]


female_up = DE_STERO_Female_ZF
male_up = DE_STERO_Male_ZF

# order genes by log2FC
female_up = female_up[order(female_up$avg_log2FC, decreasing = TRUE),]

male_up = male_up[order(male_up$avg_log2FC, decreasing = FALSE),] # higher value first 

### Background set 
background=rownames(only_STERO@assays[["SCT_RUN_STERO"]]@data) # take the good assay ! 

# ordered enrichment analysis

gp_up_ordered_F = gost(list(female_up$gene_names,male_up$gene_names), organism = 'mmusculus',
 ordered_query = TRUE,sources = c('GO:BP', 'GO:MF', 'GO:CC','KEGG', 'REAC', 'TF','WP'),domain_scope="custom",custom_bg = background)

# run individually 
gp_up_ordered_F = gost(female_up$gene_names, organism = 'mmusculus',
 ordered_query = TRUE,sources = c('GO:BP', 'GO:MF', 'GO:CC','KEGG', 'REAC', 'TF','WP'),domain_scope="custom",custom_bg = background,correction_method = "bonferroni",user_threshold = 0.01)

gp_up_ordered_M= gost(male_up$gene_names, organism = 'mmusculus',
 ordered_query = TRUE,sources = c('GO:BP', 'GO:MF', 'GO:CC','KEGG', 'REAC', 'TF','WP'),domain_scope="custom",custom_bg = background,correction_method = "bonferroni",user_threshold = 0.01)

gp_up_ordered_M
gp_up_ordered_F
#plot
gostplot(gp_up_ordered_F,capped = FALSE,interactive = TRUE) # need to highlight the symbol on the plot
gostplot(gp_up_ordered_M,capped = FALSE,interactive = TRUE) # need to highlight the symbol on the plot



# clusterprofiler

In [ ]:
library(ggplot2)
library(enrichplot)
library(clusterProfiler)
library(org.Mm.eg.db)
library(DOSE)
#######################################ENSEMBL Gene Set Enrichment Analysis

# we want the log2 fold change
DE_Female_Fasciculata_log2FC<-DE_STERO_Female_tZ$avg_log2FC
# name the vector for each log2fc get the gene names 
names(DE_Female_Fasciculata_log2FC)<- DE_STERO_Female_tZ$ensembl_gene_id # take ensemble id
# remove any NA values 
DE_Female_Fasciculata_log2FC<-na.omit(DE_Female_Fasciculata_log2FC)
DE_Female_Fasciculata_log2FC<-DE_Female_Fasciculata_log2FC[!is.na(names(DE_Female_Fasciculata_log2FC))]
# sort the list in decreasing order (required for clusterProfiler)
DE_Female_Fasciculata_log2FC = sort(DE_Female_Fasciculata_log2FC, decreasing = TRUE)
##################################################### output 
gse <- gseGO(geneList=DE_Female_Fasciculata_log2FC, 
             ont ="MF", 
             keyType = "ENSEMBL", 
             minGSSize = 3, 
             maxGSSize = 800, 
             pvalueCutoff = 0.05, 
             verbose = TRUE, 
             OrgDb = org.Mm.eg.db, 
             pAdjustMethod = "none")

# plot
dotplot(gse, showCategory=10, split=".sign") + facet_grid(.~.sign)


#######################################KEGG Gene Set Enrichment Analysis
# we want the log2 fold change
DE_Female_Fasciculata_log2FC<-`DE_Female Fasciculata`$avg_log2FC
# name the vector for each log2fc get the gene names 
names(DE_Female_Fasciculata_log2FC)<- `DE_Female Fasciculata`$ENTREZID
# remove any NA values 
DE_Female_Fasciculata_log2FC<-na.omit(DE_Female_Fasciculata_log2FC)
DE_Female_Fasciculata_log2FC<-DE_Female_Fasciculata_log2FC[!is.na(names(DE_Female_Fasciculata_log2FC))]
# sort the list in decreasing order (required for clusterProfiler)
DE_Female_Fasciculata_log2FC = sort(DE_Female_Fasciculata_log2FC, decreasing = TRUE)

##########################################Create gseKEGG object
KEG <- gseKEGG(geneList     = DE_Female_Fasciculata_log2FC,
               organism     = 'mmu',
               minGSSize    = 3,
               maxGSSize    = 800,
               pvalueCutoff = 0.05,
               pAdjustMethod = "none",
               keyType       = "ncbi-geneid",)

# plot
dotplot(KEG, title = "Enriched Pathways" , split=".sign") + facet_grid(.~.sign)
dotplot(gse, split=".sign") + facet_grid(.~.sign)


Comparing multiple gene lists

In [ ]:

all_sex_dif=FindMarkers(remove_only_stero,ident.1 =c("Female_tZone_female","Female_Fasciculata_2_female","Female_Fasciculata_1_female","Capsular_cells_female", "Female_Glomerulosa_female"),ident.2=c("Male_tZone_male","Male_Fasciculata_2_male","Male_Fasciculata_1_male","Capsular_cells_male", "Male_Glomerulosa_male"),assay = 'SCT_RUN_STERO',logfc.threshold = 0.2)
all_sex_dif=all_sex_dif[all_sex_dif$p_val_adj<0.05,]
all_sex_dif$gene_names=rownames(all_sex_dif)
all_sex_dif$val2 <- mouse_TF$V1[match(all_sex_dif$gene_names, mouse_TF$V1)]
all_sex_dif$sex <- chrx_y$chrX_and_Y[match(all_sex_dif$gene_names, chrx_y$chrX_and_Y)]

ensembl <- useMart("ensembl", dataset="mmusculus_gene_ensembl")
gene_IDs <- getBM(filters= 'mgi_symbol', attributes= c("ensembl_gene_id",'mgi_symbol','entrezgene_id','description'),values = all_sex_dif$gene_names, mart= ensembl)
# Add into DE DATAFRAME
all_sex_dif=left_join(all_sex_dif,gene_IDs, by = c("gene_names"="mgi_symbol"))
# check duplicate value 
all_sex_dif$gene[duplicated(all_sex_dif$gene_names)]
# remove duplicate value
all_sex_dif=all_sex_dif[!duplicated(all_sex_dif$gene_names), ]
write.table(all_sex_dif$gene_names,file = 'all_sex_dif.txt',quote = F,row.names = F,col.names = F)



mydf <- data.frame(Entrez=all_sex_dif$entrezgene_id,all_sex_dif)
mydf <- mydf[abs(mydf$avg_log2FC) > 0,]
mydf$group <- "Female upregulated"
mydf$group[mydf$avg_log2FC < 0] <- "Male upregulated"
mydf$othergroup <- "no significant"
mydf$othergroup[(mydf$avg_log2FC > 0.7)] <- "Female highly upregulated"
mydf$othergroup[(mydf$avg_log2FC < -0.7)] <- "Male highly upregulated"



x=mydf$gene_names[mydf$avg_log2FC>0&mydf$p_val_adj<0.05]
write.table(x = x,file = 'all_female.txt',quote = F,row.names = F,col.names = F)
getwd()

formula_res_GO <- compareCluster(Entrez~group, data=mydf, OrgDb="org.Mm.eg.db", fun="groupGO")
formula_res_KEGG <- compareCluster(Entrez~group, data=mydf, fun="enrichKEGG",org='mmu')




top_markers=all_sex_dif



top_markers$sign <- 0
top_markers$sign_logFC[top_markers$avg_log2FC > 0] <- 1
top_markers$description <- gsub(' \\[Source.*$', '', top_markers$description)

ggplot(top_markers[is.na(top_markers$description) == F, ], aes(reorder(gene_id, avg_logFC), avg_logFC))+
  geom_bar(stat='identity', aes(fill = factor(sign_logFC)))+
  coord_flip()+
  scale_x_discrete(labels = top_markers$description[is.na(top_markers$description) == F][order(top_markers$avg_logFC[is.na(top_markers$description) == F])])+
  labs(x = 'gene description', y = 'Average logFC')+
  guides(fill = F)











head(formula_res_KEGG)
dotplot(formula_res_GO,title = " Fasciculata differentially enriched pathways", x="group",size='count',showCategory =NULL ) + facet_grid(~othergroup)


# Volcano plot

In [ ]:


all_sex_dif=FindMarkers(remove_only_stero,ident.1 =c("Female_tZone_female","Female_Fasciculata_female","Capsular_cells_female", "Female_Glomerulosa_female"),ident.2=c("Male_tZone_male","Male_Fasciculata_male","Capsular_cells_male", "Male_Glomerulosa_male"),assay = 'SCT_RUN_STERO',logfc.threshold = 0)
all_sex_dif=all_sex_dif[all_sex_dif$p_val_adj<0.05,]
all_sex_dif$gene_names=rownames(all_sex_dif)
all_sex_dif$val2 <- mouse_TF$V1[match(all_sex_dif$gene_names, mouse_TF$V1)]
all_sex_dif$sex <- chrx_y$chrX_and_Y[match(all_sex_dif$gene_names, chrx_y$chrX_and_Y)]

  write.table(all_sex_dif$gene_names[all_sex_dif$avg_log2FC< - 0.5 & all_sex_dif$p_val_adj<0.05],file = 'male_all_cells.txt',quote = F,row.names = F,col.names = F)
  write.table(all_sex_dif$gene_names[all_sex_dif$avg_log2FC> 0.5 & all_sex_dif$p_val_adj<0.05],file = 'female_all_cells.txt',quote = F,row.names = F,col.names = F)

  
sexy=all_sex_dif
sexy<-sexy[!is.na(sexy$sex),]


sexF=all_sex_dif$sex[all_sex_dif$p_val<0.05& all_sex_dif$avg_log2FC >  0.1]
sexF<-sexF[!is.na(sexF)]



  # create custom key-value pairs for 'high', 'low', 'mid' expression by fold-change
  # this can be achieved with nested ifelse statements
  keyvals <- ifelse(all_sex_dif$sex %in% sexy$sex==TRUE, 'red',
              ifelse(all_sex_dif$avg_log2FC > 0 & all_sex_dif$p_val_adj < 0.05, '#D16103',
              ifelse(all_sex_dif$avg_log2FC < 0 & all_sex_dif$p_val_adj < 0.05, '#4E84C4','black')))

  keyvals[is.na(keyvals)] <- 'black'
  names(keyvals)[keyvals == '#D16103'] <- 'Female'
  names(keyvals)[keyvals == 'black'] <- 'Not significant '
  names(keyvals)[keyvals == '#4E84C4'] <- 'Male'
  names(keyvals)[keyvals == 'red'] <- 'sexual chromosome'

  
  
vector1=all_sex_dif$gene_names[all_sex_dif$p_val_adj< 0.005&all_sex_dif$avg_log2FC>1.1]
vector2=all_sex_dif$gene_names[all_sex_dif$p_val_adj< 0.005&all_sex_dif$avg_log2FC< -1.1]
vector2=append(vector1,vector2)
vector2=append(x = vector2,values = 'Smarca1')
  
  lab_italics <- paste0("italic('", rownames(all_sex_dif), "')")
  selectLab_italics = paste0(
    "italic('",
    vector2, "')")


 EnhancedVolcano(all_sex_dif,
    lab = lab_italics,
    x = 'avg_log2FC',
    y = 'p_val_adj',
  pCutoff = 0.05,
    FCcutoff = 0.9,
    cutoffLineType = 'twodash',
    cutoffLineWidth = 0.5,
    pointSize = 4.0,
    labSize = 6.0,
    colAlpha = 1,
    legendLabels=c('Not sig.','Log (base 2) FC','p-value',
      'p-value & Log (base 2) FC'),
    legendPosition = 'right',
    legendLabSize = 16,
    legendIconSize = 5.0,
  colCustom = keyvals,  

    parseLabels = TRUE, title = 'STERODOGENIC GENE EXPRESSION ')+ coord_flip()
 


)
 



# Remove sexual chromosome peaks and do a clustering

In [ ]:
DefaultAssay(multiomics)='peaks'
counts_peaks=as.data.frame(rownames(multiomics)) # first i'm taking all the peaks
names(counts_peaks)[1] <- "newName"# rename columns 


counts_peak_grep=counts_peaks$newName[!grepl("chrY.*\\d",counts_peaks$newName)] # get rid of all the peaks beginning with Y
counts_peak_grep=counts_peaks$newName[!grepl("chrX.*\\d",counts_peaks$newName)] # get rid of all the peaks beginning with X
length(counts_peak_grep)

# Subset this peaks with PEAKS ASSAY of seurat object 

toRemove <- (counts_peak_grep) 
no_Y <- multiomics[rownames(multiomics) %in% toRemove,] # subset from seurat object 
no_Y=RenameAssays(no_Y, peaks='peaks_nosex') # rename assay


# re compute a UMAP 

# re run find variable features and normalisation
no_Y <- FindTopFeatures(no_Y, min.cutoff = 'q5',assay = 'peaks_nosex')
no_Y <- RunTFIDF(no_Y)
no_Y <- RunSVD(no_Y,reduction.name = 'lsi_nosex')
DepthCor(no_Y) # remove first component ( classic)

# UMAP AND CLUSTERING
no_Y <- RunUMAP(object = no_Y,reduction = 'lsi_nosex',dims = 2:50)
no_Y <- FindNeighbors(object = no_Y,reduction = 'lsi_nosex',dims = 2:50,assay ='peaks_nosex') # use SNN for findclusters 
no_Y <- FindClusters(object = no_Y,algorithm = 1,resolution = 0.7,verbose = FALSE,graph.name = 'peaks_nosex_snn')

DimPlot(object = no_Y, label = TRUE) | DimPlot(object = no_Y, label = TRUE,group.by = 'identity') |DimPlot(object = no_Y, label = TRUE,group.by = 'Replicat')

VlnPlot(multiomics, features = 'nFeature_peaks')
# same as RNA, the glomerulosa don't split , sexe chromsome must explain a dimorphism in expression , try di

#  HOCOMOCCO PWM MATRIX

In [ ]:


 library(universalmotif)
  library(TFBSTools)
  library(rlist)
  # data taken from https://hocomoco11.autosome.ru/final_bundle/hocomoco11/full/HUMAN/mono/HOCOMOCOv11_full_pwm_HUMAN_mono.tar.gz
  pathToPWMs = "D:/ADRENAL_PROJECT/hocomoco_motif/HOCOMOCOv11_full_pwm_MOUSE_mono/pwm/"
  pwmfiles = list.files(pathToPWMs)

  empty.list <- list()
  
    for (x in 1:length(pwmfiles)) {
  
    test1 = read_matrix(paste0(pathToPWMs,pwmfiles[x]), 
                      headers = ">", sep = "", positions = "rows")
    y  =convert_motifs(test1, class = "TFBSTools-PWMatrix")
  
    empty.list <- c(empty.list, y)
  
  }

  pfm.list <- do.call(PWMatrixList, empty.list)
  
for (i in 1:length(pfm.list)){
  print(i)
  names(pfm.list)[i]=pfm.list@listData[[i]]@name
}



#  Convert genome position to dna string and get motif position

In [ ]:
library(TFBSTools)
library(tidyr)
library(dplyr)
library(motifmatchr)
library(seqinr)
library(stringr)
library(JASPAR2020)

#0) Create motif matrix 
  # prepare motif from jaspar 
  pfm <- getMatrixSet(
    x = JASPAR2020,
    opts = list(collection = "CORE", tax_group = 'vertebrates', all_versions = FALSE)
  )
  # Create motif matrix with my peaks 
  motif.matrix <- as.data.frame( CreateMotifMatrix(
    features =StringToGRanges( DA_STERO_Female_Fasciculata$peaks[DA_STERO_Female_Fasciculata$avg_log2FC< - 0.25 & DA_STERO_Female_Fasciculata$p_val_adj<0.01],sep = c("-", "-")),  # GET RIGHT PEAKS
    pwm = pfm,
    genome = BSgenome.Mmusculus.UCSC.mm10,score = F
  ))


#1) Take Peaks on AR and create Dataframe
HOMER_FASC_F=as.data.frame(rownames(motif.matrix)[motif.matrix$MA0007.3!=0])
# TAKE DATAFRAME WITH FIRST ROW AS PEAKS POSITION , convert into dna strings 
Peakspos_to_DNA <- function(x) {
      names(x)[1]='chr'
      x=as.data.frame(str_split_fixed(x$chr,"-",3))
      names(x)[2]='START'
      names(x)[3]='END'# separate columns 
      for (i in 1:length(x$START)){
      x$ID[i]=paste("ID", i,sep = '')# add unique ID columns
      } 
      # convert seq pos to dna seq 
      Start=as.integer(x$START)
      End=as.integer(x$END)
      Chr=c(x$V1)
      for (i in 1:length(Chr)){
        x$DNA_STRINGs[i] <- as.character(Biostrings::getSeq(BSgenome.Mmusculus.UCSC.mm10, Chr[i], Start[i],End[i])) 
      }
      return(x)
}
# Run function
HOMER_FASC_F=Peakspos_to_DNA(HOMER_FASC_F)
names(HOMER_FASC_F)[1]='chr'

      
#3) get AR PWM from jaspar
pwm <- getMatrixSet(
  x = JASPAR2020,
  opts = list(collection = "UNVALIDATED", tax_group = 'vertebrates', all_versions = TRUE,matrixtype='PWM',species="Mus musculus")
)
    # TAKE only Ar
    Ar= pwm$MA0007.3
    
    # Get motif positions within peaks for example motifs in peaks 
    motif_ix <- matchMotifs(pwms = Ar, subject=HOMER_FASC_F$DNA_STRINGs, genome = "mm10",out = "positions")
  
    # View motif Ar as a dataframe 
    motif_ix_dataframe=as.data.frame(motif_ix)
    motif_ix_dataframe=motif_ix_dataframe[motif_ix_dataframe$strand=='+',]
    
    # merge  motif_ix_dataframe and HOMER_FASC
    HOMER_FASC_F$Column5=as.integer(rownames(HOMER_FASC_F))
    x=left_join(HOMER_FASC_F,motif_ix_dataframe, by = c('Column5'="group"))
    
    # remove NA 
    x= x %>% drop_na('score')
    
    # new columns =  subset DNA seq based on AR neighboors
    for (i in 1:length(x$chr)){
    print(i)
    x$p300[i]=x$end[i]+300
    x$m300[i]=x$start[i]-300
    x$subsetDNA[i]=substr( x$DNA_STRINGs[i],start = x$m300[i],stop = x$p300[i])
    # add new pos
    x$new_start[i]=(as.double(x$V2[i])+((x$start[i]))-300)
    x$new_end[i]=(as.double(x$V2[i])+((x$end[i]))+300)


    }
    
for (i in 1:length(x$chr)){
  print(i)
  x$AR_M300.start[i]=(as.double(x$START[i])+x$start[i])-300
  x$AR_M300.end[i]=(as.double(x$START[i])+x$start[i])-1
  x$AR_P300.start[i]=(as.double(x$END[i])+x$end[i])+1
  x$AR_P300.end[i]=(as.double(x$END[i])+x$end[i])+300
  Start=as.integer(x$AR_M300.start)
  End=as.integer(x$AR_M300.end)
  Chr=c(x$chr)
  x$DNA_LEFT_TO_AR[i]<- as.character(Biostrings::getSeq(BSgenome.Mmusculus.UCSC.mm10, Chr[i], Start[i],End[i])) 
  Start_right=as.integer(x$AR_P300.start)
  End_right=as.integer(x$AR_P300.end)
  x$DNA_right_TO_AR[i]<- as.character(Biostrings::getSeq(BSgenome.Mmusculus.UCSC.mm10, Chr[i], Start_right[i],End_right[i])) 


}

    write.fasta(sequences =as.list(x$DNA_LEFT_TO_AR),file.out = 'FASTA_MALE_FASC_LEFT_AR.fasta',names = x$X1) # Sequence of interest  
    write.fasta(sequences =as.list(x$DNA_LEFT_TO_AR),file.out = 'FASTA_MALE_FASC_Right_AR.fasta',names = x$X1) # Sequence of interest  


    
    
    # TAKE only NRC1/NRC2
    NR3C1= pwm$MA0113.2
    # Get motif positions within peaks for example motifs in peaks 
    motif_ix_NR3C1 <- as.data.frame(matchMotifs(pwms = NR3C1, subject= x$subsetDNA, genome = "mm10",out = "positions"))
    motif_ix_NR3C1=motif_ix_NR3C1[motif_ix_NR3C1$strand=='+',]
    y=left_join(x,motif_ix_NR3C1, by = c('Column5'="group"))
    y= y %>% drop_na('score.y')

    
    # compute average distance of NRC to AR 
    for (i in 1:length(y$chr)){
    AR_loc=y$start.x[i] + (y$width.x[i]/2)
    NRC_loc=y$start.y[i] + (y$width.y[i]/2)
    y$AR_NRC[i]=(AR_loc-NRC_loc)*sign(AR_loc - NRC_loc)
    print(i)
      
    }
    
     # TAKE only TATAA
    TATAA= pwm$
    # Get motif positions within peaks for example motifs in peaks 
    motif_ix_NR3C1 <- as.data.frame(matchMotifs(pwms = NR3C1, subject= x$subsetDNA, genome = "mm10",out = "positions"))
    motif_ix_NR3C1=motif_ix_NR3C1[motif_ix_NR3C1$strand=='+',]
    y=left_join(x,motif_ix_NR3C1, by = c('Column5'="group"))
    y= y %>% drop_na('score.y')

    
    # compute average distance of NRC to AR 
    for (i in 1:length(y$chr)){
    AR_loc=y$start.x[i] + (y$width.x[i]/2)
    NRC_loc=y$start.y[i] + (y$width.y[i]/2)
    y$AR_NRC[i]=(AR_loc-NRC_loc)*sign(AR_loc - NRC_loc)
    print(i)
      
    }


    # take Granges object from only_STERO
      motif.matrix <- as.data.frame(
      CreateMotifMatrix(
        features =StringToGRanges(x$subsetDNA,sep = c("-", "-")),  # GET RIGHT PEAKS
        pwm = pfm,
        genome = BSgenome.Mmusculus.UCSC.mm10,score = F
      ))

    # write as fasta file Sequence of interest
    for (i in 1:length(x$chr)){
    x$X1[i]=paste("ID", i,sep = '')  # add unique ID columns
   }
    write.fasta(sequences =as.list(x$subsetDNA),file.out = 'FASTA_MALE_FASC.fasta',names = x$X1) # Sequence of interest  

    
    
    # write as fasta file Background sequence 
    all_peaks=as.data.frame(rownames(only_STERO))
    for (i in 1:length(all_peaks$V1)){
    all_peaks$ID[i]=paste("ID", i,sep = '')  # add unique ID columns
   }
    write.fasta(sequences =as.list(all_peaks$DNA_STRINGs),file.out = 'all_peaks.fasta',names = all_peaks$ID) # Background sequence 

    
    
                                                       f= FindMarkers(only_STERO,
                                                        ident.1 ="Male_Fasciculata",
                                                        ident.2 = "Female_Fasciculata",
                                                        assay = 'peaks',
                                                        only.pos = F, test.use = 'LR',
                                                        min.pct = 0,logfc.threshold = 0,
                                                        latent.vars = 'nCount_peaks')



    
    # Get DNAstring 
DNA_STRING=DNAStringSet(HOMER_FASC_F$DNA_STRINGs, start=NA, end=NA, width=NA, use.names=TRUE)



#  1) selection of the peaks enrichment i
# 2) select AR 
# 3) identifie withuing peaks enrichment of toher TF
# 4) among those enrich TF their postiion related to AR binding SITE 
# peaks who have AR and whatevers CO factor ( glucorticoides ?) 

# 5 ) put


# scan AR motif in sequence 
motifScanHits(DNA_STRING, Ar, minScore = "80%", seqOrder = c(1:length(regionsSeq)))

write.table(HOMER_FASC_F$DNA_STRINGs,file = 'fasc.txt',append = F,quote = F,col.names = F,row.names = T)






# write as fasta file
write.fasta(sequences =as.list(x$subsetDNA),file.out = 'FASTA_MALE_FASC.fasta',names = HOMER_FASC_F$column1)


print(motif_ix)
library(motifmatchr)
library(GenomicRanges)

motifMatches(motif_ix) # Extract matches matrix from result

  # example peak set we're interested in
    peaks.use <- x$Combined
    motif.use <- motif.matrix$MA0006.1

    for ( i in 0:length(peaks.use)){
       sum(motif.matrix[peaks.use, colnames(motif.matrix[i])]) / length(x$Combined)
       sum(motif.matrix[peaks.use, colnames(motif.matrix[i])]) / length(x$Combined)
      
    }

    

Run HOMER with DA data frame  and create .txt file 

In [ ]:
library(stringr)
#HOMER peak files should have at minimum 5 columns (separated by TABs, additional columns will be ignored):

    #Column1: Unique Peak ID
    #Column2: chromosome
    #Column3: starting position
    #Column4: ending position
    #Column5: not used
    #Column6: Strand (+/- or 0/1, where 0="+", 1="-")

# try with DA FASCICULATA :

HOMER_FASC_F=as.data.frame(peaks_Male_FASC)
HOMER_FASC_F=as.data.frame(DA_STERO_Female_Fasciculata$peaks[DA_STERO_Female_Fasciculata$avg_log2FC< 0 & DA_STERO_Female_Fasciculata$p_val_adj<0.01]) # create new object wwith peak
names(HOMER_FASC_F)[1]='Column1'

HOMER_FASC_F=as.data.frame(str_split_fixed(HOMER_FASC_F$Column1,"-",3)) # separate columns 

HOMER_FASC_F$column1=rownames(HOMER_FASC_F)
HOMER_FASC_F$column1 <- paste("ID", HOMER_FASC_F$column1, sep="_")  # add unique ID columns

HOMER_FASC_F=HOMER_FASC_F[c(4,1,2,3)]  # reorder columns ( take index of the columns)

HOMER_FASC_F$columnn5=""

HOMER_FASC_F$Column6= '+'                # STRAND ?

#HOMER_FASC_F$V1 <- sub("^", ">", HOMER_FASC_F$V1 )


names(HOMER_FASC_F)[2]<-paste("Column2")
names(HOMER_FASC_F)[3]<-paste("Column3")
names(HOMER_FASC_F)[4]<-paste("Column4") 

# save 
write.table(HOMER_FASC_F,'/home/absta/Documents/DATA_RUN/HOMER_FASC_F.bed',  sep = " ", row.names = F ,col.names = F,quote = F)

### get HOMER TO BED FORMAT 
 # remove
  bedfile=df = subset(HOMER_FASC_F, select = -c(columnn5,Column6))
 # reorder 
  bedfile <- bedfile[, c(2,3,4,1)]



# STRAND ?

# get background

HOMER_FASC_M=as.data.frame(`DA_Female Fasciculata`$peaks[`DA_Female Fasciculata`$avg_log2FC<0]) # create new object wwith peak
names(HOMER_FASC_M)[1]='Column1'

HOMER_FASC_M=as.data.frame(str_split_fixed(HOMER_FASC_M$Column1,"-",3)) # separate columns 

HOMER_FASC_M$column1=rownames(HOMER_FASC_M)
HOMER_FASC_M$column1 <- paste("ID", HOMER_FASC_M$column1, sep="_")  # add unique ID columns

HOMER_FASC_M=HOMER_FASC_M[c(4,1,2,3)]  # reorder columns ( take index of the columns)

#HOMER_FASC_M$V1 <- sub("^", ">", HOMER_FASC_M$V1 )

HOMER_FASC_M$columnn5=""

HOMER_FASC_M$Column6= '+'            # STRAND ?


names(HOMER_FASC_M)[2]<-paste("Column2")
names(HOMER_FASC_M)[3]<-paste("Column3")
names(HOMER_FASC_M)[4]<-paste("Column4") 




# save 
write.table(HOMER_FASC_M,'/home/absta/Documents/DATA_RUN/HOMER_FASC_M.bed', sep=" ",row.names = F,col.names = F,quote = F)


# run homer : findMotifsGenome.pl HOMER_FASC_F.csv mm10 /home/absta/Documents/DATA_RUN/outPUTFIRSTHOMER -size 200 -bg HOMER_FASC_M.csv




Link  peak to genes 

In [ ]:
library(BSgenome.Mmusculus.UCSC.mm10)
library(Signac)
DefaultAssay(multiomics)='peaks'
# first compute the GC content for each peak
multiomics <- RegionStats(multiomics, genome = BSgenome.Mmusculus.UCSC.mm10)

`DE_Female Glomerulosa`$gene_names=rownames(`DE_Female Glomerulosa`)
`DE_Female Fasciculata`$gene_names=rownames(`DE_Female Fasciculata`)


# # link peaks to genes Glomerulosa
   #  compute gene to link with peaks :

            Female_Fasc_gene=`DE_Female Fasciculata`$gene_names[`DE_Female Fasciculata`$avg_log2FC> 0]   #Testing 300 genes and 178944 peaks Found gene coordinates for 278 genes
            Male_Fasc_gene=`DE_Female Fasciculata`$gene_names[`DE_Female Fasciculata`$avg_log2FC< 0]     #Testing 287 genes and 178944 peaks Found gene coordinates for 279 genes
            
            Female_Glo_gene=`DE_Female Glomerulosa`$gene_names[`DE_Female Glomerulosa`$avg_log2FC> 0]    #Testing 69 genes and 178944 peaks Found gene coordinates for 65 genes
            Male_Glo_gene=`DE_Female Glomerulosa`$gene_names[`DE_Female Glomerulosa`$avg_log2FC< 0]      #Testing 39 genes and 178944 peaks Found gene coordinates for 37 genes

x<- LinkPeaks(
  object = multiomics,
  peak.assay = "peaks",
  expression.assay = "SCT",
  genes.use = Male_Fasc_gene,
)

LINK_F_M=data.frame(Links(x))
saveRDS(LINK_F_F,file = 'LINK_F_F')

vec=Genes2peaks$peak[Genes2peaks$gene=='Akr1d1']
FeaturePlot(multiomics,features = vec,reduction = 'umap.rna.pca')


# i have run it on glomerulosa and fasciculata 

# Gene to peak correlation 

In [ ]:

peak_LINK_F_F=link_F_f$peak[link_F_f$score>0.2 & link_F_f$pvalue< 0.05]
peak_LINK_F_M=link_F_M$peak[link_F_M$score>0.2 & link_F_M$pvalue< 0.05]
peak_LINK_G_F=link_G_f$peak[link_G_f$score>0.2 & link_G_f$pvalue< 0.05]
peak_LINK_G_M=link_G_m$peak[link_G_m$score>0.2 & link_G_m$pvalue< 0.05]



      open.peaks <- AccessiblePeaks(multiomics, idents = c("Female Fasciculata"),assay = 'peaks') # find a set of peaks that are open in a subset of cells
      
      meta.feature <- GetAssayData(multiomics, assay = "peaks", slot = "meta.features")# match the overall GC content in the peak set
      peaks.matched <- MatchRegionStats(
        meta.feature = meta.feature[open.peaks, ],
        query.feature = meta.feature[peak_LINK_F_F, ],
        n = 50000
      )



enriched.motifs_male <- FindMotifs(
  object = multiomics,
  features = peak_LINK_F_M  ,           # take vector of peak
   background = peaks.matched
)

enriched.motifs_female <- FindMotifs(
  object = multiomics,
  features = peak_LINK_F_F  ,           # take vector of peak
   background = peaks.matched
)



View(enriched.motifs_female)
View(enriched.motifs_male)

enriched.motifs_male=enriched.motifs_male[enriched.motifs_male$fold.enrichment>2 &enriched.motifs_male$pvalue<0.05 ,]

enriched.motifs_female=enriched.motifs_female[enriched.motifs_female$fold.enrichment>2 &enriched.motifs_female$pvalue<0.05 ,]

female_up = enriched.motifs_female$motif[order(enriched.motifs_female$fold.enrichment, decreasing = TRUE)]
female_up=head(female_up)

male_up = enriched.motifs_male$motif[order(enriched.motifs_male$fold.enrichment, decreasing = TRUE)]
male_up=head(male_up)

DefaultAssay(multiomics) <- 'chromvar'

Male_motif=c('MA0007.3','MA0113.3','MA0727.1','MA1545.1')
Female_motif=c('MA1628.1','MA0102.4','MA0043.3')


  View(enrich_motif_Fem_TZONE)

 FeaturePlot(
  object = only_STERO,
  features = 'MA0043.3',
  min.cutoff = 'q10',
  max.cutoff = 'q100',
  pt.size = 2,reduction = 'wnn.umap'
)




# Plot motif heatmap

In [ ]:


View()


Male_motif=c('MA0007.3','MA0113.3')
Female_motif=c('MA1628.1','MA0102.4','MA0043.3')

minmaxthis = function(x) { 
  (x - min(x))/(max(x) - min(x))
}

trajFA=as.data.frame(only_STERO@reductions[["wnn.umap"]]@cell.embeddings)
trajFA$index=rownames(trajFA)


# plot figure 
TFsnames =c('MA1628.1','MA0102.4','MA0043.3')
genenames=c('Agtr1b')

plot_list = list() 

    for (x in genenames) {
      print(x)
      
      DefaultAssay(only_STERO) = "SCT_RUN_STERO"
      p10 = FeaturePlot(
        object = only_STERO,
        features = str_replace_all(x, fixed("_"), "-"),
        min.cutoff = 'q10',
        max.cutoff = 'q95',
        pt.size = 3)
      
      chromvardf = p10$data
      chromvardf$index = rownames(chromvardf)
      
      mergedFAchromvar = merge(chromvardf, trajFA, by = "index")
      
      
      TFactplot = ggplot(mergedFAchromvar, aes( x = wnnUMAP_1.x, y = wnnUMAP_2.x, 
                                                colour = minmaxthis(mergedFAchromvar[,5]))) + 
        geom_point(size = 1.5, alpha = 1) + 
        scale_color_gradient(low = "gray93",high = "red1",breaks = seq(0,1,0.2))+
        xlab("WNN1") + ylab("WNN2") +
        theme(panel.border=element_blank(), 
              axis.line = element_line(colour = 'black', size = 1), 
              axis.ticks = element_blank(),
              axis.text = element_blank(), 
              panel.grid.major = element_blank(),
              panel.grid.minor = element_blank(),
              panel.background = element_blank(),
              legend.key=element_blank(),
              legend.title=element_blank(),
              legend.text=element_text(size=12)) + 
        # guides(color = guide_legend(override.aes max.cutoff= list(size=5))) + 
        coord_capped_cart(bottom='right', left='none')  + 
      ggtitle(x)
      TFactplot
      
      
      print(TFactplot)
  
}



allmarkers=FindMarkers(only_STERO,ident.1 = c("Male_Fasciculata", "Male_Glomerulosa","Male_tZone"),ident.2 = c("Female_Glomerulosa", "Female_Fasciculata","Female_tZone"),logfc.threshold = 0,assay = 'SCT_RUN_STERO')

allmarkers=FindAllMarkers(only_STERO,logfc.threshold = 0,assay = 'SCT_RUN_STERO')



https://hbctraining.github.io/scRNA-seq_online/lessons/seurat_subclustering.html





In [ ]:
#keep only female in the data set
library(Seurat)
gene1traj <- WhichCells(alldata.int, expression = orig.ident == c("WTA", "WTC1"))
subtraj <- subset(alldata.int, cells=  gene1traj )
DefaultAssay(alldata.int) <- "CCA2"



#scanpy python
#Convert RData file in h5 file
library(SeuratDisk)
library(SeuratData)
SaveH5Seurat(subtraj, filename = "subtraj.h5Seurat")

only_STERO=SCTransform(only_STERO,variable.features.n =NULL,variable.features.rv.th =1.1,return.only.var.genes = FALSE,assay = 'RNA',new.assay.name = 'SCT_RUN_STERO',vars.to.regress = 'percent.mt')

# commande linux, modules installation, run just the first time

#pip install pandas --user pip install matplotlib --user pip install scanpy --user pip install igraph --user pip install fa2 --user pip install leidenalg --user pip install anndata --user


# monocle3 trajectory analysis atac

In [ ]:
library(monocle3)
library(monocle)
library(cicero)
library(Seurat)
library(Signac)
library(Signac)
library(Seurat)
library(SeuratWrappers)
library(monocle3)
library(Matrix)
library(ggplot2)
library(patchwork)
DefaultAssay(multiomics) <- "peaks"


## choose datasets
male <- multiomics[,  multiomics$identity %in% c('male')]
female <- multiomics[, multiomics$identity %in% c('female')]

all_STERO <- multiomics[,  multiomics$cell_types %in% c("Male Fasciculata Replicate 1","Male Fasciculata Replicate 2","Female Glomerulosa","Female Fasciculate Replicate 1" ,"Male Glomerulosa","Capsular Cells","Immune Cells","Fetal adrenocortical Cells ","X zone ","Female Fasiculata Replicate 2")]


male_STERO <- multiomics[,  multiomics$identity %in% c("male")& multiomics$cell_types %in% c("Male Fasciculata Replicate 1","Male Fasciculata Replicate 2","Female Glomerulosa","Female Fasciculate Replicate 1" ,"Male Glomerulosa","Capsular Cells","Immune Cells","Fetal adrenocortical Cells ","X zone ","Female Fasiculata Replicate 2")]

female_STERO <- multiomics[, multiomics$identity %in% c("female")& multiomics$cell_types %in% c("Male Fasciculata Replicate 1","Male Fasciculata Replicate 2","Female Glomerulosa","Female Fasciculate Replicate 1" ,"Male Glomerulosa","Capsular Cells","Immune Cells","Fetal adrenocortical Cells ","X zone ","Female Fasiculata Replicate 2")]

### take reduction 

female_STERO@reductions$UMAP=female_STERO@reductions[["wnn.umap"]]
male_STERO@reductions$UMAP=male_STERO@reductions[["wnn.umap"]]
all_STERO@reductions$UMAP=all_STERO@reductions[["wnn.umap"]]



# root for trajectory 
all_root <- all_STERO[, all_STERO$cell_types %in% c("Female Glomerulosa","Male Glomerulosa")]
vector_root_all=colnames(all_root)


female_glomerulosa_root <- female_STERO[, female_STERO$cell_types %in% c("Female Glomerulosa","Male Glomerulosa")]
male_glomerulosa_root <- male_STERO[, male_STERO$cell_types %in% c("Male Glomerulosa","Female Glomerulosa")]
vector_root_M=colnames(male_glomerulosa_root)
vector_root_F=colnames(female_glomerulosa_root)


# Building trajectories with Monocle 3



all.cds <- as.cell_data_set(all_STERO)
all.cds <- cluster_cells(cds = all.cds, reduction_method = "UMAP",resolution = 30,max.overlaprs=10)
all.cds <- learn_graph(all.cds, use_partition = TRUE)

female.cds <- as.cell_data_set(female_STERO)
female.cds <- cluster_cells(cds = female.cds, reduction_method = "UMAP")
female.cds <- learn_graph(female.cds, use_partition = TRUE)

male.cds <- as.cell_data_set(male_STERO)
male.cds <- cluster_cells(cds = male.cds, reduction_method = "UMAP")
male.cds <- learn_graph(male.cds, use_partition = TRUE)

# order cells
all.cds <- order_cells(all.cds, reduction_method = "UMAP" ,root_cells =vector_root_all )


male.cds <- order_cells(male.cds, reduction_method = "UMAP" ,root_cells =vector_root_M )
female.cds <- order_cells(female.cds, reduction_method = "UMAP",root_cells = vector_root_F )



x=plot_cells(
  cds = male.cds,
  color_cells_by = "pseudotime",
  show_trajectory_graph = TRUE,cell_size = 2,trajectory_graph_segment_size = 2,graph_label_size = 2,label_cell_groups = F,label_branch_points = T,)+
  plot_cells(
  cds = female.cds,
  color_cells_by = "pseudotime",
  show_trajectory_graph = TRUE,cell_size = 2,trajectory_graph_segment_size = 2,graph_label_size = 2,label_cell_groups = F,label_branch_points = T)



# # a helper function to identify the root principal points:
get_earliest_principal_node <- function(cds){
  cell_ids <- which(colData(cds)[, "cell_types"] %in% c("Female Glomerulosa","Male Glomerulosa"))
  
  closest_vertex <-
  cds@principal_graph_aux[["UMAP"]]$pr_graph_cell_proj_closest_vertex
  closest_vertex <- as.matrix(closest_vertex[colnames(cds), ])
  root_pr_nodes <-
  igraph::V(principal_graph(cds)[["UMAP"]])$name[as.numeric(names
  (which.max(table(closest_vertex[cell_ids,]))))]
  
  root_pr_nodes
}
cds_male <- order_cells(male.cds, root_pr_nodes=get_earliest_principal_node(male.cds))
cds_female <- order_cells(female.cds, root_pr_nodes=get_earliest_principal_node(female.cds))
all.cds <- order_cells(all.cds, root_pr_nodes=get_earliest_principal_node(all.cds))


# add  umap pseudotime value in seurat object 


multiomics <- AddMetaData(
  object = multiomics,
  metadata = all.cds@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "all_sexe_trajectory_umap"
)


multiomics <- AddMetaData(
  object = multiomics,
  metadata = male.cds@principal_graph_aux@listData$UMAP$pseudotime,
  col.name = "male"
)

FeaturePlot(multiomics, c("female", "male"), pt.size = 1,reduction = 'wnn.umap',label = T) & scale_color_viridis_c()



In [ ]:
library(dplyr)


multiomics@meta.data$index=rownames(multiomics@meta.data)
TRAJECTORY_ALL_STERO = select(multiomics@meta.data, all_sexe_trajectory_umap, barcode)
TRAJECTORY_ALL_STERO=na.omit(object =TRAJECTORY_ALL_STERO)




https://mojaveazure.github.io/seurat-disk/articles/convert-anndata.html

# convert Seurat object to scanpy object 

In [ ]:
library(SeuratDisk)
multiomics_nomotif=multiomics
multiomics_nomotif@assays[["peaks"]]@motifs =NULL # error if features in motif slot 
SaveH5Seurat(multiomics_nomotif, filename = "multiomics_convert.h5Seurat")
Convert("multiomics_convert.h5Seurat", dest = "h5ad")
pbmc_ad <- ad$read_h5ad("/C:/Users/User/Downloads")


In [ ]:
%%python
import scanpy  # pip install in  R terminal
import scanpy as sc
import numpy as np

import pandas as pd
from matplotlib import rcParams
import os


import anndata2ri
adata = sc.read_h5ad("subtraj.h5ad")
scanpy_peaks=scanpy.read_h5ad('C:/Users/User/Downloads/multiomics_convert.h5ad')



# sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=4, n_pcs=20)




https://smorabit.github.io/tutorials/8_velocyto/#:~:text=Briefly%2C%20RNA%20velocity%20analysis%20allows,give%20cell%20population%20of%20interest.

# RNA velovity 

In [ ]:

library(Matrix)


#Step -1: Convert data from Seurat to Python / anndata

# save metadata table:
multiomics$barcode <- colnames(multiomics)
multiomics$UMAP_1 <- multiomics@reductions$wnn.umap@cell.embeddings[,1]
multiomics$UMAP_2 <- multiomics@reductions$wnn.umap@cell.embeddings[,2]
write.csv(multiomics@meta.data, file='metadata.csv', quote=F, row.names=F)
# write expression counts matrix
counts_matrix <- GetAssayData(multiomics, assay='peaks', slot='counts')
writeMM(counts_matrix, file= ('counts.mtx'))

# write dimesnionality reduction matrix, in this example case pca matrix
  write.csv(multiomics@reductions$lsi@cell.embeddings, file='lsi.csv', quote=F, row.names=F)

# write gene names
write.table(
  data.frame('peaks'=rownames(counts_matrix)),file='peaks_names.csv',
  quote=F,row.names=F,col.names=F
)


# PLOT FIG5 

In [ ]:
library(Seurat)
library(tidyverse)
library(BSgenome.Mmusculus.UCSC.mm10)
library(EnsDb.Mmusculus.v79)
library(Signac)
library(cowplot)
library(patchwork)
library(forcats)
    # feature matrix 
peaks=GetAssayData(object = multiomics, slot = "counts",assay = 'peaks')

ext = c(3)

# extract gene coordinates from Ensembl, and ensure name formatting is consistent with Seurat object 
gene.coords <- genes(EnsDb.Mmusculus.v79, filter = ~ gene_biotype == "protein_coding")
ucsc.levels <- str_replace(string=paste("chr",seqlevels(gene.coords),sep=""), pattern="chrMT", replacement="chrM")
seqlevels(gene.coords) <- ucsc.levels
genebody.coords <- keepStandardChromosomes(gene.coords, pruning.mode = 'coarse')

genebodyandpromoter.coords <- Extend(x = gene.coords, upstream = 1000*ext, downstream = 1000*ext)
gene.key <- genebodyandpromoter.coords$gene_name
names(gene.key) <- GRangesToString(grange = genebodyandpromoter.coords)

rownames(peaks) <- gene.key[rownames(peaks)]

peaks = peaks[unique(rownames(peaks)),]




  multiomics[[paste0("RNA_geneprom_",ext,"kb")]] <- CreateAssayObject(counts = peaks)
  # test = BinarizeCounts(object = test, assay = c("RNA"))
  
  test <- NormalizeData(
    object = test,
    assay = paste0("RNA_geneprom_",ext,"kb"),
    normalization.method = 'LogNormalize',
    scale.factor = median(test@meta.data[,paste0("nFeature_RNA_geneprom_",ext,"kb")])
  )





#top10 markers genes through all clusters 
all_markers_genes_peaks=FindAllMarkers(multiomics,assay = 'SCT')
x=all_markers_genes[all_markers_genes$p_val_adj<0.05,]

gen_marker_table <- function(x){
all_markers_genes[all_markers_genes$cluster == x, ] %>%
head(n=3)}
# view
vec=levels(multiomics)
top10_markers <- map_dfr(vec, gen_marker_table)
top10_markers=top10_markers[!duplicated(top10_markers$gene),]
top_genes=as.vector(top10_markers$gene)


dotx=DotPlot(multiomics,assay= 'SCT',features = top_genes)+ RotatedAxis() +  scale_colour_gradient2(low = "darkred", mid = "white",high = "darkgreen",midpoint = 0)

# Ploting motif along cluster  Fig6

In [ ]:
# Main plot
  minmaxthis = function(x) { 
    (x - min(x))/(max(x) - min(x))
  }
  
  data.dot = dotx$data
  
  
  scaled_data <- 
    data.dot %>%
    group_by(features.plot) %>%
    mutate(minmaxscale = minmaxthis(avg.exp))
  
  scaled_data$avg.exp2 = ( scaled_data$avg.exp - min(scaled_data$avg.exp) )    / 
    (max(scaled_data$avg.exp)- min(scaled_data$avg.exp))
  
  
  pmain <- ggplot(scaled_data)  + 
    geom_point(aes(x = features.plot, y = id, color = minmaxscale, size = pct.exp), stat = "identity") + 
    scale_size(range = c(0, 4.25), breaks  = seq(0,100,20), labels  = paste0(seq(0,100,20),"%"))  + 
    scale_color_gradient(low = "white",high = "#0f4d91",breaks = seq(0,1,0.2))  +
    guides(color = guide_colourbar(title = "Accessibility", barwidth = 1, barheight = 10),
           size = guide_legend(title = "Cells (%)",reverse=TRUE)) + 
    xlab("") + ylab("") + 
    
    theme(panel.border=element_blank(), 
          axis.line = element_line(colour = 'black', size = 1), 
          axis.ticks = element_blank(),
          axis.text = element_text(size=12, colour = "black"), 
          axis.text.x = element_text(angle = 90, hjust = 1),
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          legend.key=element_blank(),
          # legend.title=element_blank(),
          legend.text=element_text(size=12))
  

  
 
  xdens <- ggplot() +
    geom_bar(data = scaled_data, aes(x = features.plot, y = avg.exp2),stat = "summary",
             alpha = 1, size = 0.2, fill = "black") + 
    scale_y_continuous(breaks = seq(0,1,0.5), labels = seq(0,1,0.5), limits = c(0,1)) + 
    
    # theme_bw() + 
    theme(panel.border=element_blank(), 
          axis.line = element_line(colour = 'black', size = 1), 
          axis.ticks = element_blank(),
          axis.line.x = element_blank(),
          axis.text = element_text(size=12, colour = "black"), 
          panel.grid.major = element_blank(),
          panel.grid.minor = element_blank(),
          panel.background = element_blank(),
          legend.key=element_blank(),
          # legend.title=element_blank(),
          legend.text=element_text(size=12),
          axis.text.x = element_blank(),
          axis.ticks.x = element_blank(),
          axis.title.x  = element_blank()) + 
    ylab("Average \n accessibility")



  dotx1 = xdens + pmain + plot_layout(ncol = 1, heights = c(1,3))


  # Main plot

  minmaxthis = function(x) { 
    (x - min(x))/(max(x) - min(x))
  }





In [ ]:
dfATAC = only_STERO@meta.data
dfATAC$index = rownames(dfATAC)

trajFA=as.data.frame(only_STERO@reductions[["wnn.umap"]]@cell.embeddings)
trajFA$index=rownames(trajFA)


mergeFAATAC = merge(trajFA, dfATAC, by = "index")

# 
plot3C = ggplot(mergeFAATAC, aes( x = wnnUMAP_1, y = wnnUMAP_2, colour = new_ident)) + 
  geom_point(size = 1, alpha = 1) + 
  scale_colour_manual(values = c( '#3CB44B', '#FFE119', '#4363D8', '#F58231', '#911EB4', '#46F0F0','#E6194B')) +
  xlab("FA1") + ylab("FA2") +
  theme(panel.border=element_blank(), 
        axis.line = element_line(colour = 'black', size = 1), 
        axis.ticks = element_blank(),
        axis.text = element_blank(), 
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.background = element_blank(),
        legend.key=element_blank(),
        legend.title=element_blank(),
        legend.text=element_text(size=12)) + 
  guides(color = guide_legend(override.aes = list(size=5))) + 
  coord_capped_cart(bottom='right', left='none') 

plot3D = ggplot(mergeFAATAC, aes( x = wnnUMAP_1, y = wnnUMAP_2, colour = minmaxthis(nCount_peaks))) + 
  geom_point(size = 1, alpha = 1) + 
   scale_color_viridis(breaks = seq(0,1,0.2)) + 
   # scale_color_gradient(low = "gray93",high = "red1",breaks = seq(0,1,0.2), limits = c(0.4,1)) +
  xlab("FA1") + ylab("FA2") +
  theme(panel.border=element_blank(), 
        axis.line = element_line(colour = 'black', size = 1), 
        axis.ticks = element_blank(),
        axis.text = element_blank(), 
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        panel.background = element_blank(),
        legend.key=element_blank(),
        legend.title=element_blank(),
        legend.text=element_text(size=12)) + 
  # guides(color = guide_legend(override.aes = list(size=5))) + 
  coord_capped_cart(bottom='right', left='none') 

  print(plot3C)
  print(plot3D)



